<a href="https://colab.research.google.com/github/BelloBaus/item-list/blob/master/Spectralyzer_55.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# @title Header & Globals
from IPython import get_ipython
from IPython.display import display
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from google.colab import drive, files
import os
from google.colab import output
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
from IPython.display import display, clear_output
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import savgol_filter
from scipy.signal import find_peaks as scipy_find_peaks # Importiere find_peaks mit Alias
from scipy.interpolate import interp1d
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, Flatten, Dense
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Lambda
from scipy.spatial.distance import cosine
from tqdm.keras import TqdmCallback
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import re
import json
import requests
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.optimize import curve_fit
from scipy.special import wofz
from IPython import get_ipython
from IPython.display import display
from scipy.interpolate import interp1d
import shutil
import ast
import numpy as np
from sklearn.cluster import KMeans
import random
from scipy.integrate import trapezoid
from scipy.signal import savgol_filter
from scipy.signal import find_peaks as scipy_find_peaks # Importiere find_peaks mit Alias
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.palettes import Category20, Viridis256
from bokeh.colors import RGB
from google.colab import output
from tensorflow.keras.callbacks import ModelCheckpoint # Import ModelCheckpoint
import pickle

# Cache-Verzeichnis
cache_dir = "nist_asd_cache"
nist_peak_data_cache = {}
#shutil.rmtree(cache_dir, ignore_errors=True)
os.makedirs(cache_dir, exist_ok=True)
# Globale Variable für die Bilder und den aktuellen Index
bilder = []
aktueller_index = 0

# Globale Variable für das Embedding-Modell
embedding_model = None
best_embedding_model_path = '/content/drive/MyDrive/Gemini/best_embedding_model.keras'
# Parameter für das Embedding-Modell
EMBEDDING_DIMENSION = 64 # Beispielwert, kann angepasst werden
TARGET_SIZE = 50         # Muss mit extract_and_prepare_peak_data übereinstimmen
all_elements_for_nn_alt = ['Al', 'Ar', 'B', 'Ba', 'Be', 'C', 'Ca', 'Cl', 'Co', 'Cr', 'Cu', 'F', 'Fe', 'Ga', 'Ge', 'H', 'He', 'Hg', 'In', 'K', 'Li', 'Mg', 'Mn', 'Mo', 'N', 'Na', 'Ne', 'Ni', 'O', 'P', 'Pb', 'S', 'Sb', 'Si', 'Sn', 'Sr', 'Ti', 'V', 'Zn']
all_elements_for_nn = [
    'H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne',
    'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 'Ca',
    'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
    'Ga', 'Ge', 'As', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr',
    'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn',
    'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd',
    'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb',
    'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg',
    'Tl', 'Pb', 'Bi', 'Fr', 'Ra', 'Ac'
]



# Globale Variablen für die Bilder, den aktuellen Index und die NIST-Daten
bilder = []
aktueller_index = 0
nist_data = {}  # Dictionary zum Speichern der NIST-Daten
file_path = '' # Variable für den Dateipfad
identified_peaks = {} # Variable für die identifizierten Peaks
filtered_peaks = {} # Variable für die gefilterten Peaks
nn_outputs_visible = False
# Define lower_wavelength and upper_wavelength globally (or within nist_button_clicked)
lower_wavelength = 200  # Example value, adjust as needed
upper_wavelength = 900
peak_details_interactive_called = False
nist_details_visible = False
current_peak_index = 0
identified_matches = []
current_match_index = 0
nist_sum_fig = None
nist_sum_ax = None
nist_stage_fig = None
nist_stage_ax = None
prepared_peaks_info = []
nist_peak_embeddings = []
validated_matches = [] # NEUE GLOBALE VARIABLE für validierte Matches

In [9]:








# @title Widgets

output.enable_custom_widget_manager()
# Widgets erstellen
file_button = widgets.Button(description="Cloud Data")
nist_button = widgets.Button(description="NIST-Data & Elements", disabled=True)
peak_detection_button = widgets.Button(description="Detect Peaks", disabled=True)
clear_button = widgets.Button(description="Clear Output")
nist_parameter_button = widgets.Button(description="NIST-Data & Elements")
peak_details_button = widgets.Button(description="Peak Details", disabled=True)
element_identification_button = widgets.Button(description="Identify Elements", disabled=True)
load_all_and_train_button = widgets.Button(description="Load NN", disabled=True)
train_nn_button = widgets.Button(description="Train NN")
train_elements_selection = widgets.RadioButtons(
    options=['All Elements', 'Selected/Registered'],
    description='Elements for Training:',
    disabled=False
)
# Output-Widget für das Details-Fenster erstellen
details_output = widgets.Output()
neur_output = widgets.Output(layout=widgets.Layout(height='400px', width='50%', overflow_y='auto'))
neur_img_output = widgets.Output(layout=widgets.Layout(height='400px', width='50%', overflow_y='auto'))
combined_peak_output = widgets.Output(layout=widgets.Layout(height='400px', width='50%', overflow_y='auto'))
drive_path = '/content/drive/MyDrive/Gemini/best_embedding_model.keras' # Füge die Endung .keras hinzu
nist_embedding_file_path = '/content/drive/MyDrive/Gemini/nist_embeddings.npy'

drive_selection_widget = widgets.RadioButtons(
    options=['Google Drive', 'Lokale Datei'],
    description='Datasource',
    disabled=False
)
file_path_widget = widgets.Text(
    value='/content/drive/MyDrive/Gemini/Spectralyzer/Spektren/spectrum.txt',
    description='Path',
    disabled=False,
    visible=False
)
file_upload_widget = widgets.FileUpload(
    accept='.txt',
    multiple=False,
    description='Local Data',
    visible=False
)

# Widgets für die Parameter
epochs_slider = widgets.IntSlider(
    value=100,
    min=10,
    max=500,
    step=10,
    description='Epochs:',
    disabled=False,
    continuous_update=True, # Changed to True
    orientation='horizontal',
    readout=False, # Readout entfernen, da Textfeld verwendet wird
    readout_format='d',
)

batch_size_slider = widgets.IntSlider(
    value=32,
    min=8,
    max=128,
    step=8,
    description='Batch Size:',
    disabled=False,
    continuous_update=True, # Changed to True
    orientation='horizontal',
    readout=False, # Readout entfernen, da Textfeld verwendet wird
    readout_format='d',
)

learning_rate_slider = widgets.FloatSlider(
    value=0.001,
    min=0.0001,
    max=0.01,
    step=0.0001,
    description='Learnrate:',
    disabled=False,
    continuous_update=True, # Changed to True
    orientation='horizontal',
    readout=False, # Readout entfernen, da Textfeld verwendet wird
    readout_format='.4f',
)

tolerance_slider = widgets.FloatSlider(
    value=0.5,
    min=0.1,
    max=2.0,
    step=0.1,
    description='Peak-Tolerance:',
    disabled=False,
    continuous_update=True, # Already True, ensuring it stays True
    orientation='horizontal',
    readout=False,
    readout_format='.1f',
)

temp_slider = widgets.FloatSlider(
    value=0.4,
    min=0.1,
    max=1.0,
    step=0.1,
    description='Temperature (eV):',
    disabled=False,
    continuous_update=True, # Already True, ensuring it stays True
    orientation='horizontal',
    readout=False,
    readout_format='.1f',
)

resolution_slider = widgets.IntSlider(
    value=250,
    min=100,
    max=1000,
    step=10,
    description='Resolution:',
    disabled=False,
    continuous_update=True, # Already True, ensuring it stays True
    orientation='horizontal',
    readout=False,
    readout_format='d',
)

# Slider für s_value
s_value_slider = widgets.FloatSlider(
    value=1.0,  # Startwert
    min=0.1,  # Minimalwert
    max=10.0, # Maximalwert
    step=0.1, # Schrittweite
    description='S-Value:',
    disabled=False,
    orientation='horizontal',
    readout=False,
    continuous_update=True, # Already True, ensuring it stays True
    readout_format='.1f',
)

# Slider für min_distance
min_distance_slider = widgets.FloatSlider(  # Ändere den Typ zu FloatSlider
    value=1.0,  # Startwert
    min=0.1,  # Minimalwert
    max=10.0, # Maximalwert
    step=0.1, # Schrittweite
    description='Min.Distance (nm):',
    disabled=False,
    readout=False,
    continuous_update=True, # Already True, ensuring it stays True
    readout_format='.1f',  # Formatierung für Float-Werte
)


# Slider für noise_std
noise_std_slider = widgets.FloatSlider(
    value=0,  # Set initial value to 0
    min=0,  # Minimalwert: 0% des berechneten Werts
    max=50000,  # Maximalwert: 500% des berechneten Werts
    step=10,  # Schrittweite: 10% des berechneten Werts
    description='Noise:',
    disabled=False,
    orientation='horizontal',
    readout=False,
    continuous_update=True, # Already True, ensuring it stays True
    readout_format='.1f',
)

# Slider für den Suchbereich um den Peak
search_range_slider = widgets.IntSlider(
    value=5,  # Startwert
    min=1,  # Minimalwert
    max=20,  # Maximalwert auf 20 geändert
    step=1,  # Schrittweite
    description='Area (nm):',
    disabled=False,
    orientation='horizontal',
    readout=False,  # readout auf False setzen
    continuous_update=True, # Already True, ensuring it stays True
    readout_format='d',
)

# NEUE SLIDER FÜR NIST URL PARAMETER
low_w_slider = widgets.FloatSlider(
    value=200.0,
    min=100.0,
    max=1000.0,
    step=1.0,
    description='Low W (nm):',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=False,
    readout_format='.1f',
)

upp_w_slider = widgets.FloatSlider(
    value=900.0,
    min=100.0,
    max=1000.0,
    step=1.0,
    description='Upp W (nm):',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=False,
    readout_format='.1f',
)

# Angepasster eden_slider für Exponenten
eden_slider = widgets.IntSlider( # Change to IntSlider for exponent
    value=17, # Default exponent for 1e17
    min=15, # Minimum exponent
    max=19, # Maximum exponent
    step=1, # Step by 1 for exponents
    description='Eden (1eX cm-3):', # Updated description
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=False,
    readout_format='d', # Format as integer
)

maxcharge_slider = widgets.IntSlider(
    value=2,
    min=0,
    max=10,
    step=1,
    description='Max Charge:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=False,
    readout_format='d',
)

min_rel_int_slider = widgets.FloatSlider(
    value=0.01,
    min=0.001,
    max=1.0,
    step=0.001,
    description='Min Rel Int:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=False,
    readout_format='.3f',
)

int_scale_slider = widgets.IntSlider(
    value=1,
    min=0,
    max=5,
    step=1,
    description='Int Scale:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=False,
    readout_format='d',
)


# Textfeld für den Suchbereich
search_range_text = widgets.BoundedIntText(  # BoundedIntText verwenden, da der Slider ein IntSlider ist
    value=search_range_slider.value,
    min=search_range_slider.min,
    max=search_range_slider.max, # Maximalwert auf 20 geändert
    step=search_range_slider.step,
    disabled=False,
    layout=widgets.Layout(width='auto')
)

# Textfeld für noise_std
noise_std_text = widgets.BoundedFloatText(
    value=noise_std_slider.value,
    min=noise_std_slider.min,
    max=noise_std_slider.max,
    step=noise_std_slider.step,
    disabled=False,
    layout=widgets.Layout(width='auto')
)

# Textfeld für min_distance
min_distance_text = widgets.BoundedFloatText(
    value=min_distance_slider.value,
    min=min_distance_slider.min,
    max=min_distance_slider.max,
    step=min_distance_slider.step,
    disabled=False,
    layout=widgets.Layout(width='auto')
)

# Textfeld für s_value
s_value_text = widgets.BoundedFloatText(
    value=s_value_slider.value,
    min=s_value_slider.min,
    max=s_value_slider.max,
    step=s_value_slider.step,
    disabled=False,
    layout=widgets.Layout(width='auto')
)

# Textfeld für Toleranz
tolerance_text = widgets.BoundedFloatText(
       value=tolerance_slider.value,
       min=tolerance_slider.min,
       max=tolerance_slider.max,
       step=tolerance_slider.step,
       disabled=False,
       layout=widgets.Layout(width='auto') # Layout anpassen
   )

   # Textfeld für Temperatur
temp_text = widgets.BoundedFloatText(
       value=temp_slider.value,
       min=temp_slider.min,
       max=temp_slider.max,
       step=temp_slider.step,
       disabled=False,
       layout=widgets.Layout(width='auto') # Layout anpassen
   )

   # Textfeld für Auflösung
resolution_text = widgets.BoundedIntText(  # Verwenden Sie BoundedIntText für den IntSlider
       value=resolution_slider.value,
       min=resolution_slider.min,
       max=resolution_slider.max,
       step=resolution_slider.step,
       disabled=False,
       layout=widgets.Layout(width='auto') # Layout anpassen
   )

# NEUE TEXTFELDER FÜR NIST URL PARAMETER
low_w_text = widgets.BoundedFloatText(
    value=low_w_slider.value,
    min=low_w_slider.min,
    max=low_w_slider.max,
    step=low_w_slider.step,
    disabled=False,
    layout=widgets.Layout(width='auto')
)

upp_w_text = widgets.BoundedFloatText(
    value=upp_w_slider.value,
    min=upp_w_slider.min,
    max=upp_w_slider.max,
    step=upp_w_slider.step,
    disabled=False,
    layout=widgets.Layout(width='auto')
)

# Angepasstes eden_text für Exponenten
eden_text = widgets.BoundedIntText( # Change to BoundedIntText
    value=eden_slider.value,
    min=eden_slider.min,
    max=eden_slider.max,
    step=eden_slider.step,
    disabled=False,
    layout=widgets.Layout(width='auto')
)

maxcharge_text = widgets.BoundedIntText(
    value=maxcharge_slider.value,
    min=maxcharge_slider.min,
    max=maxcharge_slider.max,
    step=maxcharge_slider.step,
    disabled=False,
    layout=widgets.Layout(width='auto')
)

min_rel_int_text = widgets.BoundedFloatText(
    value=min_rel_int_slider.value,
    min=min_rel_int_slider.min,
    max=min_rel_int_slider.max,
    step=min_rel_int_slider.step,
    disabled=False,
    layout=widgets.Layout(width='auto')
)

int_scale_text = widgets.BoundedIntText(
    value=int_scale_slider.value,
    min=int_scale_slider.min,
    max=int_scale_slider.max,
    step=int_scale_slider.step,
    disabled=False,
    layout=widgets.Layout(width='auto')
)


# Textfelder für Trainingsparameter
epochs_text = widgets.BoundedIntText(
       value=epochs_slider.value,
       min=epochs_slider.min,
       max=epochs_slider.max,
       step=epochs_slider.step,
       disabled=False,
       layout=widgets.Layout(width='auto') # Layout anpassen
)

batch_size_text = widgets.BoundedIntText(
       value=batch_size_slider.value,
       min=batch_size_slider.min,
       max=batch_size_slider.max,
       step=batch_size_slider.step,
       disabled=False,
       layout=widgets.Layout(width='auto') # Layout anpassen
)

learning_rate_text = widgets.BoundedFloatText(
       value=learning_rate_slider.value,
       min=learning_rate_slider.min,
       max=learning_rate_slider.max,
       step=learning_rate_slider.step,
       disabled=False,
       layout=widgets.Layout(width='auto') # Layout anpassen
)


# Widgets für die Elemente
elements_widget = widgets.SelectMultiple(  # Ändern Sie von Select zu SelectMultiple
       options=[None] + ['Al', 'Ar', 'B', 'Ba', 'Be', 'C', 'Ca', 'Cl', 'Co', 'Cr', 'Cu', 'F', 'Fe', 'Ga', 'Ge', 'H', 'He', 'Hg', 'In', 'K', 'Li', 'Mg', 'Mn', 'Mo', 'N', 'Na', 'Ne', 'Ni', 'O', 'P', 'Pb', 'S', 'Sb', 'Se', 'Si', 'Sn', 'Sr', 'Ti', 'V', 'Zn'],
       value=[None],  # Setzen Sie value auf eine Liste mit None, um keine Vorauswahl zu treffen
       rows=10,
       description='Elements:',
       disabled=False
   )

additional_elements_widget = widgets.Text(
    value='',
    placeholder='Elements (comma-separated)',
    description='Elements (NIST-DB):',
    disabled=False
)

apply_button = widgets.Button(description="Anwenden")
output_spectrum = widgets.Output(layout=widgets.Layout(height='800px', width='50%', overflow_y='auto'))  # height und overflow_y hinzufügen
output_nist = widgets.Output(layout=widgets.Layout(height='800px', width='50%', overflow_y='auto'))  # height und overflow_y hinzufügen
output_peak_details = widgets.Output(layout=widgets.Layout(height='800px', width='50%', overflow_y='auto'))  # height und overflow_y hinzufügen
nist_details_output = widgets.Output(layout=widgets.Layout(height='800px', width='50%', overflow_y='auto'))
nist_stages_output = widgets.Output(layout=widgets.Layout(height='800px', width='50%', overflow_y='auto'))
#position_display = widgets.Output(layout=widgets.Layout(width='50%', height='400px'), visible=False)
nist_peak_detection_button = widgets.Button(description="NIST-Peakerkennung", disabled=True)

nist_details_button = widgets.Button(description="NIST Details", disabled=True)  # Initial deaktiviert
next_nist_button = widgets.Button(description="Next NIST", disabled=True)
back_nist_button = widgets.Button(description="Back NIST", disabled=True)
peak_number_nist_input = widgets.BoundedIntText(
    value=0,
    min=0,
    max=0,
    step=1,
    disabled=False,
    layout=widgets.Layout(width='50px')
)

# Add jslink calls here, after all widgets are defined
# Moved jslink calls to cell_id: huV7dEJygnTu
# widgets.jslink((noise_std_slider, 'value'), (noise_std_text, 'value'));
# widgets.jslink((min_distance_slider, 'value'), (min_distance_text, 'value'));
# widgets.jslink((s_value_slider, 'value'), (s_value_text, 'value'));
# widgets.jslink((tolerance_slider, 'value'), (tolerance_text, 'value'));
# widgets.jslink((temp_slider, 'value'), (temp_text, 'value'));
# widgets.jslink((resolution_slider, 'value'), (resolution_text, 'value'));
# widgets.jslink((search_range_slider, 'value'), (search_range_text, 'value'));
# widgets.jslink((epochs_slider, 'value'), (epochs_text, 'value'));
# widgets.jslink((batch_size_slider, 'value'), (batch_size_text, 'value'));
# widgets.jslink((learning_rate_slider, 'value'), (learning_rate_text, 'value'));

# NEUER BUTTON HINZUFÜGEN (am Ende des Blocks)
mark_match_button = widgets.Button(description="Mark Match", disabled=True) # Initial deaktiviert
# NEUER BUTTON HINZUFÜGEN für Export
export_matches_button = widgets.Button(description="Export Matches", disabled=True) # Initial deaktiviert

# NEUES RadioButtons Widget für die Modellauswahl
nn_model_selection_widget = widgets.RadioButtons(
    options=['Raw NIST NN', 'User fused NN'],
    description='Select NN Model:',
    disabled=False,
    value='User fused NN' # Default selection
)

In [10]:
# @title Plotting, Reading & Loading

def display_current_match():
    """Zeigt die Details im Text-Output und den Plot im Bild-Output des aktuellen Matches an."""
    global identified_matches, current_match_index, neur_output, neur_img_output, nist_data, identified_peaks, aktueller_index, search_range_slider, wavelengths, spectrum, combined_peak_output, nist_details_output, low_w_slider, upp_w_slider, temp_slider, resolution_slider, eden_slider, maxcharge_slider, min_rel_int_slider, int_scale_slider, validated_matches # Füge NIST parameter sliders und validated_matches hinzu

    if not identified_matches or not (0 <= current_match_index < len(identified_matches)):
        with neur_output:
            clear_output(wait=True)
            print("Kein Match zum Anzeigen vorhanden.")
        with neur_img_output:
            clear_output(wait=True)
        with combined_peak_output: # Auch den kombinierten Output leeren
            clear_output(wait=True)
        with nist_details_output: # Auch den NIST Details Output leeren
            clear_output(wait=True)
        update_match_navigation_state()
        return

    best_match = identified_matches[current_match_index]
    # Stelle sicher, dass aktueller_index gültig ist, bevor du auf identified_peaks zugreifst
    if not identified_peaks or not (0 <= aktueller_index < len(list(identified_peaks.values()))):
         with neur_output:
              clear_output(wait=True)
              print("Fehler: Informationen zum realen Peak nicht verfügbar.")
              return
    real_peak_info = list(identified_peaks.values())[aktueller_index] # Informationen des realen Peaks abrufen (verwende .values() und Index)

    with neur_output:
        clear_output(wait=True)
        print(f"Details für Match {current_match_index + 1}/{len(identified_matches)} für realen Peak bei {real_peak_info['peakpoint']:.2f} nm:")
        print(f"Zugeordnet zu: Element {best_match.get('element', 'N/A')} ({best_match.get('stage', 'N/A')}) bei {best_match.get('peakpoint', 'N/A'):.2f} nm") # Verwende .get() für sichereren Zugriff

        # Ausgabe der einzelnen Komponenten und des Scores
        print(f"  Wellenlängen-Differenz: {best_match.get('wavelength_diff', 'N/A'):.2f} nm") # Verwende .get()
        # Die normalisierten Wellenlängen- und Intensitäts-Sims werden nicht mehr direkt im best_match gespeichert
        # print(f"  Wellenlängen-Ähnlichkeit (Normalisiert): {best_match.get('normalized_wavelength_sim', 'N/A'):.4f}") # Entfernt, da Normalisierung für Score intern ist

        if best_match.get('intensity_diff') is not None: # Verwende .get() und überprüfe auf None
            print(f"  Intensitäts-Differenz: {best_match['intensity_diff']:.2f}")
            # print(f"  Intensitäts-Ähnlichkeit (Normalisiert): {best_match.get('normalized_intensity_sim', 'N/A'):.4f}") # Entfernt
        else:
            print("  Intensitäts-Differenz: N/A (NIST-Intensität fehlt)")
            # print("  Intensitäts-Ähnlichkeit (Normalisiert): N/A (NIST-Intensität fehlt)") # Entfernt


        print(f"  Form-Ähnlichkeit (Embedding-Kosinus): {best_match.get('similarity', 'N/A'):.4f}") # Verwende .get() - wird nur angezeigt
        if 'score' in best_match:
            print(f"  Kombinierter Score (Wellenlänge & Intensität): {best_match['score']:.4f}") # Den neuen Score anzeigen

        print("-" * 30)

        # --- Generate and display NIST URL as hyperlink with validation indicator ---
        matched_element = best_match.get('element')
        matched_stage = best_match.get('stage')
        matched_peakpoint = best_match.get('peakpoint')

        if matched_element and matched_stage and matched_peakpoint is not None:
             # Retrieve parameters for the URL from the global slider variables
             # Use the FULL RANGE defined by the sliders for the link
             low_w = low_w_slider.value
             upp_w = upp_w_slider.value
             temp = temp_slider.value
             resolution = resolution_slider.value
             eden_exponent = eden_slider.value
             maxcharge = maxcharge_slider.value
             min_rel_int = min_rel_int_slider.value
             int_scale = int_scale_slider.value
             eden = f"1e{eden_exponent}"

             # Convert ionization stage to NIST format (I=0, II=1, III=2, etc.)
             stage_code = matched_stage.replace('I', '0').replace('II', '1').replace('III', '2')

             # Use the full range from the sliders for the link parameters
             nist_link = f"https://physics.nist.gov/cgi-bin/ASD/lines1.pl?composition={matched_element}%3A100&mytext%5B%5D={matched_element}&myperc%5B%5D=100&spectra={matched_element}0-{stage_code}&low_w={low_w:.2f}&limits_type=0&upp_w={upp_w:.2f}&show_av=2&unit=1&resolution={resolution}&temp={temp}&eden={eden}&maxcharge={maxcharge}&min_rel_int={min_rel_int:.3f}&int_scale={int_scale}&libs=1"

             # Check if the current match is in the validated_matches list
             # Use tolerance for real_peak_wavelength comparison
             is_validated = any(
                 vm.get('real_peak_wavelength') is not None and real_peak_info.get('peakpoint') is not None and abs(float(vm.get('real_peak_wavelength')) - float(real_peak_info.get('peakpoint'))) < 1e-5 and
                 vm.get('nist_wavelength') == matched_peakpoint and
                 vm.get('nist_element') == matched_element and
                 vm.get('nist_stage') == matched_stage
                 for vm in validated_matches
             )

             # Add the original "Marked" indicator (based on whether the current match is in validated_matches)
             print('Marked:')
             original_indicator = "✅" if is_validated else "❌"
             print(f'{original_indicator}')

             # Add the new "Validated Match in NN" indicator if the match is validated
             if is_validated:
                  print('Validated Match in NN: ✅')
             else:
                  print('Validated Match in NN: ❌') # Show cross if not validated


             print('Click for NIST Data:')
             # Display the link as an HTML hyperlink
             display(widgets.HTML(f'<a href="{nist_link}" target="_blank">{matched_element} {matched_stage} at {matched_peakpoint:.2f} nm (Full Range)</a>'))

        else:
             print("NIST link cannot be generated (missing match information).")

    if 'match_number_input' in globals() and not match_number_input.disabled:
             match_number_input.max = len(identified_matches)
             match_number_input.value = current_match_index + 1 # Update the text field value
    # Plot of the assigned NIST peak alone (without green background)
    with neur_img_output:
        clear_output(wait=True)

        matched_element = best_match.get('element')
        matched_stage = best_match.get('stage')
        matched_peakpoint = best_match.get('peakpoint')

        if matched_element and matched_stage and matched_peakpoint is not None and nist_data and nist_data.get('data'):
             element_data = nist_data['data'].get(matched_element)
             if element_data and matched_stage in element_data:
                 df_stage = element_data[matched_stage]

                 fig, ax = plt.subplots(figsize=(8, 4))
                 ax.plot(df_stage['Wellenlänge (nm)'], df_stage['Intensität'], label=f"{matched_element} {matched_stage}")

                 ax.set_title(f"Zugeordneter NIST-Peak bei {matched_peakpoint:.2f} nm ({matched_element} {matched_stage})")
                 ax.set_xlabel("Wellenlänge (nm)")
                 ax.set_ylabel("Intensität")
                 ax.legend()

                 # Zoomen basierend auf dem REALEN Peakpoint und Suchbereich
                 if real_peak_info:
                      real_peak_wavelength = real_peak_info['peakpoint']
                      search_range = search_range_slider.value
                      ax.set_xlim([real_peak_wavelength - search_range, real_peak_wavelength + search_range])

                      # y-Achse basierend auf den Daten im aktuellen xlim-Bereich skalieren
                      x_min, x_max = ax.get_xlim()
                      df_filtered = df_stage[(df_stage['Wellenlänge (nm)'] >= x_min) & (df_stage['Wellenlänge (nm)'] <= x_max)]
                      if not df_filtered.empty:
                          y_min = df_filtered['Intensität'].min()
                          y_max = df_filtered['Intensität'].max()
                          ax.set_ylim([y_min * 0.9, y_max * 1.1]) # Puffer hinzugefügt


                 # Roten Marker für den NIST-Peak-Punkt hinzufügen
                 ax.axvline(x=matched_peakpoint, color='red', linestyle='--', label='NIST Match')

                 # Blauen Marker für den REALEN Peak-Punkt hinzufügen
                 if real_peak_info:
                      real_peak_wavelength = real_peak_info['peakpoint']
                      ax.axvline(x=real_peak_wavelength, color='blue', linestyle=':', label='Real Peak') # Blauer Marker hinzugefügt


                 ax.legend() # Legende nach Hinzufügen beider Marker erstellen


                 plt.show(fig)

             else:
                 print(f"Konnte NIST-Daten für {matched_element} {matched_stage} nicht finden.")
                 fig, ax = plt.subplots(figsize=(8, 4))
                 ax.text(0.5, 0.5, "Kein NIST-Plot verfügbar", horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
                 ax.set_title("NIST-Peak Plot")
                 ax.axis('off')
                 plt.show(fig)
        else:
             print("Informationen zum NIST-Match sind unvollständig.")
             fig, ax = plt.subplots(figsize=(8, 4))
             ax.text(0.5, 0.5, "Informationen zum NIST-Match sind unvollständig.", horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
             ax.set_title("NIST-Peak Plot")
             ax.axis('off')
             plt.show(fig)


    # NEU: Kombinierten Plot anzeigen
    with combined_peak_output:
        clear_output(wait=True)
        # Stelle sicher, dass alle benötigten Daten verfügbar sind, bevor plot_combined_peak_match aufgerufen wird
        # NEU: validated_matches an plot_combined_peak_match übergeben
        if wavelengths is not None and spectrum is not None and real_peak_info is not None and best_match is not None and nist_data is not None and search_range_slider is not None:
             plot_combined_peak_match(wavelengths, spectrum, real_peak_info, best_match, nist_data, search_range_slider.value, validated_matches) # Pass validated_matches
        else:
             print("Daten für kombinierten Plot nicht verfügbar.")

    with nist_details_output:
        clear_output(wait=True)
        matched_element = best_match.get('element')
        matched_stage = best_match.get('stage')
        matched_peakpoint = best_match.get('peakpoint')

        if matched_element and matched_stage and matched_peakpoint is not None and nist_data and nist_data.get('data'):
            element_data = nist_data['data'].get(matched_element)
            if element_data and matched_stage in element_data:
                df_stage = element_data[matched_stage]
                if not df_stage.empty:
                    fig, ax = plt.subplots(figsize=(10, 5)) # Anpassung der Größe
                    # Plot the full NIST spectrum for the matched element and stage
                    ax.plot(df_stage['Wellenlänge (nm)'], df_stage['Intensität'], label=f"{matched_element} {matched_stage}")
                    ax.set_title(f"NIST-Daten für {matched_element} {matched_stage}")
                    ax.set_xlabel("Wellenlänge (nm)")
                    ax.set_ylabel("Intensität")
                    ax.legend()

                    # Find and mark ALL peaks in THIS NIST data (for the matched element/stage)
                    # Use find_peaks with the specific NIST data
                    # Use a reasonable height and distance for NIST data peak finding
                    nist_peaks_in_full_range = find_peaks(df_stage['Intensität'].values, df_stage['Wellenlänge (nm)'].values, nist_data=nist_data, element=matched_element, ionization_stage=matched_stage) # Pass nist_data and element/stage

                    # Mark ALL found NIST peaks in the plot with red dots
                    nist_peak_wavelengths = [peak['peakpoint'] for peak in nist_peaks_in_full_range]
                    # Get the intensities of these peaks from the full df_stage
                    nist_peak_intensities = [df_stage['Intensität'].values[np.where(df_stage['Wellenlänge (nm)'].values == peak_wl)[0][0]] for peak_wl in nist_peak_wavelengths]

                    ax.plot(nist_peak_wavelengths, nist_peak_intensities, 'ro', markersize=4, label='NIST Peaks') # Red dots for all peaks

                    # Mark the SPECIFIC matched NIST peak (green dot and dashed line)
                    # Find the index of the matched_peakpoint in df_stage['Wellenlänge (nm)']
                    try:
                         matched_peak_index_in_df = np.where(df_stage['Wellenlänge (nm)'].values == matched_peakpoint)[0][0]
                         ax.plot(matched_peakpoint, df_stage['Intensität'].values[matched_peak_index_in_df], 'go', markersize=8, label='Zugeordneter NIST Peak') # Grüner Marker
                         ax.axvline(x=matched_peakpoint, color='green', linestyle='--', label='Zugeordneter NIST Peak') # Grüne gestrichelte Linie
                    except IndexError:
                         # Fallback, wenn exakte Wellenlänge nicht gefunden wird (kann bei interpolated data vorkommen)
                         print(f"Warnung: Exakte Wellenlänge {matched_peakpoint:.4f} nm nicht in NIST-Daten für Plot gefunden.")
                         # Try to find the nearest point
                         closest_index = np.abs(df_stage['Wellenlänge (nm)'].values - matched_peakpoint).argmin()
                         ax.plot(df_stage['Wellenlänge (nm)'].values[closest_index], df_stage['Intensität'].values[closest_index], 'go', markersize=8, label='Zugeordneter NIST Peak (closest)')
                         ax.axvline(x=df_stage['Wellenlänge (nm)'].values[closest_index], color='green', linestyle='--', label='Zugeordneter NIST Peak (closest)')


                    # Markiere den Bereich des REALEN Peaks (grünes Rechteck)
                    if real_peak_info:
                         real_peak_wavelength = real_peak_info['peakpoint']
                         search_range = search_range_slider.value
                         ax.axvspan(real_peak_wavelength - search_range, real_peak_wavelength + search_range, color='lightgreen', alpha=0.3, label='Bereich des realen Peaks') # Grünes Rechteck

                    ax.legend()
                    ax.grid(True) # Füge Grid hinzu

                    # Do NOT zoom by default, show the full range loaded by NIST
                    # If you want to zoom, you can uncomment and adapt the zoom logic below
                    # # Optional: Zoomen auf den Bereich um den NIST-Peak
                    # zoom_range = 5 # Beispiel: Zoom 5 nm um den Peak
                    # x_min = matched_peakpoint - zoom_range
                    # x_max = matched_peakpoint + zoom_range
                    # ax.set_xlim([x_min, x_max])

                    # # y-Achse basierend auf den Daten im aktuellen xlim-Bereich skalieren
                    # df_filtered = df_stage[(df_stage['Wellenlänge (nm)'] >= x_min) & (df_stage['Wellenlänge (nm)'] <= x_max)]
                    # if not df_filtered.empty:
                    #     y_min = df_filtered['Intensität'].min()
                    #     y_max = df_filtered['Intensität'].max()
                    #     ax.set_ylim([y_min * 0.9, y_max * 1.1]) # Puffer hinzugefügt


                    plt.show(fig)

                else:
                     print(f"Keine Daten für {matched_element} {matched_stage} zum Plotten verfügbar.")
                     fig, ax = plt.subplots(figsize=(10, 5))
                     ax.text(0.5, 0.5, f"Keine NIST-Daten für {matched_element} {matched_stage} zum Plotten verfügbar.", horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
                     ax.set_title("NIST Details Plot")
                     ax.axis('off')
                     plt.show(fig)

            else:
                print(f"Konnte NIST-Daten für Element {matched_element} oder Stage {matched_stage} nicht finden.")
                fig, ax = plt.subplots(figsize=(10, 5))
                ax.text(0.5, 0.5, f"Konnte NIST-Daten für Element {matched_element} oder Stage {matched_stage} nicht finden.", horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
                ax.set_title("NIST Details Plot")
                ax.axis('off')
                plt.show(fig)

        else:
            print("Informationen zum NIST-Match sind unvollständig oder NIST-Daten nicht geladen.")
            fig, ax = plt.subplots(figsize=(10, 5))
            ax.text(0.5, 0.5, "Informationen zum NIST-Match sind unvollständig oder NIST-Daten nicht geladen.", horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
            ax.set_title("NIST Details Plot")
            ax.axis('off')
            plt.show(fig)


    update_match_navigation_state()

def mark_nist_peaks(ax, wavelengths, spectrum, peak_indices, stage, current_peak_wavelength=None):
    """Markiert die NIST-Peaks im gegebenen Plot, farblich nach Ionisationsstufe."""
    peak_wavelengths = [peak['peakpoint'] for peak in peak_indices]
    peak_intensities = [spectrum[np.where(wavelengths == peak_wavelength)[0][0]] for peak_wavelength in peak_wavelengths]

    # Definiere Farben für die Ionisationsstufen
    stage_colors = {
        'I': 'blue',      # Blau für I
        'II': 'purple',   # Lila für II
        'III': 'magenta'  # Magenta für III
    }

    # Verwende die Farbe für die aktuelle Ionisationsstufe
    color = stage_colors.get(stage, 'black') # Standardfarbe: Schwarz, falls stage nicht in stage_colors

    num_peaks = len(peak_wavelengths)  # Anzahl der Peaks ermitteln

    # Separate Scatter-Plots für jede Ionisationsstufe
    scatter_handles = {}  # Dictionary für Scatter-Handles

    # Wenn es noch keinen Scatterplot für den Stage gibt
    if stage not in scatter_handles:
        scatter_handles[stage] = ax.scatter([], [], s=8, color=color, label=f"Peaks {stage} ({num_peaks})")  # Initialer Scatter-Plot

    # Update die Daten des Scatterplots für den Stage
    scatter_handles[stage].set_offsets(np.c_[peak_wavelengths, peak_intensities])  # Daten aktualisieren

    # Legende aktualisieren
    ax.legend(handles=list(scatter_handles.values()))  # Alle Scatter-Handles in der Legende anzeigen

    # Debug-Ausgabe
    #print("Peaks zum Markieren:", peak_wavelengths)

def plot_spectrum(wavelengths, spectrum, peak_indices=None, noise_std=None):
    """Plots the spectrum and optionally highlights peak indices and the noise level."""
    plt.figure(figsize=(10, 5))
    plt.plot(wavelengths, spectrum, label="LIBS-Data")

    # Rauschlinie hinzufügen, falls noise_std angegeben ist
    if noise_std is not None:
        plt.axhline(y=noise_std, color='red', linestyle='--', label=f"Rauschen ({noise_std:.2f})")

    if peak_indices is not None:  # Überprüfe, ob peak_indices nicht None ist
        peak_wavelengths = [peak['peakpoint'] for peak in peak_indices]
        peak_intensities = [spectrum[np.where(wavelengths == peak_wavelength)[0][0]] for peak_wavelength in peak_wavelengths]
        plt.scatter(peak_wavelengths, peak_intensities, s=8, color="red", label="Peaks")  # size in s geändert
        plt.legend() # Legende hier anzeigen, nachdem der Plot hinzugefügt wurde

    plt.title("LIBS Spectrum")
    plt.xlabel("Wavelength (nm)")
    plt.ylabel("Counts (a.u)")

def smooth_spectrum(spectrum, window_size):
    """Glättet das Spektrum mit einem gleitenden Durchschnitt."""
    window = np.ones(window_size) / window_size
    smoothed_spectrum = np.convolve(spectrum, window, mode='same')
    return smoothed_spectrum

def filter_peaks_min_distance(peak_indices, min_distance, intensity_threshold, noise_std):
    """Filtert Peaks basierend auf min_distance und Intensitätsunterschied.
    Bevorzugt größere Peaks bei gleicher Distanz.
    Ignoriert min_distance, wenn die Intensität ein Vielfaches des Rauschens ist.
    """
    filtered_peaks = []
    # Korrektur: Verwende den Index 'i' aus enumerate
    for i, peak in enumerate(peak_indices):
        is_valid_peak = True
        # Korrektur: Verwende den Index 'i' im range
        for j in range(i - 1, -1, -1):  # Überprüfe vorherige Peaks
            previous_peak = peak_indices[j]
            distance = abs(peak['peakpoint'] - previous_peak['peakpoint'])

            # Überprüfe, ob die Intensität ein Vielfaches des Rauschens ist
            if peak['intensity'] > intensity_threshold * noise_std:
                # Mindestabstand ignorieren
                break  # Beende die innere Schleife, da dieser Peak gültig ist

            if distance < min_distance:
                # Bevorzuge den Peak mit der höheren Intensität
                if peak['intensity'] < previous_peak['intensity']:
                    is_valid_peak = False
                    break  # Beende die innere Schleife, da dieser Peak ungültig ist
                else:
                    # Entferne den vorherigen Peak, da der aktuelle Peak größer ist
                    if previous_peak in filtered_peaks:
                        filtered_peaks.remove(previous_peak)

        if is_valid_peak:
            filtered_peaks.append(peak)
    return filtered_peaks

def plot_peak_details(wavelengths, spectrum, peak_indices, identified_peaks=None, window_size=20):
    """Erstellt Nahaufnahmen der Peaks mit Positionsanzeige im Gesamtspektrum und zeigt NIST-Daten an."""

    num_peaks = len(peak_indices)
    fig, axes = plt.subplots(num_peaks, 3, figsize=(20, 5 * num_peaks))
    fig.suptitle("Peak-Nahaufnahmen", fontsize=16)

    for i, peak_index in enumerate(peak_indices):
        peak_wavelength_key = f"{peak_index['peakpoint']:.2f}"
        peak_wavelength = peak_index['peakpoint']  # peakpoint aus Dictionary extrahieren
        peak_index_num = np.where(wavelengths == peak_wavelength)[0][0] # Index in wavelengths finden

        # Gefitteter Peak (linke Spalte)
        ax_fitted = axes[i, 0]
        if identified_peaks and peak_wavelength_key in identified_peaks:
            peak_data = identified_peaks[peak_wavelength_key]

            # Bestes Match abrufen
            best_match = peak_data['matches'][0] if peak_data['matches'] else None

            if best_match:
                ax_fitted.set_title(f"NIST-Peak bei {best_match['nist_wavelength']:.2f} nm")
                # Datenpunkte aus identified_peaks abrufen:
                peak_wavelengths_range = peak_data['measured_wavelengths']  # 'range_wavelengths' durch 'measured_wavelengths' ersetzt
                peak_spectrum_range = peak_data['measured_intensities']

                # Bereich der NIST-Daten basierend auf den Wellenlängen bestimmen
                lower_w = best_match['nist_wavelengths'].min()
                upper_w = best_match['nist_wavelengths'].max()

                # Indizes für den Bereich der NIST-Daten finden
                start_index_nist = np.where(best_match['nist_wavelengths'] >= lower_w)[0][0]
                end_index_nist = np.where(best_match['nist_wavelengths'] <= upper_w)[0][-1]

                # NIST-Daten für den Plot extrahieren
                peak_wavelengths_range = best_match['nist_wavelengths'][start_index_nist : end_index_nist + 1]
                peak_spectrum_range = best_match['nist_intensities'][start_index_nist : end_index_nist + 1]

                # Plot erstellen
                ax_fitted.plot(peak_wavelengths_range, peak_spectrum_range, 'b-', label=f"{best_match['element']} {best_match['ionization_stage']} ({best_match['match_percentage']:.2f}%)")

                # NIST-Peak grün markieren
                # finde den Index der NIST-Wellenlänge in der Liste der NIST-Wellenlängen
                nist_wavelength_index = np.where(best_match['nist_wavelengths'] == best_match['nist_wavelength'])[0][0]
                # extrahiere die NIST-Intensität aus der Liste der NIST-Intensitäten mithilfe des Index
                nist_intensity = best_match['nist_intensities'][nist_wavelength_index]

                ax_fitted.plot(best_match['nist_wavelength'], nist_intensity, 'go', markersize=6, label="NIST-Peak")

                # Andere identifizierte Peaks rot markieren
                for other_match in peak_data['matches'][1:]:  # Alle Matches außer dem besten Match
                    ax_fitted.plot(other_match['nist_wavelength'], other_match['nist_intensities'][nist_wavelength_index], 'ro', markersize=4, label=f"{other_match['element']} {other_match['ionization_stage']} ({other_match['match_percentage']:.2f}%)") # Label mit Element und Ionisationsstufe

                # ylim setzen auf Basis der Intensitäten aus identified_peaks
                ax_fitted.set_ylim([peak_spectrum_range.min(), peak_spectrum_range.max() * 1.1])

                ax_fitted.legend()  # Legende für gefitteten Peak-Plot anzeigen



        # Detail-Aufnahme (mittlere Spalte)
        ax_detail = axes[i, 1]

        # Bereich um den Peak aus der Spectrum-Datei extrahieren
        start_index = max(0, peak_index_num - window_size // 2)
        end_index = min(len(wavelengths), peak_index_num + window_size // 2)
        peak_wavelengths_range = wavelengths[start_index:end_index]
        peak_spectrum_range = spectrum[start_index:end_index]

        # Peak-Nahaufnahme darstellen
        ax_detail.plot(peak_wavelengths_range, peak_spectrum_range, label="Gefundenes Spektrum")

        # Alle Peaks im Subplot-Bereich markieren
        x_min, x_max = ax_detail.get_xlim()  # xlim-Werte des mittleren Fensters abrufen
        first_peak_in_subplot = True  # Flag für den ersten Peak im Subplot
        for p_index in peak_indices:
            if x_min <= p_index['peakpoint'] <= x_max:  # Bedingung anpassen
                peak_wavelength_in_subplot = p_index['peakpoint'] # peakpoint nutzen für x-Koordinate
                peak_intensity_in_subplot_index = np.where(wavelengths == peak_wavelength_in_subplot)[0][0] # Index in wavelengths finden
                peak_intensity_in_subplot = spectrum[peak_intensity_in_subplot_index] # Intensität aus spectrum extrahieren

                # Label nur beim ersten Peak im Subplot setzen
                if first_peak_in_subplot:
                    ax_detail.plot(peak_wavelength_in_subplot, peak_intensity_in_subplot, "ro", markersize=4, label="Gefundene Peaks")  # ax_detail statt ax_fitted
                    first_peak_in_subplot = False
                else:
                    ax_detail.plot(peak_wavelength_in_subplot, peak_intensity_in_subplot, "ro", markersize=4)  # Ohne Label # ax_detail statt ax_fitted

        # Fokussierten Peak im Subplot markieren und mittig darstellen
        peak_wavelength_in_subplot = wavelengths[peak_index_num]
        peak_intensity_in_subplot = spectrum[peak_index_num]
        ax_detail.plot(peak_wavelength_in_subplot, peak_intensity_in_subplot, "go", markersize=6, label="Fokussierter Peak")  # ax_detail statt ax_fitted

        ax_detail.set_title(f"Peak bei {wavelengths[peak_index_num]:.2f} nm")
        ax_detail.set_xlabel("Wellenlänge (nm)")
        ax_detail.set_ylabel("Counts (a.u)")

        # ylim anpassen für ähnliche Skala:
        ax_detail.set_ylim(bottom=0, top=np.max(peak_spectrum_range) * 1.1)
        ax_detail.legend()  # Legende für Detail-Plot anzeigen

        # Positionsanzeige im Gesamtspektrum (rechte Spalte)
        ax_position = axes[i, 2]
        ax_position.plot(wavelengths, spectrum)
        ax_position.set_title("Position im Gesamtspektrum")
        ax_position.set_xlabel("Wellenlänge (nm)")
        ax_position.set_ylabel("Counts (a.u)")

        # Bereich der Detail-Aufnahme im Gesamtspektrum markieren
        ax_position.axvspan(wavelengths[start_index], wavelengths[end_index], color='lightgray', alpha=0.5)

        # Fokussierten Peak im Positions-Plot markieren
        ax_position.plot(wavelengths[peak_index_num], spectrum[peak_index_num], "go", markersize=6)


    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

def plot_nist_data(nist_data, elements):
    plt.figure(figsize=(10, 5))
    for element in elements:
        element_data = nist_data.get(element)
        if element_data:
            for stage, df_stage in element_data.items():
                plt.plot(df_stage['Wellenlänge (nm)'], df_stage['Intensität'], label=f"{element} {stage}")  # label für Legende festlegen
                plt.legend()  # Legende hier anzeigen, nachdem der Plot hinzugefügt wurde
    plt.title("NIST ASD-Daten")
    plt.xlabel("Wellenlänge (nm)")
    plt.ylabel("Intensität")

def print_summary(identified_peaks, peak_wavelengths, wavelengths, spectrum, window_size, temp=0.4, resolution=250):
    """Gibt eine Zusammenfassung der identifizierten Elemente aus und zeigt einen Link zum identifizierten Peak in den NIST-Daten an."""
    print("\nZusammenfassung:")

    # Element-Zusammenfassung
    element_counts = {}
    element_matches = {}  # Dictionary zum Speichern der Matches für jedes Element

    for peak_data in identified_peaks.values():
        for match in peak_data['matches']:
            element = match['element']
            element_counts[element] = element_counts.get(element, 0) + 1

            # Matches für das Element speichern
            if element not in element_matches:
                element_matches[element] = []
            element_matches[element].append(match)

    # Ausgabe anpassen:
    for element, count in element_counts.items():
        print(f"Element: {element} ({count} Übereinstimmungen)")

        # Beste Übereinstimmungen für das Element finden und ausgeben
        best_matches = sorted(element_matches[element], key=lambda x: x['match_percentage'], reverse=True)
        for match in best_matches:
            # Berechne den Bereich um den Peak
            lower_w = max(200, match['wavelength'] - window_size / 2)
            upper_w = min(870, match['wavelength'] + window_size / 2)

            # Generiere den Link mit dem berechneten Bereich
            nist_link = f"https://physics.nist.gov/cgi-bin/ASD/lines1.pl?composition={match['element']}%3A100&mytext%5B%5D={match['element']}&myperc%5B%5D=100&spectra={match['element']}0-2&low_w={lower_w}&limits_type=0&upp_w={upper_w}&show_av=2&unit=1&resolution={resolution}&temp={temp}&eden=1e17&maxcharge=2&min_rel_int=0.01&int_scale=1&libs=1"

            print(f"  - Wellenlänge: {match['wavelength']:.2f} nm ({match['match_percentage']:.2f}%) anhand Peak bei {match['peakpoint']:.2f} nm: {nist_link}") # peakpoint hinzugefügt

    print("\nElement-Zusammenfassung:")
    for element, count in element_counts.items():
        print(f"  - {element}: {count}")

def find_nist_peaks(nist_data, element, ionization_stage, scipy_height=0, scipy_distance=1):
    """
    Findet Peaks in NIST-Daten mit scipy_find_peaks. KEINE GLÄTTUNG FÜR NIST-DATEN.
    Für "Sum" werden alle Peaks aus allen Ionisationsstufen zusammengefasst.
    """
    all_peak_data = []

    if ionization_stage == "Sum":
        # Für "Sum": Alle Peaks aus allen Ionisationsstufen zusammenfassen
        for stage, df_stage in nist_data['data'].get(element, {}).items():
            if stage != "Sum":
                spectrum = df_stage['Intensität'].values
                wavelengths = df_stage['Wellenlänge (nm)'].values

                # Verwendung von scipy_find_peaks DIREKT auf das Original-Spektrum
                # Passe height und distance an die NIST-Daten an!
                # height=10 (Beispiel): Minimalhöhe von 10 Counts. Passe dies an die typischen Intensitäten an.
                # distance=1 (Beispiel): Mindestabstand von 1 Datenpunkt. Gut für rauschfreie Daten.
                peaks_indices, properties = scipy_find_peaks(spectrum, height=scipy_height, distance=scipy_distance)

                for peak_index in peaks_indices:
                    all_peak_data.append({
                        'peakpoint': wavelengths[peak_index],
                        'intensity': spectrum[peak_index], # Nutze die Intensität aus dem Originalspektrum
                        'wavelength': wavelengths[peak_index],
                        'stage': stage
                    })
    else:
        # Für einzelne Ionisationsstufen: Peaks finden
        df_nist = nist_data['data'].get(element, {}).get(ionization_stage)
        if df_nist is None:
            print(f"Keine NIST-Daten für {element} {ionization_stage} gefunden.")
            return []

        spectrum = df_nist['Intensität'].values
        wavelengths = df_nist['Wellenlänge (nm)'].values

        # Verwendung von scipy_find_peaks DIREKT auf das Original-Spektrum
        # Passe height und distance an die NIST-Daten an!
        peaks_indices, properties = scipy_find_peaks(spectrum, height=scipy_height, distance=scipy_distance)

        for peak_index in peaks_indices:
            all_peak_data.append({
                'peakpoint': wavelengths[peak_index],
                'intensity': spectrum[peak_index],
                'wavelength': wavelengths[peak_index],
                'stage': ionization_stage
            })

    # Sortiere die Peaks nach Wellenlänge (optional, aber nützlich)
    all_peak_data.sort(key=lambda x: x['peakpoint'])

    return all_peak_data

def all_nist_data(elements, low_w, upp_w, temp, resolution, eden_exponent, maxcharge, min_rel_int, int_scale):
    """Lädt NIST ASD-Daten für alle Elemente im gesamten Wellenlängenbereich herunter und speichert sie im Cache."""
    nist_data = {}
    dataDopplerArray_str = None

    # Calculate eden value from exponent
    eden = f"1e{eden_exponent}" # Correctly format the eden parameter

    # Cache leeren, falls erforderlich (z.B. bei Problemen mit dem Cache)
    #shutil.rmtree(cache_dir, ignore_errors=True)  # Entfernen, um den Cache nicht immer zu leeren
    os.makedirs(cache_dir, exist_ok=True)

    for element in elements:
        # Updated URL with new parameters
        url = f"https://physics.nist.gov/cgi-bin/ASD/lines1.pl?composition={element}%3A100&mytext%5B%5D={element}&myperc%5B%5D=100&spectra={element}0-2&low_w={low_w}&limits_type=0&upp_w={upp_w}&show_av=2&unit=1&resolution={resolution}&temp={temp}&eden={eden}&maxcharge={maxcharge}&min_rel_int={min_rel_int}&int_scale=1&libs=1"
        # Updated cache file name to include new parameters
        cache_file = os.path.join(cache_dir, f"{element}_{low_w}_{upp_w}_{temp}_{resolution}_{eden}_{maxcharge}_{min_rel_int}_{int_scale}.json")

        print(f"Lade Daten für {element} herunter von: {url}")

        if os.path.exists(cache_file):
            with open(cache_file, 'r') as f:
                dataDopplerArray = json.load(f)
        else:
            try:
                response = requests.get(url)
                response.raise_for_status()

                # Extrahiere 'dataDopplerArray' aus dem JavaScript-Code
                match = re.search(r'var dataDopplerArray=(.*?);', response.text, re.DOTALL)

                dataDopplerArray = []  # Initialisiere dataDopplerArray vor dem if-else-block

                if match:
                    dataDopplerArray_str = match.group(1)

                    # Verbesserte Bereinigung des JSON-Strings
                    dataDopplerArray_str = dataDopplerArray_str.replace("'", '"')  # Ersetze einfache Anführungszeichen durch doppelte
                    dataDopplerArray_str = dataDopplerArray_str.replace('null', 'None')  # Ersetze null durch None
                    dataDopplerArray_str = re.sub(r'//.*?\n', '', dataDopplerArray_str)  # Entferne Kommentare
                    dataDopplerArray_str = re.sub(r'\\x[0-9a-fA-F]{2}', '', dataDopplerArray_str)  # Entferne ungültige Escape-Sequenzen
                    dataDopplerArray_str = re.sub(r'([a-zA-Z_][a-zA-Z0-9_]*):', r'"\1":', dataDopplerArray_str)  # Füge doppelte Anführungszeichen um Property-Namen hinzu
                    # Ersetze 'null' durch 'None' und evaluiere den String
                    dataDopplerArray_str = dataDopplerArray_str.replace('null', 'None')
                    dataDopplerArray_str = ast.literal_eval(dataDopplerArray_str)
                    dataDopplerArray_str = json.dumps(dataDopplerArray_str) # Wandle das Ergebnis zurück in einen JSON-String um
                    # Parse den bereinigten String
                    dataDopplerArray = json.loads(dataDopplerArray_str)

                else:
                    print(f"Fehler: 'dataDopplerArray' nicht im Quelltext gefunden.")
                    nist_data[element] = {}
                    continue

                with open(cache_file, 'w') as f:
                    json.dump(dataDopplerArray, f)

            except (requests.exceptions.RequestException, json.JSONDecodeError, ValueError) as e:
                print(f"Fehler beim Laden der NIST-Daten für {element}: {e}")
                nist_data[element] = {}
                continue
        # DataFrames für die Spektren erstellen
        df_nist_interp = {}
        df_nist_interp['Sum'] = pd.DataFrame({'Wellenlänge (nm)': [row[0] for row in dataDopplerArray[1:]],
                                           'Intensität': [row[1] for row in dataDopplerArray[1:]]})

        # Iteriere über die Ionisationsstufen (I, II, III, etc.)
        for i in range(2, len(dataDopplerArray[0])):  # Beginne bei Index 2, da 0 die Wellenlänge und 1 die Summe sind
            ionization_stage = 'I' if i == 2 else 'II' if i == 3 else 'III' if i == 4 else f'{i - 1}' # Bestimme die Ionisationsstufe dynamisch
            df_nist_interp[ionization_stage] = pd.DataFrame({'Wellenlänge (nm)': [row[0] for row in dataDopplerArray[1:]],
                                                           'Intensität': [row[i] for row in dataDopplerArray[1:]]})

        nist_data[element] = df_nist_interp  # Speichere die DataFrames für das Element

    return nist_data


def read_spectrum(file_path):
    """Liest das Spektrum aus einer Datei ein und gibt Wellenlängen und Intensitäten zurück."""
    spectrum_data = pd.read_csv(file_path, sep='\t', header=None, skiprows=1, names=['Wellenlänge', 'Intensität'])
    wavelengths = spectrum_data['Wellenlänge'].values
    spectrum = spectrum_data['Intensität'].values
    return wavelengths, spectrum

def calculate_noise(spectrum, noise_region):
    """Berechnet das Grundrauschen des Spektrums."""
    noise = spectrum[noise_region[0]:noise_region[1]]
    noise_std = np.std(noise)
    return noise_std

def find_peaks(spectrum, wavelengths, noise_std=0, s_value=1, min_distance=5, gradient_threshold=0.1, nist_data=None, element=None, ionization_stage=None, s_value_nist=1000, base_level_factor=0.1, nist_peak_threshold_factor=0.01):
    """
    Findet Peaks in einem Spektrum und berücksichtigt breite Peaks mithilfe der Gradientenanalyse.
    Berechnet zusätzlich die Peakbreite (FWHM) und speichert sie in den Peak-Daten.
    """
    # Debugging-Ausgabe: Überprüfen, ob die Funktion als NIST-Peak-Suche aufgerufen wird
    #print(f"DEBUG: find_peaks aufgerufen. nist_data is None: {nist_data is None}, element is None: {element is None}, ionization_stage is None: {ionization_stage is None}")


    if nist_data and element and ionization_stage:
        # Debugging-Ausgabe: Bestätigen, dass der NIST-Daten-Zweig erreicht wird
        #print(f"DEBUG: Verarbeite NIST-Daten für Element: {element}, Stage: {ionization_stage}")

        # NIST-Daten verwenden, noise_std ignorieren
        df_nist = nist_data.get('data', {}).get(element, {}).get(ionization_stage) # Greife korrekt auf die Datenstruktur zu
        if df_nist is None:
            print(f"DEBUG: Keine NIST-Daten für {element} {ionization_stage} gefunden.")
            return []

        spectrum_nist = df_nist['Intensität'].values
        wavelengths_nist = df_nist['Wellenlänge (nm)'].values
        threshold = 0  # Schwellenwert auf 0 setzen, da kein Rauschen erwartet wird

        # Peak-Erkennung für NIST-Daten mit scipy.signal.find_peaks:
        # Passe height und distance an die NIST-Daten an!
        # height=10 (Beispiel): Minimalhöhe von 10 Counts. Passe dies an die typischen Intensitäten an.
        # distance=1 (Beispiel): Mindestabstand von 1 Datenpunkt. Gut für rauschfreie Daten.
        # Teste mit anderen Parametern, z.B. prominenz oder relativer Höhe
        # Beispiel: prominence=10 (mindestens 10 Counts höher als die benachbarten Täler)
        # Beispiel: height=(None, 50000) (Peaks bis zu einer Höhe von 50000)
        # Beispiel: distance=5 (Mindestabstand von 5 Datenpunkten)

        # Debugging-Ausgabe: Parameter für scipy_find_peaks
        scipy_height = 10 # Beispielwert, anpassen!
        scipy_distance = 5 # Beispielwert, anpassen!
        scipy_prominence = 10 # Beispielwert, anpassen!

        #print(f"DEBUG: Rufe scipy_find_peaks auf mit height={scipy_height}, distance={scipy_distance}, prominence={scipy_prominence}")


        peaks_indices, properties = scipy_find_peaks(spectrum_nist, height=scipy_height, distance=scipy_distance, prominence=scipy_prominence)   # find_peaks_scipy mit prominence und distance

        # Debugging-Ausgabe: Anzahl der gefundenen Peaks
        #print(f"DEBUG: scipy_find_peaks gefunden: {len(peaks_indices)} Peaks.")


        # Hier werden die Daten für die gefundenen Peaks gesammelt:
        peak_data = []

        for peak_index in peaks_indices:
            # Für NIST-Peaks: Schwellenwert basierend auf Peakhöhe (oder einem festen Wert, je nach Bedarf)
            # Hier verwenden wir einen festen Schwellenwert, der niedrig genug ist,
            # um auch kleinere Peaks zu berücksichtigen, die von scipy_find_peaks gefunden wurden.
            # Oder wir verwenden den Schwellenwert, der in scipy_find_peaks verwendet wurde (z.B. height)
            peak_height = spectrum_nist[peak_index]
            # Verwenden Sie die Höhe, die in scipy_find_peaks verwendet wurde, oder einen anderen geeigneten Wert
            base_level = scipy_height # Oder ein kleiner Bruchteil von peak_height, z.B. peak_height * 0.05
            threshold = base_level

            # Basisbreite berechnen
            left_index = peak_index
            right_index = peak_index
            # Stellen Sie sicher, dass die Indizes innerhalb der Grenzen von spectrum_nist liegen
            while left_index > 0 and spectrum_nist[left_index] >= threshold: # >= verwenden, um den Startpunkt einzuschließen
                left_index -= 1
            while right_index < len(spectrum_nist) - 1 and spectrum_nist[right_index] >= threshold: # >= verwenden
                right_index += 1

            # range für NIST-Peaks basierend auf Basisbreite berechnen
            # Stellen Sie sicher, dass die Indizes für wavelengths_nist gültig sind
            lower_bound = wavelengths_nist[max(0, left_index)]
            upper_bound = wavelengths_nist[min(len(wavelengths_nist) - 1, right_index)]


            # Intensitäten und Wellenlängen im Bereich extrahieren:
            mask = (wavelengths_nist >= lower_bound) & (wavelengths_nist <= upper_bound)
            intensities_in_range = spectrum_nist[mask]
            wavelengths_in_range = wavelengths_nist[mask]

            # Peakbreite berechnen (FWHM)
            half_max = peak_height / 2
            left_fwhm_index = peak_index
            right_fwhm_index = peak_index
            # Stellen Sie sicher, dass die Indizes innerhalb der Grenzen liegen
            while left_fwhm_index > 0 and spectrum_nist[left_fwhm_index] > half_max:
                left_fwhm_index -= 1
            while right_fwhm_index < len(spectrum_nist) - 1 and spectrum_nist[right_fwhm_index] > half_max:
                right_fwhm_index += 1
            # Stellen Sie sicher, dass die Indizes für wavelengths_nist gültig sind
            width = wavelengths_nist[min(len(wavelengths_nist) - 1, right_fwhm_index)] - wavelengths_nist[max(0, left_fwhm_index)]


            # peak_data aktualisieren
            peak_data.append({
                'peakpoint': wavelengths_nist[peak_index],
                'range': (lower_bound, upper_bound),
                'peakrange': (lower_bound, upper_bound), # peakrange ist oft dasselbe wie range für NIST-Daten
                'intensities': intensities_in_range,  # Speichere die Intensitäten im Bereich
                'intensity': spectrum_nist[peak_index],  # Speichere die maximale Intensität
                'wavelengths': wavelengths_in_range,
                'wavelength': wavelengths_nist[peak_index],
                'width': width,  # Peakbreite hinzufügen
                'element': element, # Element und Ionisationsstufe hinzufügen
                'stage': ionization_stage
            })

        # Debug-Ausgabe für NIST-Peaks
        #print(f"DEBUG: Gefundene NIST-Peaks für {element} {ionization_stage}: {len(peak_data)}")
        # for peak in peak_data:
        #     print(f"  - Wellenlänge: {peak['peakpoint']:.2f} nm, Intensität: {peak['intensity']:.2f}")

        return peak_data


    else:
        # Debugging-Ausgabe: Bestätigen, dass der Messdaten-Zweig erreicht wird
        #print("DEBUG: Verarbeite Messdaten.")

        # Messdaten verwenden, wavelengths müssen separat übergeben werden
        threshold = 3 * noise_std / s_value  # Schwellenwert basierend auf 3σ/S-Regel

        # Peak-Erkennung für Messdaten (wie zuvor)
        peak_indices = []
        for i in range(1, len(spectrum) - 1):
            if spectrum[i] > spectrum[i - 1] and spectrum[i] > spectrum[i + 1]:
                if not peak_indices or i - peak_indices[-1] >= min_distance:
                    if spectrum[i] > threshold:
                        peak_indices.append(i)

        # Hier werden die Daten für die gefundenen Peaks gesammelt:
        peak_data = []

        for peak_index in peak_indices:
            # Für gemessene Peaks: Schwellenwert basierend auf base_level_factor
            peak_height = spectrum[peak_index]  # peak_height für FWHM-Berechnung definieren
            base_level = spectrum[peak_index] * base_level_factor
            threshold = base_level

            # Basisbreite berechnen (für beide Peak-Typen)
            left_index = peak_index
            right_index = peak_index
            while left_index > 0 and spectrum[left_index] > threshold:
                left_index -= 1
            while right_index < len(spectrum) - 1 and spectrum[right_index] > threshold:
                right_index += 1

            # range für gemessene oder NIST-Peaks basierend auf Basisbreite berechnen
            lower_bound = wavelengths[left_index]
            upper_bound = wavelengths[right_index]

            # Intensitäten und Wellenlängen im Bereich extrahieren:
            mask = (wavelengths >= lower_bound) & (wavelengths <= upper_bound)
            intensities_in_range = spectrum[mask]
            wavelengths_in_range = wavelengths[mask]

            # Peakbreite berechnen (FWHM)
            half_max = peak_height / 2
            left_fwhm_index = peak_index
            right_fwhm_index = peak_index
            while left_fwhm_index > 0 and spectrum[left_fwhm_index] > half_max:
                left_fwhm_index -= 1
            while right_fwhm_index < len(spectrum) - 1 and spectrum[right_fwhm_index] > half_max:
                right_fwhm_index += 1
            width = wavelengths[right_fwhm_index] - wavelengths[left_fwhm_index]


            # peak_data aktualisieren
            peak_data.append({
                'peakpoint': wavelengths[peak_index],
                'range': (lower_bound, upper_bound),
                'peakrange': (lower_bound, upper_bound),
                'intensities': intensities_in_range,  # Speichere die Intensitäten im Bereich
                'intensity': spectrum[peak_index],  # Speichere die maximale Intensität
                'wavelengths': wavelengths_in_range,
                'wavelength': wavelengths[peak_index],
                'width': width  # Peakbreite hinzufügen
            })

        # Debug-Ausgabe für Messdaten-Peaks
        #print(f"DEBUG: Gefundene Messdaten-Peaks: {len(peak_data)}")


        return peak_data

def bilder_anzeigen(index):
    global current_peak_index, nn_outputs_visible
    current_peak_index = index  # index ist der Index des Peaks im Detail-Output

    # NIST-Detail-Output aktualisieren, nur wenn NIST Details sichtbar sind
    if nn_outputs_visible: # Überprüfen Sie, ob die NN-Outputs sichtbar sind
        update_element_identification_outputs()
    if nist_details_visible:
        # nist_details_button_clicked Funktion simulieren
        nist_details_button_clicked(None)
        update_nist_markers(index)
    with details_output:
        clear_output(wait=True)
        display(widgets.HBox([bilder[index]], layout=widgets.Layout(width='100%')))#, layout=widgets.Layout(width='100%'))) # Added layout for the HBox

    with output_spectrum:  # Zugriff auf das output_spectrum Widget
        clear_output(wait=True)

        # Spektrum plotten
        fig, ax = plt.subplots(figsize=(10, 5)) # Figure und Axes erstellen
        ax.plot(wavelengths, spectrum, label="LIBS-Daten") # Label für die Spektrumdaten hinzufügen

        # Peaks markieren
        # Modified to iterate over the values of the identified_peaks dictionary
        peak_wavelengths = [peak['peakpoint'] for peak in identified_peaks.values()]
        peak_intensities = [spectrum[np.where(wavelengths == peak_wavelength)[0][0]] for peak_wavelength in peak_wavelengths]
        num_peaks = len(peak_wavelengths)  # Anzahl der Peaks ermitteln
        peak_scatter = ax.scatter(peak_wavelengths, peak_intensities, s=8, color="red", label=f"Peaks ({num_peaks})")  # Label anpassen
        globals()['peak_scatter'] = peak_scatter # Speichere die Scatter-Punkte in einer globalen Variable
        plt.axhline(y=noise_std_slider.value, color='red', linestyle='--', label=f"Rauschen ({noise_std_slider.value:.2f})")

        # Aktuellen Peak grün markieren
        # Access the peak info using the index from the identified_peaks dictionary
        peak_wavelength = list(identified_peaks.values())[aktueller_index]['peakpoint']  # Wellenlänge des aktuellen Peaks
        peak_intensity = spectrum[np.where(wavelengths == peak_wavelength)[0][0]]  # Intensität des aktuellen Peaks
        ax.scatter(peak_wavelength, peak_intensity, s=12, color="green",
                label=f"Peak ({aktueller_index + 1} von {num_peaks}), λ = {peak_wavelength:.2f} nm")  # Aktuellen Peak grün markieren

        # Positionsanzeige im Spektrum integrieren
        peak_index_num = np.where(wavelengths == peak_wavelength)[0][0]  # Index des aktuellen Peaks
        window_size = 20  # Fenstergröße für die Positionsanzeige
        start_index = max(0, peak_index_num - window_size // 2)
        end_index = min(len(wavelengths), peak_index_num + window_size // 2)
        ax.axvspan(wavelengths[start_index], wavelengths[end_index], color='lightgreen', alpha=0.5) # Bereich grün markieren

        ax.legend() # Legende hier anzeigen, nachdem der Plot hinzugefügt wurde
        ax.set_title("LIBS Spectrum")
        ax.set_xlabel("Wellenlänge (nm)")
        ax.set_ylabel("Counts (a.u)")
        plt.show()
    peak_number_input.value = index + 1  # Textfeld mit dem aktuellen Index aktualisieren


def peak_details_interactive(wavelengths, spectrum, peak_indices, identified_peaks=None, window_size=20):
    """Erstellt Nahaufnahmen der Peaks als Widgets für die interaktive Anzeige."""
    global bilder, aktueller_index, output_peak_details # Add output_peak_details
    bilder = []
    aktueller_index = 0 # Reset index when regenerating interactive plots
    num_peaks = len(peak_indices)


    for i, peak_index_info in enumerate(peak_indices): # Iteriere über die Peak-Infos (Dictionaries)
        peak_wavelength = peak_index_info['peakpoint'] # Wellenlänge aus Dictionary
        peak_index_num = np.where(wavelengths == peak_wavelength)[0][0] # Index in wavelengths finden

        # Detail-Aufnahme
        fig_detail, ax_detail = plt.subplots(figsize=(10, 5))
        start_index = max(0, peak_index_num - window_size // 2)
        end_index = min(len(wavelengths), peak_index_num + window_size // 2)
        peak_wavelengths_range = wavelengths[start_index:end_index]
        peak_spectrum_range = spectrum[start_index:end_index]
        ax_detail.plot(peak_wavelengths_range, peak_spectrum_range, label="LIBS-Spektrum")
        # Label des Peaks in der Legende anpassen
        peak_number = i + 1  # Nummer des aktuellen Peaks
        peak_label = f"Peak ({peak_number} von {num_peaks})"  # Label erstellen
        ax_detail.plot(wavelengths[peak_index_num], spectrum[peak_index_num], "ro", markersize=8, label=peak_label)  # Peak markieren
        ax_detail.set_title(f"Peak bei {wavelengths[peak_index_num]:.2f} nm (Detail)")
        ax_detail.set_xlabel("Wellenlänge (nm)")
        ax_detail.set_ylabel("Counts (a.u)")
        ax_detail.legend()

        # Create an Output widget for the individual plot
        out_detail = widgets.Output(layout=widgets.Layout(height='400px', width='50%'))
        with out_detail: # Display the figure in the output widget
           display(fig_detail)
        plt.close(fig_detail) # Close the matplotlib figure to save memory

        bilder.append(out_detail)  # Add the Output widget containing the plot to the bilder list


def drive_selection_changed(change):
    """Zeigt/versteckt das file_path_widget oder file_upload_widget je nach Auswahl."""
    if change['new'] == 'Google Drive':
        file_path_widget.visible = True
        file_upload_widget.visible = False
    else:
        file_path_widget.visible = False
        file_upload_widget.visible = True

def read_spectrum(file_path):
    """Liest das Spektrum aus einer Datei ein und gibt Wellenlängen und Intensitäten zurück."""
    spectrum_data = pd.read_csv(file_path, sep='\t', header=None, skiprows=1, names=['Wellenlänge', 'Intensität'])
    wavelengths = spectrum_data['Wellenlänge'].values
    spectrum = spectrum_data['Intensität'].values
    return wavelengths, spectrum

def plot_combined_peak_match(wavelengths, spectrum, real_peak_info, best_match, nist_data, search_range_nm, validated_matches):
    """
    Plottet den Bereich des realen Peaks und den zugeordneten NIST-Peak in einem Diagramm.
    Markiert validierte reale Peaks speziell.
    """
    fig, ax = plt.subplots(figsize=(10, 5))

    # Plot Real Peak region
    if wavelengths is not None and spectrum is not None and real_peak_info is not None:
        real_peak_wavelength = real_peak_info['peakpoint']
        min_wl = real_peak_wavelength - search_range_nm
        max_wl = real_peak_wavelength + search_range_nm
        idx_min = np.searchsorted(wavelengths, min_wl)
        idx_max = np.searchsorted(wavelengths, max_wl, side='right')
        # Erweitern Sie den Bereich leicht, um sicherzustellen, dass die Endpunkte des Plots sichtbar sind
        idx_min = max(0, idx_min - 1)
        idx_max = min(len(wavelengths), idx_max + 1)

        if idx_max > idx_min:
            ax.plot(wavelengths[idx_min:idx_max], spectrum[idx_min:idx_max], label='Reales Spektrum', color='blue')
            # Mark the real peak point
            real_peak_index_in_range = np.abs(wavelengths[idx_min:idx_max] - real_peak_wavelength).argmin()
            ax.plot(wavelengths[idx_min:idx_max][real_peak_index_in_range], spectrum[idx_min:idx_max][real_peak_index_in_range], 'bo', markersize=8, label=f'Real Peak ({real_peak_wavelength:.2f} nm)')
            ax.axvline(x=real_peak_wavelength, color='blue', linestyle='--')

            # NEU: Check if this real peak is in the validated_matches and mark it
            is_validated_real_peak = any(
                vm.get('real_peak_wavelength') is not None and abs(float(vm.get('real_peak_wavelength')) - float(real_peak_wavelength)) < 1e-5 # Use tolerance
                for vm in validated_matches
            )
            if is_validated_real_peak:
                 ax.plot(wavelengths[idx_min:idx_max][real_peak_index_in_range], spectrum[idx_min:idx_max][real_peak_index_in_range], 'go', markersize=12, fillstyle='none', markeredgewidth=2, label='Validated Real Peak') # Green circle, not filled


    # Plot NIST Match peak
    if best_match is not None and nist_data is not None and nist_data.get('data'):
        matched_element = best_match.get('element')
        matched_stage = best_match.get('stage')
        matched_peakpoint = best_match.get('peakpoint')

        if matched_element and matched_stage and matched_peakpoint is not None:
            element_data = nist_data['data'].get(matched_element)
            if element_data and matched_stage in element_data:
                df_stage = element_data[matched_stage]
                if not df_stage.empty:
                    # Find NIST data points within the real peak's search range
                    nist_min_wl = real_peak_wavelength - search_range_nm
                    nist_max_wl = real_peak_wavelength + search_range_nm
                    df_stage_filtered = df_stage[(df_stage['Wellenlänge (nm)'] >= nist_min_wl) & (df_stage['Wellenlänge (nm)'] <= nist_max_wl)]

                    if not df_stage_filtered.empty:
                         ax.plot(df_stage_filtered['Wellenlänge (nm)'], df_stage_filtered['Intensität'], label=f"NIST {matched_element} {matched_stage}", color='red', linestyle='-')

                         # Mark the specific matched NIST peak point
                         nist_peak_index_in_df = np.abs(df_stage_filtered['Wellenlänge (nm)'].values - matched_peakpoint).argmin() # Find index in filtered data
                         ax.plot(df_stage_filtered['Wellenlänge (nm)'].values[nist_peak_index_in_df], df_stage_filtered['Intensität'].values[nist_peak_index_in_df], 'ro', markersize=8, label=f'NIST Match ({matched_peakpoint:.2f} nm)') # Red dot

                         # Add dashed line for the specific matched NIST peak
                         ax.axvline(x=matched_peakpoint, color='red', linestyle='--')

    ax.set_title("Combined Real Peak and NIST Match")
    ax.set_xlabel("Wavelength (nm)")
    ax.set_ylabel("Intensity")
    ax.legend()
    ax.grid(True)

    # Set x-axis limits based on the real peak and search range
    if real_peak_info is not None:
        real_peak_wavelength = real_peak_info['peakpoint']
        ax.set_xlim([real_peak_wavelength - search_range_nm, real_peak_wavelength + search_range_nm])

        # Adjust y-axis limits based on data within the x-axis limits
        x_min, x_max = ax.get_xlim()
        combined_y_data = []
        # Add real spectrum data in the current x-range
        if wavelengths is not None and spectrum is not None:
             idx_min_vis = np.searchsorted(wavelengths, x_min)
             idx_max_vis = np.searchsorted(wavelengths, x_max, side='right')
             combined_y_data.extend(spectrum[max(0, idx_min_vis - 1):min(len(spectrum), idx_max_vis + 1)])

        # Add NIST data in the current x-range
        if best_match is not None and nist_data is not None and nist_data.get('data'):
             matched_element = best_match.get('element')
             matched_stage = best_match.get('stage')
             if matched_element and matched_stage:
                  element_data = nist_data['data'].get(matched_element)
                  if element_data and matched_stage in element_data:
                       df_stage = element_data[matched_stage]
                       if not df_stage.empty:
                            df_filtered_vis = df_stage[(df_stage['Wellenlänge (nm)'] >= x_min) & (df_stage['Wellenlänge (nm)'] <= x_max)]
                            if not df_filtered_vis.empty:
                                 combined_y_data.extend(df_filtered_vis['Intensität'].values)

        if combined_y_data:
            y_min_combined = min(combined_y_data)
            y_max_combined = max(combined_y_data)
            ax.set_ylim([y_min_combined * 0.9, y_max_combined * 1.1]) # Add a 10% buffer

    plt.show(fig)

In [11]:
# @title Neuronales Netz

def convert_validated_matches_to_triplets(validated_matches, nist_data, window_size_nm, target_size, num_negatives_per_match=5):
    """
    Konvertiert validierte Matches in Trainings-Tripel (Anker, Positiv, Negativ).

    Args:
        validated_matches (list): Liste der validierten Match-Dictionaries.
        nist_data (dict): Das geladene NIST-Daten-Dictionary (benötigt für negative Samples).
        window_size_nm (float): Fenstergröße für die Peak-Datenvorbereitung in nm.
        target_size (int): Zielgröße nach dem Resampling.
        num_negatives_per_match (int): Anzahl der negativen Samples, die pro validiertem Match generiert werden sollen.

    Returns:
        tuple: Ein Tuple von NumPy-Arrays: (anker_data, positive_data, negative_data).
               Gibt None, None, None zurück, wenn keine Tripel erstellt werden können.
    """
    anker_data = []
    positive_data = []
    negative_data = []

    # Sammeln Sie alle NIST-Peaks (außer "Sum") für die Auswahl negativer Beispiele
    all_nist_peaks_for_negatives = []
    for element, stages in nist_data.get('data', {}).items(): # Use .get() for safer access
        for stage, df_stage in stages.items():
            if stage != "Sum" and not df_stage.empty:
                # Finde Peaks in NIST-Daten (verwende find_peaks)
                # KORREKTUR: find_peaks muss nun auch element und ionization_stage für NIST-Daten verarbeiten
                peak_indices_nist = find_peaks(df_stage['Intensität'].values, df_stage['Wellenlänge (nm)'].values, nist_data=nist_data, element=element, ionization_stage=stage)
                for peak_info in peak_indices_nist:
                    # Füge die benötigten Daten für extract_and_prepare_peak_data hinzu
                    peak_info_with_spectrum = peak_info.copy()
                    peak_info_with_spectrum['wavelengths'] = df_stage['Wellenlänge (nm)'].values
                    peak_info_with_spectrum['spectrum'] = df_stage['Intensität'].values
                    peak_info_with_spectrum['element'] = element
                    peak_info_with_spectrum['stage'] = stage
                    all_nist_peaks_for_negatives.append(peak_info_with_spectrum)

    if not all_nist_peaks_for_negatives:
        print("Keine NIST-Peaks zur Auswahl negativer Beispiele gefunden.")
        # Sie können trotzdem positive Tripel erstellen, aber keine negativen.
        # Je nach Triplet Loss Implementierung kann dies problematisch sein.
        # Für Semi-Hard Triplet Loss werden negative Beispiele benötigt.
        return None, None, None


    for match in validated_matches:
        # Anker: Der reale Peak
        real_peak_prepared = match.get('real_peak_prepared_data') # Verwenden Sie die bereits vorbereiteten Daten

        if real_peak_prepared is None:
             print(f"Warnung: Vorbereitete Daten für realen Peak bei {match.get('real_peak_wavelength', 'N/A'):.2f} nm fehlen. Überspringe Match.")
             continue # Überspringe diesen Match, wenn vorbereitete Daten fehlen

        # Positiv: Der zugeordnete NIST-Peak
        matched_nist_element = match.get('nist_element')
        matched_nist_stage = match.get('nist_stage')
        matched_nist_wavelength = match.get('nist_wavelength')

        # Finde die Original-NIST-Daten für den positiven Peak
        nist_positive_peak_info = None
        if matched_nist_element and matched_nist_stage and matched_nist_wavelength is not None:
             element_data = nist_data.get('data', {}).get(matched_nist_element)
             if element_data and matched_nist_stage in element_data:
                  df_stage = element_data[matched_nist_stage]
                  # Finden Sie den Peak in den NIST-Daten, der der Wellenlänge am nächsten liegt
                  if not df_stage.empty:
                       closest_index = np.abs(df_stage['Wellenlänge (nm)'].values - matched_nist_wavelength).argmin()
                       # Erstellen Sie ein Peak-Info-Dictionary ähnlich wie bei find_peaks
                       nist_positive_peak_info = {
                           'peakpoint': df_stage['Wellenlänge (nm)'].values[closest_index],
                           'intensity': df_stage['Intensität'].values[closest_index],
                           'wavelengths': df_stage['Wellenlänge (nm)'].values, # Füge gesamte Wellenlängen hinzu
                           'spectrum': df_stage['Intensität'].values # Füge gesamtes Spektrum hinzu
                       }

        if nist_positive_peak_info is None:
             print(f"Warnung: Original NIST-Daten für Match {matched_nist_element} {matched_nist_stage} bei {matched_nist_wavelength:.2f} nm nicht gefunden. Überspringe Match.")
             continue # Überspringe diesen Match, wenn die Original NIST-Daten fehlen


        # Bereite die positiven Daten vor (verwende extract_and_prepare_peak_data)
        positive_prepared = extract_and_prepare_peak_data(
            nist_positive_peak_info['wavelengths'],
            nist_positive_peak_info['spectrum'],
            nist_positive_peak_info['peakpoint'],
            window_size_nm,
            target_size
        )

        if positive_prepared is None or positive_prepared.size == 0 or np.sum(positive_prepared) == 0:
             print(f"Warnung: Vorbereitung der positiven NIST-Daten für Match {matched_nist_element} {matched_nist_stage} bei {matched_nist_wavelength:.2f} nm fehlgeschlagen. Überspringe Match.")
             continue # Überspringe diesen Match, wenn die positiven Daten nicht vorbereitet werden können


        # Negative Beispiele: Zufällige NIST-Peaks, die NICHT dem zugeordneten Element/Stage entsprechen
        generated_negatives = 0
        attempts = 0
        max_attempts_for_negatives = num_negatives_per_match * 10 # Vermeide unendliche Schleifen

        while generated_negatives < num_negatives_per_match and attempts < max_attempts_for_negatives:
             attempts += 1
             # Wählen Sie zufällig einen NIST-Peak aus der Liste aller NIST-Peaks
             random_nist_peak = random.choice(all_nist_peaks_for_negatives)

             # Stellen Sie sicher, dass der zufällige Peak NICHT dem Element/Stage des positiven Peaks entspricht
             if (random_nist_peak.get('element') != matched_nist_element or
                 random_nist_peak.get('stage') != matched_nist_stage):

                  # Bereite die negativen Daten vor
                  negative_prepared = extract_and_prepare_peak_data(
                      random_nist_peak['wavelengths'],
                      random_nist_peak['spectrum'],
                      random_nist_peak['peakpoint'],
                      window_size_nm,
                      target_size
                  )

                  if negative_prepared is not None and negative_prepared.size > 0 and np.sum(negative_prepared) > 0:
                      # Füge das Tripel hinzu
                      anker_data.append(real_peak_prepared)
                      positive_data.append(positive_prepared)
                      negative_data.append(negative_prepared)
                      generated_negatives += 1
                  #else:
                      # Optional: Warnung für fehlgeschlagene negative Datenvorbereitung
                      # print(f"Warnung: Datenvorbereitung für negatives Sample fehlgeschlagen (Peak bei {random_nist_peak.get('peakpoint', 'N/A'):.2f} nm).")

        # Optional: Warnung, wenn nicht genügend negative Samples gefunden wurden
        if generated_negatives < num_negatives_per_match:
             print(f"Warnung: Konnte nur {generated_negatives} von {num_negatives_per_match} negativen Samples für Match bei {match.get('real_peak_wavelength', 'N/A'):.2f} nm generieren.")


    # In NumPy-Arrays konvertieren
    if anker_data:
        anker_data_np = np.array(anker_data)
        positive_data_np = np.array(positive_data)
        negative_data_np = np.array(negative_data)
        return anker_data_np, positive_data_np, negative_data_np
    else:
        return None, None, None

def train_nn_button_clicked(b):
    global nist_data, prepared_peaks_info, embedding_model, nist_peak_embeddings, all_elements_for_nn, validated_matches, nn_model_selection_widget

    print("DEBUG: train_nn_button_clicked started.")

    with neur_output:
        clear_output(wait=True)
        print("Starting NN Training (Raw NIST NN)...")

    all_elements_for_training = all_elements_for_nn
    additional_elements = [elem.strip() for elem in additional_elements_widget.value.split(',') if elem.strip() and elem.strip() not in all_elements_for_training]
    all_elements_for_training.extend(additional_elements)

    model_loaded_successfully = False

    with neur_output:
        print(f"Elements for Raw NIST NN training: {all_elements_for_training}")

    temp = temp_slider.value
    resolution = resolution_slider.value
    low_w = low_w_slider.value
    upp_w = upp_w_slider.value
    eden_exponent = eden_slider.value
    maxcharge = maxcharge_slider.value
    min_rel_int = min_rel_int_slider.value
    int_scale = int_scale_slider.value

    current_elements_in_nist_data = nist_data.get('elements', [])
    current_nist_params = nist_data.get('params', {})
    new_nist_params = {
        'low_w': low_w, 'upp_w': upp_w, 'temp': temp, 'resolution': resolution,
        'eden_exponent': eden_exponent, 'maxcharge': maxcharge,
        'min_rel_int': min_rel_int, 'int_scale': int_scale
    }

    if not nist_data or sorted(current_elements_in_nist_data) != sorted(all_elements_for_training) or current_nist_params != new_nist_params:
        with neur_output:
            print("Loading NIST data for training...")
        nist_data['elements'] = all_elements_for_training
        nist_data['data'] = all_nist_data(all_elements_for_training, low_w, upp_w, temp, resolution, eden_exponent, maxcharge, min_rel_int, int_scale)
        nist_data['params'] = new_nist_params
        if nist_data['data']:
             with neur_output:
                  print("NIST data for training successfully loaded.")
        else:
             print("Error loading NIST data for training.")
             element_identification_button.disabled = True
             train_nn_button.disabled = True
             return
    else:
         with neur_output:
              print("NIST data for training is already loaded.")

    with neur_output:
        print("Creating training triplets from NIST data...")
    window_size_nm = search_range_slider.value
    anker_data_nist, positive_data_nist, negative_data_nist = create_triplets_from_nist(nist_data, window_size_nm=window_size_nm, target_size=TARGET_SIZE)

    if anker_data_nist is None or len(anker_data_nist) == 0:
        with neur_output:
            print("Could not create training triplets from NIST data. Training aborted.")
        element_identification_button.disabled = True
        return

    split_index = int(anker_data_nist.shape[0] * 0.8)
    anker_train = anker_data_nist[:split_index]
    positive_train = positive_data_nist[:split_index]
    negative_train = negative_data_nist[:split_index]

    anker_val = anker_data_nist[split_index:]
    positive_val = positive_data_nist[split_index:]
    negative_val = negative_data_nist[split_index:]

    X_val = [anker_val, positive_val, negative_val]
    y_val = np.zeros((anker_val.shape[0], 1))

    input_shape = (TARGET_SIZE, 1)
    embedding_model = build_embedding_model(input_shape, EMBEDDING_DIMENSION)
    with neur_output:
         print(f"Model built for Raw NIST NN training. embedding_model status: {embedding_model is not None}")

    with neur_output:
        print("Starting Raw NIST NN training...")

    epochs = epochs_slider.value
    batch_size = batch_size_slider.value
    learning_rate = learning_rate_slider.value
    elements_str = "_".join(sorted(all_elements_for_training))
    nist_params_str = f"lw{low_w:.1f}_uw{upp_w:.1f}_t{temp:.1f}_r{resolution}_e{eden_exponent}_mc{maxcharge}_mri{min_rel_int:.3f}_is{int_scale}".replace('.', '_')
    # Use a distinct filename for the Raw NIST NN
    model_filename = f"embedding_model_e{epochs}_bs{batch_size}_lr{learning_rate:.4f}_RawNISTNN_{nist_params_str}.keras"
    model_path = os.path.join('/content/drive/MyDrive/Gemini/Gemini_Embeddings/Models/', model_filename)

    model_dir = os.path.dirname(model_path)
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
        with neur_output:
             print(f"Created storage folder for model: {model_dir}")

    train_embedding_model(
        embedding_model,
        anker_train, positive_train, negative_train,
        X_val, y_val,
        epochs=epochs,
        batch_size=batch_size,
        learning_rate=learning_rate,
    )

    with neur_output:
        print("Raw NIST NN Training completed.")
        print(f"embedding_model status after training: {embedding_model is not None}")

    if embedding_model is not None:
        try:
            with neur_output:
                 print(f"Attempting to save Raw NIST NN model to: {model_path}")
            embedding_model.save(model_path)
            with neur_output:
                print(f"Raw NIST NN model successfully saved at: {model_path}")
            model_loaded_successfully = True
            nn_model_selection_widget.value = 'Raw NIST NN'
            with neur_output:
                 print("NN Model selection updated to 'Raw NIST NN'.")

        except Exception as save_e:
             with neur_output:
                  print(f"Error saving the trained Raw NIST NN model: {save_e}")
                  element_identification_button.disabled = True
                  return
    else:
        with neur_output:
            print("embedding_model is None after training. Raw NIST NN model will not be saved.")

    embeddings_filename = f"nist_embeddings_RawNISTNN_all_available_elements_{nist_params_str}.npy"
    embeddings_save_path = os.path.join('/content/drive/MyDrive/Gemini/Gemini_Embeddings/Embeddings/', embeddings_filename)

    embeddings_dir = os.path.dirname(embeddings_save_path)
    if not os.path.exists(embeddings_dir):
        os.makedirs(embeddings_dir)
        with neur_output:
             print(f"Created storage folder for Embeddings: {embeddings_dir}")

    nist_peak_embeddings = None

    if model_loaded_successfully and nist_data and nist_data.get('data') and embedding_model is not None:
        with neur_output:
            print("Calculating and storing NIST Peak Embeddings after Raw NIST NN training...")
        calculate_and_store_nist_embeddings(nist_data, embedding_model, window_size_nm=search_range_slider.value, target_size=TARGET_SIZE, embeddings_save_path=embeddings_save_path)
        with neur_output:
             if nist_peak_embeddings is not None and len(nist_peak_embeddings) > 0:
                 print("NIST Peak Embeddings process completed successfully.")
             else:
                 print("NIST Peak Embeddings could not be calculated or loaded.")

    else:
         with neur_output:
              print("Model or NIST data not available after training. NIST Peak Embeddings will not be recalculated.")
         nist_peak_embeddings = None

    if wavelengths is not None and spectrum is not None and nist_data and nist_data.get('data'):
        train_nn_button.disabled = False
    else:
        train_nn_button.disabled = True

    if embedding_model is not None and hasattr(embedding_model, 'inputs') and len(embedding_model.inputs) == 1 and nist_peak_embeddings is not None and len(nist_peak_embeddings) > 0:
         with neur_output:
              print("Enabling Element Identification Button.")
         element_identification_button.disabled = False
    else:
         with neur_output:
              print("Conditions for enabling Element Identification Button not met.")
              element_identification_button.disabled = True


def build_embedding_model(input_shape, embedding_dimension=64):
    """
    Baut das Neuronale Netz (Embedding-Modell) auf.

    Args:
        input_shape (tuple): Die Form der Eingabedaten (target_size, 1).
        embedding_dimension (int): Die Größe des Ausgabe-Embedding-Vektors.

    Returns:
        tf.keras.models.Model: Das aufgebaute Keras Modell.
    """
    # Eingangsschicht
    input_layer = Input(shape=input_shape, name='input_peak')

    # 1D Convolutional Layers
    # Diese Schichten lernen, lokale Muster in den Peak-Daten zu erkennen.
    x = Conv1D(filters=32, kernel_size=5, activation='relu', padding='causal')(input_layer)
    x = Conv1D(filters=64, kernel_size=5, activation='relu', padding='causal')(x)
    x = Conv1D(filters=128, kernel_size=5, activation='relu', padding='causal')(x)

    # Flatten Schicht: Wandelt die Ausgabe der Faltungsschichten in einen flachen Vektor um
    x = Flatten()(x)

    # Dense Layers (vollständig verbundene Schichten)
    # Diese Schichten lernen, die extrahierten Merkmale zu kombinieren.
    x = Dense(128, activation='relu')(x)

    # Embedding Layer: Die letzte Dense Schicht mit der gewünschten Embedding-Dimension
    # Hier verwenden wir keine Aktivierungsfunktion, um ein Embedding zu erhalten,
    # das jeden beliebigen Wert annehmen kann.
    embedding_output = Dense(embedding_dimension, name='embedding')(x)

    # Modell erstellen
    model = Model(inputs=input_layer, outputs=embedding_output)

    return model

# Globale Definition der Funktion zum Erstellen des Triplet-Trainingsmodells
# Modell aufbauen (nach den Imports)
input_shape = (TARGET_SIZE, 1)
embedding_model = build_embedding_model(input_shape, EMBEDDING_DIMENSION)
# Dieses Modell nimmt 3 Inputs (Anker, Positiv, Negativ) und gibt die konkatenierten Embeddings aus
def build_triplet_train_model(embedding_model):
    anchor_input = Input(shape=embedding_model.input_shape[1:])
    positive_input = Input(shape=embedding_model.input_shape[1:])
    negative_input = Input(shape=embedding_model.input_shape[1:])

    # Wende das Embedding-Modell auf jeden Input an
    anchor_embedding_output = embedding_model(anchor_input)
    positive_embedding_output = embedding_model(positive_input)
    negative_embedding_output = embedding_model(negative_input) # KORRIGIERTE DIESE ZEILE

    # Verwenden Sie eine Lambda-Schicht zur Konkatenierung (wie zuvor empfohlen)
    # Explizit output_shape angeben, um das Laden des Modells zu ermöglichen
    merged_output = Lambda(lambda x: tf.concat(x, axis=-1), output_shape=(3 * EMBEDDING_DIMENSION,))( # NEU: output_shape hinzugefügt
        [anchor_embedding_output, positive_embedding_output, negative_embedding_output]
    )

    model = Model(inputs=[anchor_input, positive_input, negative_input], outputs=merged_output)
    return model

def find_closest_nist_matches(real_peak_prepared_data, embedding_model, nist_peak_embeddings, num_matches=5):
    """
    Berechnet das Embedding für einen realen Peak und findet die ähnlichsten NIST-Peaks.

    Args:
        real_peak_prepared_data (np.ndarray): Die vorbereiteten Daten des realen Peaks.
                                              Shape: (target_size, 1)
        embedding_model (tf.keras.models.Model): Das trainierte Embedding-Modell.
        nist_peak_embeddings (list): Liste der berechneten NIST-Peak-Embeddings.
        num_matches (int): Die Anzahl der Top-Matches, die zurückgegeben werden sollen.

    Returns:
        list: Eine Liste von Dictionaries, die die Top-N NIST-Matches enthalten,
              sortiert nach Ähnlichkeit (absteigend).
              Jedes Dictionary enthält: 'element', 'stage', 'peakpoint', 'similarity'.
    """
    if embedding_model is None:
        print("Fehler: Embedding-Modell ist nicht trainiert oder geladen.")
        return []

    if not nist_peak_embeddings:
        print("Fehler: Keine NIST-Peak-Embeddings verfügbar.")
        return []

    # 1. Embedding für den realen Peak berechnen
    # Das Modell erwartet einen Batch, also fügen wir eine Batch-Dimension hinzu
    real_peak_data_batch = np.expand_dims(real_peak_prepared_data, axis=0) # Shape wird (1, target_size, 1)
    real_peak_embedding = embedding_model.predict(real_peak_data_batch)
    real_peak_embedding = real_peak_embedding[0] # Nehme das Embedding aus dem Batch

    # 2. Ähnlichkeit mit allen NIST-Embeddings berechnen
    similarities = []
    for nist_peak_info in nist_peak_embeddings:
        nist_embedding = nist_peak_info['embedding']

        # Ähnlichkeitsmetrik: Kosinus-Ähnlichkeit ist oft gut für Embeddings
        # Kosinus-Distanz = 1 - Kosinus-Ähnlichkeit
        # Eine niedrigere Kosinus-Distanz bedeutet höhere Ähnlichkeit.
        similarity = 1 - cosine(real_peak_embedding, nist_embedding)

        similarities.append({
            'element': nist_peak_info['element'],
            'stage': nist_peak_info['stage'],
            'peakpoint': nist_peak_info['peakpoint'],
            'similarity': similarity
        })

    # 3. Matches nach Ähnlichkeit sortieren (absteigend)
    similarities.sort(key=lambda x: x['similarity'], reverse=True)

    # 4. Top-N Matches zurückgeben
    return similarities[:num_matches]

def find_closest_nist_matches_hybrid(real_peak_prepared_data, real_peak_info, embedding_model, nist_peak_embeddings, num_matches=5, wavelength_tolerance=0.5, intensity_tolerance_factor=0.2):
    """
    Findet die ähnlichsten NIST-Peaks unter Berücksichtigung von Wellenlängen- und Intensitätsnähe.
    Die Form-Ähnlichkeit (Embedding) wird berechnet, aber nicht für das Ranking verwendet.

    Args:
        real_peak_prepared_data (...): Die vorbereiteten Daten des realen Peaks.
        real_peak_info (dict): Informationen zum realen Peak (inkl. 'peakpoint' und 'intensity').
        embedding_model (...): Das trainierte Embedding-Modell.
        nist_peak_embeddings (list): Liste der berechneten NIST-Peak-Embeddings.
        num_matches (int): Anzahl der Top-Matches.
        wavelength_tolerance (float): Maximale zulässige Wellenlängenabweichung in nm.
        intensity_tolerance_factor (float): Maximal zulässiger Intensitätsunterschied als Faktor der realen Peak-Intensität.

    Returns:
        list: Top-N Matches, sortiert nach Wellenlängen- und Intensitätsnähe, inkl. Form-Ähnlichkeit.
        Jedes Dictionary enthält: 'element', 'stage', 'peakpoint', 'similarity', 'wavelength_diff',
                                  'intensity_diff', 'score'.
    """
    if embedding_model is None or not nist_peak_embeddings:
        print("Fehler: Modell oder NIST-Embeddings nicht verfügbar.")
        return []

    real_peak_wavelength = real_peak_info['peakpoint']
    real_peak_intensity = real_peak_info['intensity']

    # 1. Embedding für den realen Peak berechnen (wird für 'similarity' benötigt)
    real_peak_data_batch = np.expand_dims(real_peak_prepared_data, axis=0)
    if real_peak_data_batch.ndim == 2:
         real_peak_data_batch = np.expand_dims(real_peak_data_batch, axis=-1)
    # KORREKTUR: Sicherstellen, dass hier das reine embedding_model verwendet wird
    # Explicitly use the embedding_model variable to make the prediction on a single input
    real_peak_embedding = embedding_model.predict(real_peak_data_batch, verbose=0)[0]


    potential_matches_with_scores = []

    for nist_peak_info in nist_peak_embeddings:
        nist_peak_wavelength = nist_peak_info['peakpoint']
        nist_peak_intensity = nist_peak_info.get('intensity', None)


        # 2. Wellenlängen- und Intensitätsprüfung und Score-Berechnung
        wavelength_diff = abs(real_peak_wavelength - nist_peak_wavelength)
        # Setze intensity_diff auf unendlich, wenn NIST-Intensität fehlt, damit diese Matches beim Ranking abfallen
        intensity_diff = abs(real_peak_intensity - nist_peak_intensity) if nist_peak_intensity is not None else float('inf')


        # Nur Peaks innerhalb der Wellenlängen-Toleranz und (falls Intensität vorhanden) innerhalb der Intensitäts-Toleranz berücksichtigen
        if wavelength_diff <= wavelength_tolerance and (nist_peak_intensity is None or intensity_diff <= real_peak_intensity * intensity_tolerance_factor):

             # 3. Embedding-Ähnlichkeit berechnen (nur zur Anzeige)
             embedding_similarity = 1 - cosine(real_peak_embedding, nist_peak_info.get('embedding', real_peak_embedding)) # Standardwert, falls Embedding fehlt

             # 4. Score basierend auf Wellenlängen- und Intensitätsnähe berechnen
             # Ziel: Höherer Score für geringere Wellenlängen- und Intensitätsdifferenz
             # Eine einfache Möglichkeit: Inverse der Differenzen oder eine kombinierte Metrik
             # Beispiel: score = 1 / (wavelength_diff + small_epsilon) + 1 / (intensity_diff + small_epsilon)
             # Besser: Normalisierte inverse Differenzen
             # Normalisiere die Differenzen (kleiner ist besser)
             normalized_wavelength_diff = wavelength_diff / wavelength_tolerance if wavelength_tolerance > 0 else 0
             # Max. Intensitäts-Diff ist real_peak_intensity * intensity_tolerance_factor
             max_intensity_diff_observed = real_peak_intensity * intensity_tolerance_factor if real_peak_intensity > 0 else 1.0 # Vermeide Division durch Null
             normalized_intensity_diff = intensity_diff / max_intensity_diff_observed if max_intensity_diff_observed > 0 and intensity_diff != float('inf') else (1.0 if intensity_diff != float('inf') else float('inf'))

             # Berechne einen Score, bei dem niedrigere normalisierte Differenzen zu einem höheren Score führen
             # Verwende eine kleine Konstante, um Division durch Null oder sehr kleine Zahlen zu vermeiden
             epsilon = 1e-6
             score = (1 - normalized_wavelength_diff) + (1 - normalized_intensity_diff) if intensity_diff != float('inf') else (1 - normalized_wavelength_diff) # Intensität nur einbeziehen, wenn verfügbar

             # Füge Match-Details zur Liste hinzu
             potential_matches_with_scores.append({
                'element': nist_peak_info['element'],
                'stage': nist_peak_info['stage'],
                'peakpoint': nist_peak_info['peakpoint'],
                'intensity': nist_peak_info.get('intensity', None), # Speichere None, wenn Intensität fehlt
                'similarity': embedding_similarity,  # Form-Ähnlichkeit beibehalten
                'wavelength_diff': wavelength_diff,
                'intensity_diff': intensity_diff if intensity_diff != float('inf') else None, # Speichere None, wenn Intensität fehlt
                'score': score # Der kombinierte Score
            })

    # Sortieren der potenziellen Matches basierend auf dem NEUEN kombinierten Score (absteigend)
    potential_matches_with_scores.sort(key=lambda x: x['score'], reverse=True)

    return potential_matches_with_scores[:num_matches]


def extract_and_prepare_peak_data(wavelengths, spectrum, peak_wavelength, window_size_nm=10, target_size=50):
    """
    Extrahiert, normalisiert und resampled Peak-Daten für das Embedding-Modell.

    Args:
        wavelengths (np.ndarray): Array der Wellenlängen des Gesamtspektrums.
        spectrum (np.ndarray): Array der Intensitäten des Gesamtspektrums.
        peak_wavelength (float): Die Wellenlänge des Peakmaximums (in nm).
        window_size_nm (float): Die Größe des Fensters um den Peak (in nm).
        target_size (int): Die Zielgröße (Anzahl der Datenpunkte) nach dem Resampling.

    Returns:
        np.ndarray: Das vorbereitete Peak-Daten-Array, bereit für das Embedding-Modell.
                    Shape: (target_size, 1)
    """
    # 1. Index des Peaks finden: Findet den Index, dessen Wellenlänge am nächsten
    #    an der peak_wavelength liegt.
    peak_index = np.abs(wavelengths - peak_wavelength).argmin()

    # 2. Fenster basierend auf Wellenlängenbereich bestimmen
    #    Findet die Indizes, die dem Wellenlängenbereich entsprechen.
    lower_bound_nm = peak_wavelength - window_size_nm / 2
    upper_bound_nm = peak_wavelength + window_size_nm / 2

    # Indizes für den Bereich finden
    start_index = np.abs(wavelengths - lower_bound_nm).argmin()
    end_index = np.abs(wavelengths - upper_bound_nm).argmin()

    # Stellen Sie sicher, dass start_index kleiner als end_index ist
    if start_index > end_index:
        start_index, end_index = end_index, start_index

    # Bereich extrahieren (inklusive end_index)
    peak_wavelengths_range = wavelengths[start_index : end_index + 1]
    peak_spectrum_range = spectrum[start_index : end_index + 1]

    # Prüfen, ob genügend Datenpunkte im Bereich sind
    if len(peak_spectrum_range) < 2:
        # Fall, wenn der Bereich zu klein ist, um zu interpolieren
        # Rückgabe eines leeren oder standardisierten Arrays je nach Bedarf des Modells
        # Hier geben wir ein Array von Nullen zurück, die richtige Größe.
        print(f"Warnung: Nicht genügend Datenpunkte im Bereich um {peak_wavelength:.2f} nm. Rückgabe von Nullen.")
        return np.zeros((target_size, 1), dtype=np.float32)


    # 3. Datenvorbereitung (Normalisierung und Resampling)
    #    a. Normalisierung: Skaliert die Intensitäten auf einen Bereich zwischen 0 und 1.
    min_intensity = np.min(peak_spectrum_range)
    max_intensity = np.max(peak_spectrum_range)

    if max_intensity - min_intensity == 0: # Vermeidet Division durch Null bei flacher Linie
        normalized_spectrum = np.zeros_like(peak_spectrum_range, dtype=np.float32)
    else:
        normalized_spectrum = (peak_spectrum_range - min_intensity) / (max_intensity - min_intensity)

    #    b. Resampling: Interpoliert die Daten auf die feste target_size.
    #       Wir verwenden np.linspace, um gleichmäßig verteilte Punkte im ursprünglichen Wellenlängenbereich zu erhalten.
    #       Die Interpolationsfunktion wird auf diese neuen Punkte angewendet.
    try:
        # Interpolation auf eine feste Anzahl von Punkten
        interpolation_function = interp1d(peak_wavelengths_range, normalized_spectrum, kind='linear')
        resampled_wavelengths = np.linspace(peak_wavelengths_range[0], peak_wavelengths_range[-1], target_size)
        resampled_spectrum = interpolation_function(resampled_wavelengths)
    except ValueError as e:
        print(f"Fehler beim Interpolieren des Peaks bei {peak_wavelength:.2f} nm: {e}. Rückgabe von Nullen.")
        return np.zeros((target_size, 1), dtype=np.float32)


    # 4. Form für das Modell anpassen: Keras Conv1D Schichten erwarten oft eine
    #    Form von (Batch-Größe, Steps, Channels). Für einen einzelnen Peak
    #    ohne Batch ist das (target_size, 1).
    prepared_data = np.expand_dims(resampled_spectrum, axis=-1) # Fügt die Channel-Dimension hinzu
    prepared_data = prepared_data.astype(np.float32) # Stellt sicher, dass der Datentyp float32 ist (oft für TF/Keras benötigt)

    return prepared_data

def prepare_all_detected_peaks(wavelengths, spectrum, detected_peaks, window_size_nm, target_size):
    """
    Bereitet die Daten für alle detektierten Peaks auf, indem ein Fenster extrahiert,
    resampelt und normalisiert wird. Speichert auch die ursprünglichen Peak-Infos.

    Args:
        wavelengths (np.ndarray): Wellenlängen-Array des Spektrums.
        spectrum (np.ndarray): Intensitäts-Array des Spektrums.
        detected_peaks (list): Liste von Dictionaries für detektierte Peaks
                               ([{'peakpoint': ..., 'intensity': ..., 'original_index': ...}, ...]).
                               Erwartet 'peakpoint' und 'intensity' und optional 'original_index'.
        window_size_nm (float): Die Fenstergröße in nm um jeden Peak.
        target_size (int): Die Zielgröße des resampelten Fensters.

    Returns:
        list: Eine Liste von Dictionaries, wobei jedes Dictionary die vorbereiteten Daten
              ('prepared_data') und die ursprünglichen Peak-Informationen ('original_peak_info') enthält.
              Gibt eine leere Liste zurück, wenn keine Peaks detektierten wurden oder ein Fehler auftritt.
    """
    global prepared_peaks_info # Stelle sicher, dass wir die globale Variable modifizieren
    prepared_peaks_info = [] # Leere die Liste am Anfang

    if not detected_peaks:
        print("Keine Peaks zum Vorbereiten vorhanden.")
        return [] # Gib eine leere Liste zurück

    for i, peak_info in enumerate(detected_peaks):
        peak_wavelength = peak_info.get('peakpoint') # Use .get() for safety
        peak_intensity = peak_info.get('intensity') # Use .get() for safety

        if peak_wavelength is None or peak_intensity is None:
            print(f"Warnung: Peak-Informationen für Peak {i} unvollständig (Wellenlänge oder Intensität fehlt). Überspringe Peak.")
            continue # Überspringe diesen Peak, wenn wichtige Infos fehlen

        # Speichere die ursprünglichen Peak-Infos
        # Stelle sicher, dass alle relevanten Informationen hier gespeichert werden
        original_peak_info = {
            'peakpoint': peak_wavelength,
            'intensity': peak_intensity,
            # Füge weitere originale Informationen hinzu, falls verfügbar und relevant
            # 'original_index': peak_info.get('original_index'), # Falls vorhanden - Dies ist der Index VOR dem Filtern, nicht der Index in detected_peaks
        }


        # Extrahiere und bereite das Fenster um den Peak vor
        prepared_data = extract_and_prepare_peak_data(
            wavelengths,
            spectrum,
            peak_wavelength,
            window_size_nm,
            target_size
        )

        if prepared_data is not None and prepared_data.size > 0:
            # Füge die vorbereiteten Daten und die ursprünglichen Infos zur Liste hinzu
            prepared_peaks_info.append({
                'original_peak_info': original_peak_info,
                'prepared_data': prepared_data,
                # Füge hier weitere Metadaten hinzu, falls nötig (z.B. original_index, etc.)
                'original_peak_index': i # Füge den Index in der 'detected_peaks' Liste (die gefilterten Peaks enthält) hinzu
            })
        else:
            print(f"Warnung: Vorbereitung der Daten für Peak bei {peak_wavelength:.2f} nm fehlgeschlagen oder ergab leere Daten. Überspringe Peak.")

    # Rückgabe der globalen Liste (optional, da die globale Variable direkt modifiziert wird)
    return prepared_peaks_info



def build_embedding_model(input_shape, embedding_dimension=64):
    """
    Baut das Neuronale Netz (Embedding-Modell) auf.

    Args:
        input_shape (tuple): Die Form der Eingabedaten (target_size, 1).
        embedding_dimension (int): Die Größe des Ausgabe-Embedding-Vektors.

    Returns:
        tf.keras.models.Model: Das aufgebaute Keras Modell.
    """
    # Eingangsschicht
    input_layer = Input(shape=input_shape, name='input_peak')

    # 1D Convolutional Layers
    # Diese Schichten lernen, lokale Muster in den Peak-Daten zu erkennen.
    x = Conv1D(filters=32, kernel_size=5, activation='relu', padding='causal')(input_layer)
    x = Conv1D(filters=64, kernel_size=5, activation='relu', padding='causal')(x)
    x = Conv1D(filters=128, kernel_size=5, activation='relu', padding='causal')(x)

    # Flatten Schicht: Wandelt die Ausgabe der Faltungsschichten in einen flachen Vektor um
    x = Flatten()(x)

    # Dense Layers (vollständig verbundene Schichten)
    # Diese Schichten lernen, die extrahierten Merkmale zu kombinieren.
    x = Dense(128, activation='relu')(x)

    # Embedding Layer: Die letzte Dense Schicht mit der gewünschten Embedding-Dimension
    # Hier verwenden wir keine Aktivierungsfunktion, um ein Embedding zu erhalten,
    # das jeden beliebigen Wert annehmen kann.
    embedding_output = Dense(embedding_dimension, name='embedding')(x)

    # Modell erstellen
    model = Model(inputs=input_layer, outputs=embedding_output)

    return model

# Globale Definition der Funktion zum Erstellen des Triplet-Trainingsmodells
# Modell aufbauen (nach den Imports)
input_shape = (TARGET_SIZE, 1)
embedding_model = build_embedding_model(input_shape, EMBEDDING_DIMENSION)
# Dieses Modell nimmt 3 Inputs (Anker, Positiv, Negativ) und gibt die konkatenierten Embeddings aus
def build_triplet_train_model(embedding_model):
    anchor_input = Input(shape=embedding_model.input_shape[1:])
    positive_input = Input(shape=embedding_model.input_shape[1:])
    negative_input = Input(shape=embedding_model.input_shape[1:])

    # Wende das Embedding-Modell auf jeden Input an
    anchor_embedding_output = embedding_model(anchor_input)
    positive_embedding_output = embedding_model(positive_input)
    negative_embedding_output = embedding_model(negative_input) # KORRIGIERTE DIESE ZEILE

    # Verwenden Sie eine Lambda-Schicht zur Konkatenierung (wie zuvor empfohlen)
    # Explizit output_shape angeben, um das Laden des Modells zu ermöglichen
    merged_output = Lambda(lambda x: tf.concat(x, axis=-1), output_shape=(3 * EMBEDDING_DIMENSION,))( # NEU: output_shape hinzugefügt
        [anchor_embedding_output, positive_embedding_output, negative_embedding_output]
    )

    model = Model(inputs=[anchor_input, positive_input, negative_input], outputs=merged_output)
    return model

def create_triplets_from_nist(nist_data, window_size_nm, target_size, num_triplets_per_element=100):
    """
    Erstellt Trainings-Tripel (Anker, Positiv, Negativ) aus den NIST-Daten.

    Args:
        nist_data (dict): Das geladene NIST-Daten-Dictionary.
        window_size_nm (float): Fenstergröße für die Peak-Datenvorbereitung in nm.
        target_size (int): Zielgröße nach dem Resampling.
        num_triplets_per_element (int): Anzahl der Tripel, die pro einzigartigem Element/Stage-Kombination
                                        in den NIST-Daten generiert werden sollen.

    Returns:
        tuple: Ein Tuple von NumPy-Arrays: (anker_data, positive_data, negative_data).
               Gibt None, None, None zurück, wenn keine Tripel erstellt werden können.
    """
    anker_data = []
    positive_data = []
    negative_data = []

    all_nist_peaks_for_triplets = [] # Liste zum Sammeln aller NIST-Peaks mit Wellenlängen/Spektrum-Daten

    # Iteriere über alle Elemente in den geladenen NIST-Daten
    for element, stages in nist_data['data'].items():
        for stage, df_stage in stages.items():
            if stage != "Sum" and not df_stage.empty:
                # Peaks in NIST-Daten finden (verwende find_peaks, die Intensität zurückgibt)
                # Verwende hier Standardparameter, da wir Peaks für das Training suchen, nicht für die Anzeige
                # Wichtig: find_peaks sollte die Peak-Daten als Liste von Dictionaries zurückgeben,
                # wobei jedes Dictionary 'peakpoint' und 'intensity' enthält.
                # KORREKTUR: Übergebe das vollständige nist_data Dictionary
                peak_indices_nist = find_peaks(df_stage['Intensität'].values, df_stage['Wellenlänge (nm)'].values, nist_data=nist_data, element=element, ionization_stage=stage)


                for peak_info in peak_indices_nist:
                    # Füge die Wellenlängen und das Spektrum des gesamten NIST-Datensatzes hinzu
                    # Diese werden für extract_and_prepare_peak_data benötigt
                    peak_info_with_spectrum = peak_info.copy() # Kopiere, um ursprüngliches dict nicht zu ändern
                    peak_info_with_spectrum['wavelengths'] = df_stage['Wellenlänge (nm)'].values # KORRIGIERTE DIESE ZEILE
                    peak_info_with_spectrum['spectrum'] = df_stage['Intensität'].values
                    peak_info_with_spectrum['element'] = element # Stelle sicher, dass Element und Stage dabei sind
                    peak_info_with_spectrum['stage'] = stage
                    all_nist_peaks_for_triplets.append(peak_info_with_spectrum)

    if not all_nist_peaks_for_triplets:
        print("Keine Peaks in den NIST-Daten zur Erstellung von Tripeln gefunden.")
        return None, None, None

    # Gruppieren Sie Peaks nach Element und Stage, um positive Beispiele einfacher zu finden
    peaks_by_element_stage = {}
    for peak_info in all_nist_peaks_for_triplets:
        key = (peak_info['element'], peak_info['stage'])
        if key not in peaks_by_element_stage:
            peaks_by_element_stage[key] = []
        peaks_by_element_stage[key].append(peak_info)

    # Debugging: Print Anzahl der Peaks pro Element/Stage-Kombination
    print("\nAnzahl der Peaks pro Element/Stage-Kombination:")
    for key, peaks in peaks_by_element_stage.items():
        print(f"  {key}: {len(peaks)} Peaks")


    # Entfernen Sie Element/Stage-Kombinationen, die weniger als 2 Peaks haben
    # (benötigt mindestens 2 für Anker und Positiv)
    keys_to_remove = [key for key, peaks in peaks_by_element_stage.items() if len(peaks) < 2]
    for key in keys_to_remove:
        print(f"Entferne {key} für Tripel-Erstellung (weniger als 2 Peaks).")
        del peaks_by_element_stage[key]

    unique_element_stages_keys = list(peaks_by_element_stage.keys())

    if not unique_element_stages_keys:
         print("Nicht genügend Peaks pro Element/Stage-Kombination zur Erstellung von positiven Tripeln.")
         return None, None, None


    # Begrenzen Sie die Anzahl der Versuche, um unendliche Schleifen bei wenigen Peaks zu vermeiden
    # Versuchen Sie, eine bestimmte Anzahl von Tripeln pro einzigartigem Element/Stage-Kombination zu generieren.
    total_triplets_to_generate = num_triplets_per_element * len(unique_element_stages_keys)
    max_attempts = total_triplets_to_generate * 10 # Erhöhe die Anzahl der Versuche

    generated_triplets_count = 0
    attempt_count = 0

    # Generieren Sie Tripel, bis die gewünschte Anzahl erreicht ist oder die maximale Anzahl von Versuchen überschritten ist
    while generated_triplets_count < total_triplets_to_generate and attempt_count < max_attempts:
        attempt_count += 1

        # Anker auswählen (zufälliger Peak aus einer gültigen Element/Stage-Kombination)
        anker_key = random.choice(unique_element_stages_keys)
        anker_options = peaks_by_element_stage[anker_key]
        if not anker_options: # Sollte nicht passieren, da wir leere entfernt haben, aber zur Sicherheit
             continue
        anker = random.choice(anker_options)

        # Positives Beispiel auswählen (anderer Peak vom selben Element/Stage)
        possible_positives = [p for p in anker_options if p != anker]
        if not possible_positives:
            continue # Überspringe, wenn kein positives Beispiel gefunden wird (sollte nicht passieren, wenn len(peaks) >= 2)

        positive = random.choice(possible_positives)


        # Negatives Beispiel auswählen (Peak von einem anderen Element/Stage)
        # Wählen Sie zuerst eine andere Element/Stage-Kombination
        other_keys = [key for key in unique_element_stages_keys if key != anker_key]
        if not other_keys:
             # Wenn es nur eine Element/Stage-Kombination gibt, können wir keine negativen Beispiele
             # von ANDEREN Element/Stage-Kombinationen finden. In diesem Fall können wir keine Tripel erstellen.
             continue


        negative_key = random.choice(other_keys)
        possible_negatives = peaks_by_element_stage[negative_key] # Alle Peaks aus einer anderen Kombination

        if not possible_negatives:
             continue # Überspringe, wenn kein negatives Beispiel gefunden wird

        negative = random.choice(possible_negatives)


        # Sicherstellen, dass alle benötigten Felder vorhanden sind, bevor extract_and_prepare_peak_data aufgerufen wird
        if ('wavelengths' in anker and 'spectrum' in anker and 'peakpoint' in anker and
            'wavelengths' in positive and 'spectrum' in positive and 'peakpoint' in positive and
            'wavelengths' in negative and 'spectrum' in negative and 'peakpoint' in negative):

            # Daten für die Tripel vorbereiten
            try:
                anker_prepared = extract_and_prepare_peak_data(anker['wavelengths'], anker['spectrum'], anker['peakpoint'], window_size_nm, target_size)
                positive_prepared = extract_and_prepare_peak_data(positive['wavelengths'], positive['spectrum'], positive['peakpoint'], window_size_nm, target_size)
                negative_prepared = extract_and_prepare_peak_data(negative['wavelengths'], negative['spectrum'], negative['peakpoint'], window_size_nm, target_size)

                 # Prüfen, ob die Datenvorbereitung erfolgreich war (nicht None und richtige Form)
                if (anker_prepared is not None and positive_prepared is not None and negative_prepared is not None and
                     anker_prepared.shape == (target_size, 1) and positive_prepared.shape == (target_size, 1) and negative_prepared.shape == (target_size, 1)):

                     anker_data.append(anker_prepared)
                     positive_data.append(positive_prepared)
                     negative_data.append(negative_prepared)
                     generated_triplets_count += 1
                else:
                    # Optional: Warnung ausgeben, wenn Datenvorbereitung fehlschlägt
                    # print("Warnung: Datenvorbereitung für ein Tripel fehlgeschlagen.")
                    pass # Einfach überspringen

            except Exception as e:
                 # Optional: Fehler beim Datenvorbereitung protokollieren
                 # print(f"Fehler bei Datenvorbereitung für ein Tripel: {e}")
                 pass # Einfach überspringen

        else:
            # Optional: Warnung ausgeben, wenn benötigte Felder in Peak-Info fehlen
            # print("Warnung: Benötigte Felder in Peak-Info für Tripel fehlen.")
            pass # Überspringen, wenn benötigte Felder fehlen


    # In NumPy-Arrays konvertieren
    # Sicherstellen, dass die Listen nicht leer sind, bevor np.array aufgerufen wird
    # Geben Sie None zurück, wenn keine Tripel generiert wurden, anstatt leere Arrays
    if anker_data:
        anker_data_np = np.array(anker_data)
        positive_data_np = np.array(positive_data)
        negative_data_np = np.array(negative_data)
        print(f"Erfolgreich {anker_data_np.shape[0]} Tripel generiert.")
        return anker_data_np, positive_data_np, negative_data_np
    else:
        print("Konnte keine Tripel generieren.")
        return None, None, None

def train_embedding_model(model, anker_data, positive_data, negative_data, X_val, y_val, epochs=20, batch_size=32, learning_rate=0.001): # Removed model_save_path
    """
    Trainiert das Embedding-Modell mit Triplet Loss.

    Args:
        model (tf.keras.models.Model): Das zu trainierende Modell (hier das embedding_model).
        anker_data (np.ndarray): Vorbereitete Daten für die Anker.
        positive_data (np.ndarray): Vorbereitete Daten für die positiven Beispiele.
        negative_data (np.ndarray): Vorbereitete Daten für die negativen Beispiele.
        X_val (np.ndarray): Validierungsdaten.
        y_val (np.ndarray): Validierungs-Labels (für Triplet Loss ist dies oft nicht direkt relevant).
        epochs (int): Anzahl der Trainingsepochen.
        batch_size (int): Die Batch-Größe für das Training.
        learning_rate (float): Die Lernrate für den Optimizer.
    """
    with neur_output:
        clear_output(wait=True)
        print("Training des Embedding-Modells gestartet...")

        # Optimizer mit der übergebenen Lernrate erstellen
        optimizer = Adam(learning_rate=learning_rate)

        # Erstellen Sie das Triplet Trainingsmodell, das die drei Inputs nimmt
        triplet_train_model = build_triplet_train_model(model) # Verwenden Sie das übergebene embedding_model

        # Das Modell kompilieren: Definieren Sie den Optimierer und die Verlustfunktion
        # Stellen Sie sicher, dass triplet_semi_hard_loss definiert ist
        triplet_train_model.compile(optimizer=optimizer, loss=triplet_semi_hard_loss) # <-- Verwenden Sie Ihre globale Loss-Funktion

        # Trainingsdaten zusammenfügen: Die Daten müssen in der Reihenfolge Anker, Positiv, Negativ sein
        # Stellen Sie sicher, dass anker_data, positive_data, negative_data NumPy-Arrays sind.
        # Concatenation expects a list of tensors/arrays.
        training_data = [anker_data, positive_data, negative_data]


        # Dummy-Labels (y_true) für das Training. Die Größe muss der Anzahl der Trainingsbeispiele entsprechen.
        # Für Triplet Loss ist der tatsächliche Inhalt dieser Labels irrelevant, nur die Größe ist wichtig.
        # Die Anzahl der Trainingsbeispiele ist die Batch-Größe pro Triplet-Element.
        dummy_labels = np.zeros((anker_data.shape[0], 1)) # Shape sollte (num_triplets, 1) sein


        callbacks = [TqdmCallback(verbose=1)] # TqdmCallback immer hinzufügen

        # ModelCheckpoint Callback entfernen, da das Modell am Ende manuell gespeichert wird
        # if model_save_path:
        #     model_checkpoint_callback = ModelCheckpoint(
        #         filepath=model_save_path,
        #         monitor='val_loss',
        #         save_best_only=True,
        #         mode='min',
        #         verbose=1
        #     )
        #     callbacks.append(model_checkpoint_callback)


        # Modell trainieren
        # Passen Sie die Eingabe an model.fit an, um die drei separaten Arrays zu übergeben
        # und die Batch-Größe korrekt zu setzen.
        history = triplet_train_model.fit( # Trainiere das Triplet-Modell
          training_data, # Liste der drei Arrays
          dummy_labels, # Dummy-Labels
          batch_size=batch_size, # Batch-Größe für das Triplet-Modell
          epochs=epochs,
          verbose=0, # Setze verbose=0, da TqdmCallback die Fortschrittsanzeige übernimmt
          callbacks=callbacks,
          validation_data=(X_val, y_val) # Validierungsdaten übergeben
        )

        with neur_output:
            print("Training abgeschlossen.")

    # Das beste Modell wird hier NICHT automatisch gespeichert.
    # Es muss nach dem Aufruf dieser Funktion manuell gespeichert werden.

    # Rückgabe des trainierten Modells (optional)
    return model


def calculate_and_store_nist_embeddings(nist_data, embedding_model, window_size_nm, target_size, embeddings_save_path):
    """
    Lädt NIST-Peak-Embeddings, falls vorhanden, andernfalls berechnet sie neu
    und speichert sie dann. Die Embeddings werden immer für die Elemente berechnet,
    die in 'nist_data' geladen sind.

    Args:
        nist_data (dict): Das geladene NIST-Daten-Dictionary.
        embedding_model (tf.keras.models.Model): Das trainierte Embedding-Modell.
        window_size_nm (float): Fenstergröße für die Peak-Datenvorbereitung in nm.
        target_size (int): Zielgröße nach dem Resampling.
        embeddings_save_path (str): Der vollständige Pfad zum Speichern/Laden der Embeddings-Datei.
    """
    global nist_peak_embeddings # Zugriff auf die globale Variable

    with neur_output:
        print("Starte calculate_and_store_nist_embeddings Funktion.")

    # Erstelle den Speicherordner, falls er nicht existiert
    embedding_dir = os.path.dirname(embeddings_save_path)
    if not os.path.exists(embedding_dir):
        os.makedirs(embedding_dir)
        with neur_output:
             print(f"Erstellt Speicherordner für Embeddings: {embedding_dir}")


    if os.path.exists(embeddings_save_path):
        with neur_output:
            print(f"Versuche NIST-Peak-Embeddings zu laden von: {embeddings_save_path}")
        try:
            # Embeddings laden, wenn die Datei existiert
            # BEWARE: Using allow_pickle=True is not recommended for loading data from untrusted sources.
            nist_peak_embeddings = np.load(embeddings_save_path, allow_pickle=True).tolist() # .tolist() konvertiert das Array von Objekten zurück in eine Liste
            with neur_output:
                print(f"NIST-Peak-Embeddings erfolgreich geladen. Anzahl: {len(nist_peak_embeddings)}")

        except Exception as e:
            with neur_output:
                print(f"Fehler beim Laden der NIST-Peak-Embeddings von {embeddings_save_path}: {e}")
                print("NIST-Peak-Embeddings werden neu berechnet...")

            # Fehler beim Laden oder Datei nicht gefunden -> neu berechnen
            nist_peak_embeddings = [] # Liste leeren für neue Embeddings

            with neur_output:
                 print("Starte Sammlung und Verarbeitung aller NIST-Peak-Instanzen für Neuberechnung.")
            # Alle einzelnen NIST-Peak-Instanzen sammeln aus den geladenen NIST-Daten
            # Iteriere über die Elemente, die tatsächlich in nist_data['data'] vorhanden sind
            for element in nist_data.get('data', {}).keys():
                stages = nist_data['data'][element] # Greife direkt auf die Datenstruktur zu
                #with neur_output:
                     #print(f"Verarbeite Element: {element}")
                for stage, df_stage in stages.items():
                    if stage != "Sum" and not df_stage.empty:
                        #with neur_output:
                             #print(f"  Verarbeite Ionisationsstufe: {stage}")
                        # Finde Peak-Indizes in den NIST-Daten
                        peak_indices_nist = find_peaks(df_stage['Intensität'].values, df_stage['Wellenlänge (nm)'].values, nist_data=nist_data, element=element, ionization_stage=stage)

                        #with neur_output:
                             #print(f"    find_peaks für {element} {stage} gefunden: {len(peak_indices_nist)} Peaks.")
                        for peak_info in peak_indices_nist:
                             # Übergeben Sie die gesamten Wellenlängen/Spektrum des NIST-Datensatzes
                             # an extract_and_prepare_peak_data
                            prepared_data = extract_and_prepare_peak_data(
                                df_stage['Wellenlänge (nm)'].values,
                                df_stage['Intensität'].values,
                                peak_info['peakpoint'],
                                window_size_nm,
                                target_size
                            )
                            # Debugging: Überprüfen Sie die vorbereiteten Daten
                            # with neur_output:
                            #     print(f"      Daten vorbereitet für Peak bei {peak_info['peakpoint']:.2f} nm. Shape: {prepared_data.shape}, Sum: {np.sum(prepared_data):.2f}")


                            # Nur fortfahren, wenn die Datenvorbereitung erfolgreich war (Summe > 0, um Nullen zu vermeiden)
                            # Prüfen Sie auch auf None oder eine leere Form
                            if prepared_data is not None and prepared_data.size > 0 and np.sum(prepared_data) > 0:
                                # Embedding berechnen
                                #with neur_output:
                                     #print("      Berechne Embedding...")
                                # Das Modell erwartet einen Batch, also fügen wir eine Batch-Dimension hinzu
                                prepared_data_batch = np.expand_dims(prepared_data, axis=0) # Shape wird (1, target_size, 1)
                                # Stellen Sie sicher, dass das embedding_model nicht None ist, bevor Sie predict aufrufen
                                if embedding_model is not None:
                                     try:
                                          # Use the embedding_model directly which expects a single input
                                          embedding = embedding_model.predict(prepared_data_batch, verbose=0) # verbose=0, um Ausgabe während predict zu vermeiden
                                          #with neur_output:
                                               #print(f"      Embedding berechnet. Shape: {embedding.shape}")

                                          # Embedding speichern zusammen mit den Peak-Informationen
                                          nist_peak_embeddings.append({
                                              'element': element,
                                              'stage': stage,
                                              'peakpoint': peak_info['peakpoint'],
                                              'intensity': peak_info.get('intensity', None), # Optional: Intensität hinzufügen, falls in find_peaks vorhanden
                                              'embedding': embedding[0] # embedding_model.predict gibt Batch zurück, wir nehmen das erste (und einzige) Element
                                          })
                                          #with neur_output:
                                               #print(f"      Embedding für Peak bei {peak_info['peakpoint']:.2f} nm hinzugefügt. Aktuelle Anzahl Embeddings: {len(nist_peak_embeddings)}")
                                     except Exception as predict_e:
                                          with neur_output:
                                               print(f"      Fehler bei der Embedding-Berechnung für Peak bei {peak_info['peakpoint']:.2f} nm: {predict_e}. Überspringe diesen Peak.")

                                else:
                                     with neur_output:
                                          print("      Embedding-Modell ist None. Kann kein Embedding berechnen.")


                            else:
                                 with neur_output:
                                      print(f"      Datenvorbereitung für Peak bei {peak_info['peakpoint']:.2f} nm fehlgeschlagen oder Summe ist 0. Überspringe Embedding-Berechnung.")


            # Nach der Berechnung: Embeddings speichern
            with neur_output:
                 print("Alle NIST-Peak-Instanzen verarbeitet. Versuche Embeddings zu speichern.")
            try:
                 # BEWARE: Using allow_pickle=True is not recommended for loading data from untrusted sources.
                 np.save(embeddings_save_path, nist_peak_embeddings, allow_pickle=True)
                 with neur_output:
                     print(f"{len(nist_peak_embeddings)} NIST-Peak-Embeddings neu berechnet und erfolgreich gespeichert unter: {embeddings_save_path}")
            except Exception as e:
                 with neur_output:
                      print(f"Fehler beim Speichern der neu berechneten NIST-Peak-Embeddings: {e}")

    else:
        # Wenn die Datei nicht existiert, berechnen Sie die Embeddings
        with neur_output:
            clear_output(wait=True) # Output leeren, bevor neue Nachrichten erscheinen
            print("NIST-Peak-Embeddings-Datei nicht gefunden. Embeddings werden berechnet und gespeichert.")

        nist_peak_embeddings = [] # Liste leeren für neue Embeddings
        with neur_output:
             print("Starte Sammlung und Verarbeitung aller NIST-Peak-Instanzen.")
        # Alle einzelnen NIST-Peak-Instanzen sammeln aus den geladenen NIST-Daten
        # Iteriere über die Elemente, die tatsächlich in nist_data['data'] vorhanden sind
        elements_to_process = nist_data.get('data', {}).keys()
        if not elements_to_process:
             with neur_output:
                  print("Keine NIST-Daten zum Verarbeiten verfügbar.")
             nist_peak_embeddings = None # Setzen Sie die Variable auf None, wenn keine Daten da sind
             return # Beenden Sie die Funktion, wenn keine Daten da sind

        for element in elements_to_process:
            stages = nist_data['data'][element] # Greife direkt auf die Datenstruktur zu
            #with neur_output:
                 #print(f"Verarbeite Element: {element}")
            for stage, df_stage in stages.items():
                if stage != "Sum" and not df_stage.empty:
                    #with neur_output:
                         #print(f"  Verarbeite Ionisationsstufe: {stage}")
                    # Finde Peak-Indizes in den NIST-Daten
                    peak_indices_nist = find_peaks(df_stage['Intensität'].values, df_stage['Wellenlänge (nm)'].values, nist_data=nist_data, element=element, ionization_stage=stage)
                    #with neur_output:
                         #print(f"    find_peaks für {element} {stage} gefunden: {len(peak_indices_nist)} Peaks.")

                    for peak_info in peak_indices_nist:
                         # Übergeben Sie die gesamten Wellenlängen/Spektrum des NIST-Datensatzes
                         # an extract_and_prepare_peak_data
                        prepared_data = extract_and_prepare_peak_data(
                            df_stage['Wellenlänge (nm)'].values,
                            df_stage['Intensität'].values,
                            peak_info['peakpoint'],
                            window_size_nm,
                            target_size
                        )
                        # Debugging: Überprüfen Sie die vorbereiteten Daten
                        # with neur_output:
                        #      print(f"      Daten vorbereitet für Peak bei {peak_info['peakpoint']:.2f} nm. Shape: {prepared_data.shape}, Sum: {np.sum(prepared_data):.2f}")


                        # Nur fortfahren, wenn die Datenvorbereitung erfolgreich war (Summe > 0)
                        # Prüfen Sie auch auf None oder eine leere Form
                        if prepared_data is not None and prepared_data.size > 0 and np.sum(prepared_data) > 0:
                            # Embedding berechnen
                            #with neur_output:
                                 #print("      Berechne Embedding...")
                            # Das Modell erwartet einen Batch, also fügen wir eine Batch-Dimension hinzu
                            prepared_data_batch = np.expand_dims(prepared_data, axis=0) # Shape wird (1, target_size, 1)
                            # Stellen Sie sicher, dass das embedding_model nicht None ist, bevor Sie predict aufrufen
                            if embedding_model is not None:
                                 try:
                                      # Use the embedding_model directly which expects a single input
                                      embedding = embedding_model.predict(prepared_data_batch, verbose=0) # verbose=0
                                      #with neur_output:
                                           #print(f"      Embedding berechnet. Shape: {embedding.shape}")


                                      # Embedding speichern zusammen mit den Peak-Informationen
                                      nist_peak_embeddings.append({
                                          'element': element,
                                          'stage': stage,
                                          'peakpoint': peak_info['peakpoint'],
                                          'intensity': peak_info.get('intensity', None), # Optional: Intensität hinzufügen, falls in find_peaks vorhanden
                                          'embedding': embedding[0] # embedding_model.predict gibt Batch zurück, wir nehmen das erste (und einzige) Element
                                      })
                                      #with neur_output:
                                           #print(f"      Embedding für Peak bei {peak_info['peakpoint']:.2f} nm hinzugefügt. Aktuelle Anzahl Embeddings: {len(nist_peak_embeddings)}")
                                 except Exception as predict_e:
                                      with neur_output:
                                           print(f"      Fehler bei der Embedding-Berechnung für Peak bei {peak_info['peakpoint']:.2f} nm: {predict_e}. Überspringe diesen Peak.")
                            else:
                                 with neur_output:
                                      print("      Embedding-Modell ist None. Kann kein Embedding berechnen.")

                        else:
                             with neur_output:
                                  print(f"      Datenvorbereitung für Peak bei {peak_info['peakpoint']:.2f} nm fehlgeschlagen oder Summe ist 0. Überspringe Embedding-Berechnung.")


        # Nach der Berechnung: Embeddings speichern
        try:
             # BEWARE: Using allow_pickle=True is not recommended for loading data from untrusted sources.
             np.save(embeddings_save_path, nist_peak_embeddings, allow_pickle=True)
             with neur_output:
                 print(f"{len(nist_peak_embeddings)} NIST-Peak-Embeddings berechnet und erfolgreich gespeichert unter: {embeddings_save_path}")
        except Exception as e:
             with neur_output:
                  print(f"Fehler beim Speichern der NIST-Peak-Embeddings: {e}")

    with neur_output:
        print("Ende calculate_and_store_nist_embeddings Funktion.")


def triplet_semi_hard_loss(y_true, y_pred, margin=1.0):
    """
    Implementiert den Triplet Semi-Hard Loss für konkatenierte Embeddings.

    Args:
        y_true: Die Labels. Wird in dieser Implementierung nicht direkt verwendet.
        y_pred: Die konkatenierten Embeddings [Anker_Embeddings, Positiv_Embeddings, Negativ_Embeddings].
                Shape: (Batch_Size, 3 * EMBEDDING_DIMENSION).
        margin: Der Margin für den Triplet Loss.

    Returns:
        Der berechnete Triplet Loss.
    """
    # EMBEDDING_DIMENSION sollte hier global verfügbar sein
    # oder aus dem Shape von y_pred abgeleitet werden:
    # embedding_dimension = K.int_shape(y_pred)[1] // 3 # Originale Ableitung

    # Annahme: Die ersten embedding_dimension Spalten sind Anker-Embeddings
    # Leite die embedding_dimension direkt aus der Form des Ankers ab
    embedding_dimension = K.int_shape(y_pred)[1] // 3 # Use the original calculation as a fallback

    # Explicitly split based on the calculated embedding_dimension
    anchor = y_pred[:, :embedding_dimension]
    positive = y_pred[:, embedding_dimension:2*embedding_dimension]
    negative = y_pred[:, 2*embedding_dimension:]

    # Optional: Add a check here to see the shapes after splitting
    # tf.print("Anchor shape:", tf.shape(anchor))
    # tf.print("Positive shape:", tf.shape(positive))
    # tf.print("Negative shape:", tf.shape(negative))

    # Abstände berechnen (quadrierter Euklidischer Abstand)
    pos_dist = K.sum(K.square(anchor - positive), axis=1)
    neg_dist = K.sum(K.square(anchor - negative), axis=1)

    # Triplet-Loss berechnen
    loss = K.maximum(0.0, pos_dist - neg_dist + margin)

    return K.mean(loss)

In [12]:
# @title Buttons

def mark_match_button_clicked(b):
    global identified_matches, current_match_index, validated_matches, filtered_peaks, aktueller_index, prepared_peaks_info
    with neur_output:
        clear_output(wait=True)
        print("DEBUG: mark_match_button_clicked started.")
        print(f"DEBUG: aktueller_index: {aktueller_index}")
        print(f"DEBUG: len(identified_matches): {len(identified_matches) if identified_matches else 0}")
        print(f"DEBUG: current_match_index: {current_match_index}")
        print(f"DEBUG: len(validated_matches): {len(validated_matches)}")
        print(f"DEBUG: prepared_peaks_info is None: {prepared_peaks_info is None}")
        if prepared_peaks_info:
            print(f"DEBUG: len(prepared_peaks_info): {len(prepared_peaks_info)}")


        if not identified_matches or not (0 <= current_match_index < len(identified_matches)):
            print("Kein Match zum Markieren vorhanden.")
            print("DEBUG: No match to mark.")
            print("DEBUG: mark_match_button_clicked finished.")
            return

        current_match = identified_matches[current_match_index]
        print(f"DEBUG: current_match element: {current_match.get('element')}, wavelength: {current_match.get('peakpoint')}")


        # Find the real peak info and prepared data from prepared_peaks_info using aktueller_index
        real_peak_info_for_validation = None
        real_peak_prepared_data_for_validation = None

        # Assuming prepared_peaks_info is ordered corresponding to filtered_peaks by index
        if prepared_peaks_info is not None and 0 <= aktueller_index < len(prepared_peaks_info):
             selected_prepared_item = prepared_peaks_info[aktueller_index]
             real_peak_info_for_validation = selected_prepared_item.get('original_peak_info') # Get original peak info dictionary
             real_peak_prepared_data_for_validation = selected_prepared_item.get('prepared_data') # Get prepared data array
             print(f"DEBUG: Accessed prepared_peaks_info at index {aktueller_index}.")
             if real_peak_info_for_validation:
                  print(f"DEBUG: real_peak_info_for_validation peakpoint: {real_peak_info_for_validation.get('peakpoint')}")
             else:
                  print("DEBUG: real_peak_info_for_validation is None.")
             if real_peak_prepared_data_for_validation is not None:
                  print(f"DEBUG: real_peak_prepared_data_for_validation shape: {real_peak_prepared_data_for_validation.shape}")
             else:
                  print("DEBUG: real_peak_prepared_data_for_validation is None.")


        # Add a check to see if the peakpoint from real_peak_info_for_validation matches the one from filtered_peaks as expected
        peakpoint_from_filtered = None
        if isinstance(filtered_peaks, dict) and aktueller_index in filtered_peaks:
             peakpoint_from_filtered = filtered_peaks[aktueller_index].get('peakpoint')
        elif isinstance(filtered_peaks, list) and 0 <= aktueller_index < len(filtered_peaks):
             peakpoint_from_filtered = filtered_peaks[aktueller_index].get('peakpoint')

        if real_peak_info_for_validation and peakpoint_from_filtered is not None:
             tolerance = 1e-5 # Use tolerance for verification
             if abs(float(real_peak_info_for_validation.get('peakpoint', -1)) - float(peakpoint_from_filtered)) > tolerance:
                  print(f"DEBUG: WARNING: Peakpoint mismatch between prepared_peaks_info ({real_peak_info_for_validation.get('peakpoint')}) and filtered_peaks ({peakpoint_from_filtered}) at index {aktueller_index}.")
                  # If mismatch, maybe revert to searching? Or indicate error?
                  # For now, proceed but warn. If this warning appears, the direct access by index is unreliable.
             else:
                  print(f"DEBUG: Peakpoint match verified between prepared_peaks_info and filtered_peaks at index {aktueller_index}.")
        elif real_peak_info_for_validation is None and peakpoint_from_filtered is not None:
             print(f"DEBUG: WARNING: Could not get real_peak_info_for_validation but filtered_peaks[{aktueller_index}] has peakpoint {peakpoint_from_filtered}.")


        if real_peak_info_for_validation is not None and real_peak_prepared_data_for_validation is not None:
             print("DEBUG: real_peak_info_for_validation and real_peak_prepared_data_for_validation found (via index).")
             validated_match_data = {
                 # Store the real peak data from the spectrum (using .get() for safety as in current code)
                 'real_peak_wavelength': real_peak_info_for_validation.get('peakpoint'),
                 'real_peak_intensity': real_peak_info_for_validation.get('intensity'),
                 'real_peak_prepared_data': real_peak_prepared_data_for_validation, # Store the prepared data of the real peak
                 # Store the assigned NIST data (using .get() as in current code)
                 'nist_element': current_match.get('element'),
                 'nist_stage': current_match.get('stage'),
                 'nist_wavelength': current_match.get('peakpoint'),
                 'nist_intensity': current_match.get('intensity'),
                 'match_similarity': current_match.get('similarity'), # Embedding-Ähnlichkeit
                 'match_score': current_match.get('score'), # Kombinierter Score
                 'validation_timestamp': pd.Timestamp.now().isoformat() # Zeitstempel der Validierung
             }
             print(f"DEBUG: validated_match_data created: {validated_match_data}")


             # Check if an existing validated match for this real peak exists and replace it
             existing_validated_index = -1
             # Use the real_peak_wavelength from the validated_match_data for comparison (using .get() for safety)
             real_peak_wl_to_check = validated_match_data.get('real_peak_wavelength')
             if real_peak_wl_to_check is not None: # Still check for None just in case
                  print(f"DEBUG: Checking for existing validated match for real_peak_wavelength close to: {real_peak_wl_to_check}")
                  tolerance = 1e-5 # Use same tolerance here
                  for i, vm in enumerate(validated_matches):
                       # Use .get() here for safety when checking existing validated matches
                       vm_real_peak_wl = vm.get('real_peak_wavelength')
                       # Use tolerance for comparison
                       if vm_real_peak_wl is not None and abs(float(vm_real_peak_wl) - float(real_peak_wl_to_check)) < tolerance: # Explicitly cast to float and use tolerance
                            existing_validated_index = i
                            print(f"DEBUG: Existing validated match found at index {i} with wavelength {vm_real_peak_wl}")
                            break

             if existing_validated_index != -1:
                  removed_match = validated_matches.pop(existing_validated_index)
                  # Use .get() for printing removed match details for safety
                  print(f"Vorheriger validierter Match für realen Peak bei {validated_match_data.get('real_peak_wavelength', 'N/A'):.2f} nm entfernt: {removed_match.get('nist_element', 'N/A')} ({removed_match.get('nist_stage', 'N/A')}) bei {removed_match.get('nist_wavelength', 'N/A'):.2f} nm.")


             # Add the new match
             # Add a check to prevent adding duplicate if user clicks again on the same match
             # Use .get() for safety when checking existing validated matches and tolerance for wavelength comparison
             tolerance = 1e-5 # Use same tolerance here
             is_already_validated = any(
                  vm.get('real_peak_wavelength') is not None and validated_match_data.get('real_peak_wavelength') is not None and abs(float(vm.get('real_peak_wavelength')) - float(validated_match_data.get('real_peak_wavelength'))) < tolerance and
                  vm.get('nist_wavelength') == validated_match_data.get('nist_wavelength') and
                  vm.get('nist_element') == validated_match_data.get('nist_element') and
                  vm.get('nist_stage') == validated_match_data.get('nist_stage')
                  for vm in validated_matches
             )
             print(f"DEBUG: is_already_validated: {is_already_validated}")


             if not is_already_validated:
                 validated_matches.append(validated_match_data)
                 # Use .get() for consistency with how real_peak_info_for_validation was populated
                 print(f"Match als validiert markiert: {current_match.get('element', 'N/A')} ({current_match.get('stage', 'N/A')}) bei {current_match.get('peakpoint', 'N/A'):.2f} nm für realen Peak bei {real_peak_info_for_validation.get('peakpoint', 'N/A'):.2f} nm.")
             else:
                  print("DEBUG: Current match is already in validated_matches. Not adding again.")


             print(f"Aktuelle Anzahl validierter Matches: {len(validated_matches)}")


             # After marking, update the display of the current match to show the checkmark
             display_current_match()
             update_match_navigation_state() # Update button states


        else:
             print("Fehler: Informationen oder vorbereitete Daten für den aktuellen realen Peak nicht gefunden oder unvollständig.")
             print("DEBUG: real_peak_info_for_validation or real_peak_prepared_data_for_validation is None (after attempting index access).")
             # Ensure match navigation state is updated even on error
             update_match_navigation_state()


        print("DEBUG: mark_match_button_clicked finished.")


def file_button_clicked(b):
    global file_path, wavelengths, spectrum, identified_peaks, filtered_peaks, noise_std_calculated # noise_std_calculated hinzufügen
    # Textausgabe im allgemeinen Output
    print("Loading spectrum...")

    # clear_output(wait=True) # Dies würde den allgemeinen Output leeren, was nicht gewünscht ist
    if drive_selection_widget.value == 'Google Drive' and not os.path.exists('/content/drive'):
        print("Google Drive wird eingebunden")  # Debug-Print
        drive.mount('/content/drive')

    if drive_selection_widget.value == 'Google Drive':
        file_path = file_path_widget.value
        if not os.path.exists(file_path):
            print(f"Error: File not found at {file_path}")
            return
    # else: # Lokale Datei - Entferne das unnötige 'else' hier
    elif drive_selection_widget.value == 'Local File': # Explizit auf 'Local File' prüfen
        if not file_upload_widget.value:
            print("Error: No local file selected.")
            return
        uploaded_file = list(file_upload_widget.value.values())[0]
        file_path = uploaded_file['name']
        with open(file_path, 'wb') as f:
            f.write(uploaded_file['content'])
    else: # Fallback für andere mögliche Werte, obwohl nur 2 erwartet werden
         print("Error: Invalid drive selection.")
         return


    try:
        wavelengths, spectrum = read_spectrum(file_path)
        # Rauschbereich definieren (Beispiel: erster und letzter 5% des Spektrums)
        noise_region_start = int(len(spectrum) * 0.05)
        noise_region_end = int(len(spectrum) * 0.95)
        # Sicherstellen, dass noise_region_start kleiner als noise_region_end ist
        if noise_region_start >= noise_region_end:
            # Fallback: Verwenden Sie den gesamten Bereich, wenn die Berechnung fehlschlägt
            noise_region_start = 0
            noise_region_end = len(spectrum) - 1

        noise_std_calculated = calculate_noise(spectrum, (noise_region_start, noise_region_end))  # Berechne das Rauschen
        noise_std_slider.value = noise_std_calculated # Setze den Slider-Wert

        # Direkt nach dem Laden das Spektrum plotten (ohne Peaks) im output_spectrum Widget
        with output_spectrum:
            clear_output(wait=True) # Leeren Sie nur das spectrum_output Widget
            plot_spectrum(wavelengths, spectrum, peak_indices=None, noise_std=noise_std_calculated)
            plt.show() # Explizit plt.show() aufrufen, um den Plot im Output Widget anzuzeigen

        peak_detection_button.disabled = False # Enable the peak detection button
        load_all_and_train_button.disabled = False # Enable the Load NN button

        # Initialisiere identified_peaks und filtered_peaks, wenn eine neue Datei geladen wird
        identified_peaks = {}
        filtered_peaks = {}


    except Exception as e:
        # Fehler im allgemeinen Output
        print(f"Error loading spectrum: {e}")

def peak_detection_button_clicked(b):
    global identified_peaks, filtered_peaks, wavelengths, spectrum, noise_std_calculated # noise_std_calculated hinzufügen
    # Textausgabe im allgemeinen Output
    print("Detecting peaks...")

    # clear_output(wait=True) # Dies würde den allgemeinen Output leeren, was nicht gewünscht ist

    if wavelengths is None or spectrum is None:
        print("Please load a spectrum first.")
        return

    # Parameter aus den Sliders abrufen
    s_value = s_value_slider.value
    min_distance = min_distance_slider.value
    noise_std_manual = noise_std_slider.value # Manueller Rauschwert vom Slider


    # Verwenden Sie den manuellen Rauschwert vom Slider, wenn er gesetzt ist,
    # ansonsten den automatisch berechneten Wert.
    # Hier verwenden wir immer den Wert des Sliders, da der Slider-Wert
    # initial mit dem berechneten Wert befüllt wird.
    current_noise_std = noise_std_manual if noise_std_manual > 0 else noise_std_calculated


    # Peaks finden (verwende find_peaks)
    identified_peaks_list = find_peaks(spectrum, wavelengths, noise_std=current_noise_std, s_value=s_value)


    # Peaks filtern
    # Hinweis: filter_peaks_min_distance erwartet peak_indices als Liste von Dictionaries mit 'peakpoint' und 'intensity'
    # find_peaks gibt bereits eine Liste von Dictionaries zurück.
    intensity_threshold_for_filter = 10 # Beispielwert, anpassen, um z.B. 10x Rauschen zu sein, wo min_distance ignoriert wird
    filtered_peaks_list = filter_peaks_min_distance(identified_peaks_list, min_distance, intensity_threshold_for_filter, current_noise_std)

    # Konvertiere die gefilterte Liste in ein Dictionary für identified_peaks, falls benötigt
    # Wenn identified_peaks ein Dictionary sein soll, wie in anderen Teilen des Codes verwendet
    identified_peaks = {i: peak for i, peak in enumerate(filtered_peaks_list)} # NEU: Filtern und in Dict speichern
    filtered_peaks = identified_peaks # Weise filtered_peaks ebenfalls zu (oder passe die Verwendung von filtered_peaks an)


    # Debugging-Ausgabe: Anzahl der gefundenen und gefilterten Peaks im allgemeinen Output
    print(f"Found {len(identified_peaks_list)} peaks before filtering.")
    print(f"Filtered down to {len(filtered_peaks_list)} peaks.")


    # Aktualisiere den Spektrum-Plot mit den gefilterten Peaks im output_spectrum Widget
    with output_spectrum:
        clear_output(wait=True) # Leeren Sie nur das spectrum_output Widget
        plot_spectrum(wavelengths, spectrum, peak_indices=filtered_peaks_list, noise_std=current_noise_std)
        plt.show() # Explizit plt.show() aufrufen


    # Aktualisiere den Maximalwert des peak_number_input Widgets
    peak_number_input.max = len(filtered_peaks_list) if filtered_peaks_list else 0
    peak_number_input.value = 1 if filtered_peaks_list else 0  # Setze den Wert auf 1 oder 0

    # Aktiviere/Deaktiviere die Navigationsbuttons
    if len(filtered_peaks_list) > 0:
        peak_details_button.disabled = False

    else:
        peak_details_button.disabled = True
        next_button.disabled = True
        back_button.disabled = True
        peak_number_input.disabled = True
        element_identification_button.disabled = True # Deaktiviere Element Identification Button

def clear_button_clicked(b):
    global wavelengths, spectrum, identified_peaks, filtered_peaks, bilder, positions, aktueller_index, nist_data, identified_matches, current_match_index, prepared_peaks_info, nist_peak_embeddings, nn_outputs_visible, nist_details_visible, validated_matches # NIST-bezogene globale Variablen hinzufügen und validated_matches

    with output_spectrum:
        clear_output(wait=True)
        print("Output cleared.") # Diese Meldung kann im spectrum_output bleiben oder auch verschoben werden, je nach Wunsch
    with output_nist:
        clear_output(wait=True)
    with output_peak_details:
        clear_output(wait=True)
    with details_output:
        clear_output(wait=True)
    with neur_output:
        clear_output(wait=True)
    with neur_img_output:
        clear_output(wait=True)
    with combined_peak_output:
        clear_output(wait=True)
    with nist_details_output:
         clear_output(wait=True)
    with nist_stages_output:
         clear_output(wait=True)


    # Variablen zurücksetzen
    wavelengths = None
    spectrum = None
    identified_peaks = {}
    filtered_peaks = {}
    bilder = []
    positions = []
    aktueller_index = 0
    nist_data = {} # NIST-Daten zurücksetzen
    identified_matches = [] # Identifizierte Matches zurücksetzen
    current_match_index = 0 # Aktuellen Match-Index zurücksetzen
    prepared_peaks_info = [] # Vorbereitete Peaks zurücksetzen
    nist_peak_embeddings = None # NIST-Embeddings zurücksetzen (oder [] je nach gewünschtem Initialzustand)
    nn_outputs_visible = False # NN-Outputs als nicht sichtbar markieren
    nist_details_visible = False # NIST-Details als nicht sichtbar markieren
    validated_matches = [] # Validierte Matches zurücksetzen


    # Buttons deaktivieren
    peak_detection_button.disabled = True
    nist_button.disabled = True
    peak_details_button.disabled = True
    next_button.disabled = True
    back_button.disabled = True
    peak_number_input.disabled = True
    peak_number_input.max = 0 # Max auf 0 setzen
    peak_number_input.value = 0 # Wert auf 0 setzen
    element_identification_button.disabled = True
    nist_peak_detection_button.disabled = True
    nist_details_button.disabled = True
    next_nist_button.disabled = True
    back_nist_button.disabled = True
    peak_number_nist_input.disabled = True
    peak_number_nist_input.max = 0
    peak_number_nist_input.value = 0
    next_match_button.disabled = True
    back_match_button.disabled = True
    match_number_input.disabled = True
    match_number_input.max = 0
    match_number_input.value = 0
    load_all_and_train_button.disabled = True # Load NN Button deaktivieren


def peak_details_button_clicked(b):
    global identified_peaks, filtered_peaks, peak_details_interactive_called, current_peak_index # Füge identified_peaks und filtered_peaks als global hinzu

    current_peak_index = aktueller_index  # aktueller_index ist der Index des Peaks im Detail-Output


    with output_peak_details:  # Ausgabe im output_peak_details Widget
        clear_output(wait=True)

        # Peak-Erkennung mit aktuellen Parametern erneut durchführen
        peak_indices_list = find_peaks(spectrum, wavelengths, noise_std=noise_std_slider.value)
        filtered_peaks_list = filter_peaks_min_distance(peak_indices_list, min_distance=min_distance_slider.value, intensity_threshold=10, noise_std=noise_std_slider.value)
        #identified_peaks = filtered_peaks_list # identified_peaks wird hier nur mit den gefilterten Peaks gefüllt # OLD LINE
        identified_peaks = {i: peak for i, peak in enumerate(filtered_peaks_list)} # NEU: Filtern und in Dict speichern

        # Peak-Daten für Embedding-Modell vorbereiten (wichtig für Elementidentifizierung)
        prepare_all_detected_peaks(wavelengths, spectrum, list(identified_peaks.values()), window_size_nm=10, target_size=TARGET_SIZE) # identified_peaks.values() übergeben

        peak_details_interactive(wavelengths, spectrum, list(identified_peaks.values()))  # identified_peaks.values() übergeben
        next_button.disabled = False
        back_button.disabled = False
        display(details_output)  # details_output anzeigen
        # Textfeld aktualisieren
        peak_number_input.max = len(identified_peaks) - 1  # Maximalwert auf Anzahl der Peaks - 1 setzen
        peak_number_input.value = 1
        peak_details_interactive_called = True

    with output_spectrum:  # Zugriff auf das output_spectrum Widget
        clear_output(wait=True)

        # Spektrum plotten
        fig, ax = plt.subplots(figsize=(10, 5)) # Figure und Axes erstellen
        ax.plot(wavelengths, spectrum, label="LIBS-Daten") # Label für die Spektrumdaten hinzufügen

        # Peaks markieren
        peak_wavelengths = [peak['peakpoint'] for peak in identified_peaks.values()] # .values() hinzugefügt
        peak_intensities = [spectrum[np.where(wavelengths == peak_wavelength)[0][0]] for peak_wavelength in peak_wavelengths]
        num_peaks = len(peak_wavelengths)  # Anzahl der Peaks ermitteln
        peak_scatter = ax.scatter(peak_wavelengths, peak_intensities, s=8, color="red", label=f"Peaks ({num_peaks})")  # Label anpassen
        globals()['peak_scatter'] = peak_scatter # Speichere die Scatter-Punkte in einer globalen Variable
        plt.axhline(y=noise_std_slider.value, color='red', linestyle='--', label=f"Rauschen ({noise_std_calculated:.2f})")

        # Aktuellen Peak grün markieren
        peak_wavelength = list(identified_peaks.values())[0]['peakpoint']  # Wellenlänge des ersten Peaks
        peak_intensity = spectrum[np.where(wavelengths == peak_wavelength)[0][0]]  # Intensität des ersten Peaks
        ax.scatter(peak_wavelength, peak_intensity, s=12, color="green", label=f"Peak ({1} von {num_peaks})")  # Aktuellen Peak grün markieren

        # Positionsanzeige im Spektrum integrieren
        peak_index_num = np.where(wavelengths == peak_wavelength)[0][0]  # Index des aktuellen Peaks
        window_size = 20  # Fenstergröße für die Positionsanzeige
        start_index = max(0, peak_index_num - window_size // 2)
        end_index = min(len(wavelengths), peak_index_num + window_size // 2)
        ax.axvspan(wavelengths[start_index], wavelengths[end_index], color='lightgreen', alpha=0.5) # Bereich grün markieren

        ax.legend() # Legende hier anzeigen, nachdem der Plot hinzugefügt wurde
        ax.set_title("LIBS Spectrum")
        ax.set_xlabel("Wellenlänge (nm)")
        ax.set_ylabel("Counts (a.u)")
        plt.show()
    bilder_anzeigen(0)

def next_button_clicked(b):
    global aktueller_index, bilder
    if bilder:
        aktueller_index = (aktueller_index + 1) % len(bilder)
        bilder_anzeigen(aktueller_index)

def back_button_clicked(b):
    global aktueller_index, bilder
    if bilder:
        aktueller_index = (aktueller_index - 1 + len(bilder)) % len(bilder)
        bilder_anzeigen(aktueller_index)

def jump_to_peak(change):
    global aktueller_index, bilder
    # Stelle sicher, dass der Wert gültig ist (nicht 0 und innerhalb der Grenzen)
    # Beachte: change['new'] ist der 1-basierte Wert aus dem Textfeld
    new_index = change['new'] - 1 # Konvertiere zu 0-basiertem Index

    if bilder and 0 <= new_index < len(bilder):
        aktueller_index = new_index
        bilder_anzeigen(aktueller_index)
    # Wenn der Wert außerhalb des gültigen Bereichs liegt, setzen Sie das Textfeld auf den aktuellen Wert zurück
    elif bilder and len(bilder) > 0: # Nur zurücksetzen, wenn es Peaks gibt
         peak_number_input.value = aktueller_index + 1
    else: # Wenn keine Peaks da sind, Wert auf 0 setzen
        peak_number_input.value = 0


def nist_button_clicked(b):
    global nist_data, all_elements # all_elements global machen, wenn in anderen Funktionen benötigt

    with output_nist:
        clear_output(wait=True)
        print("Loading NIST data...")

        # Elemente aus beiden Widgets abrufen
        selected_elements = list(elements_widget.value)  # Konvertiere in eine Liste
        additional_elements = [elem.strip() for elem in additional_elements_widget.value.split(',') if elem.strip()]

        # 'None' nur entfernen, wenn andere Elemente ausgewählt sind oder Text im Textfeld steht
        if selected_elements and selected_elements != [None] or additional_elements:
             if None in selected_elements:
                 selected_elements.remove(None)

        all_elements = selected_elements + additional_elements # Globale Variable setzen


        if not all_elements:
            print("Please select at least one element.")
            nist_peak_detection_button.disabled = True  # Deaktiviere den NIST-Peakerkennungsbutton
            return

        # Parameter aus den Sliders abrufen
        tolerance = tolerance_slider.value
        temp = temp_slider.value
        resolution = resolution_slider.value
        # NEU: Werte von den neuen Sliders abrufen
        low_w = low_w_slider.value
        upp_w = upp_w_slider.value
        eden_exponent = eden_slider.value # Get exponent value
        maxcharge = maxcharge_slider.value
        min_rel_int = min_rel_int_slider.value
        int_scale = int_scale_slider.value

        # NIST-Daten laden und neue Parameter übergeben
        nist_data = {'elements': all_elements, 'data': all_nist_data(all_elements, low_w, upp_w, temp, resolution, eden_exponent, maxcharge, min_rel_int, int_scale)} # Pass exponent

        # Anzahl der ausgewählten Elemente ermitteln
        num_elements = len(all_elements)

        # Bedingung hinzufügen, um Subplots nur zu erstellen, wenn Elemente ausgewählt sind
        if num_elements > 0:
            # Sum-Daten im normalen nist output anzeigen
            with output_nist:
                clear_output(wait=True)

                fig, axes = plt.subplots(1, num_elements, figsize=(5 * num_elements, 4))  # Erstelle Subplots

                if num_elements == 1:  # Sonderfall für ein Element
                    axes = [axes]  # Pack axes in eine Liste, um die Iteration zu vereinfachen

                for i, element in enumerate(all_elements):
                    element_data = nist_data['data'].get(element)
                    if element_data:
                        df_sum = element_data.get("Sum")  # Sum-Daten abrufen
                        if df_sum is not None:
                            ax = axes[i]  # Aktuelles Axes-objekt auswählen
                            ax.plot(df_sum['Wellenlänge (nm)'], df_sum['Intensität'], label=f"{element} Sum")
                            ax.set_title(f"{element} Sum")  # Titel für jedes Subplot
                            ax.set_xlabel("Wellenlänge (nm)")
                            ax.set_ylabel("Intensität")
                            ax.legend()

                            # Peaks in NIST-Daten finden und markieren (only for Sum data)
                            peak_indices_nist = find_nist_peaks(nist_data, element, "Sum")
                            # Corrected: Use df_sum['Wellenlänge (nm)'].values and df_sum['Intensität'].values for mark_nist_peaks
                            mark_nist_peaks(ax, df_sum['Wellenlänge (nm)'].values, df_sum['Intensität'].values, peak_indices_nist, "Sum")  # Add markers
                            # nist_sum_marker = ax.axvline(x=nist_peak_data[current_nist_peak_index]['peakpoint'], color='red') # Removed: nist_peak_data not defined here
                            # peak_wavelength = identified_peaks[aktueller_index]['peakpoint']  # Wavelength of the current peak # Removed: identified_peaks not defined here
                            # search_range = search_range_slider.value  # Retrieve search range from slider # Removed
                            # ax.axvspan(peak_wavelength - search_range, peak_wavelength + search_range, color='lightgreen', alpha=0.5) # Removed


                plt.tight_layout()  # Adjust spacing between subplots
                plt.show()  # Display the plot for all elements
                nist_peak_detection_button.disabled = False  # Enable the NIST peak detection button

        else:
             with output_nist:
                  clear_output(wait=True)
                  print("Please select at least one element.")
                  nist_peak_detection_button.disabled = True  # Disable the NIST peak detection button


        # Display ionization stages in the new output
        with nist_stages_output:
            clear_output(wait=True)
            for element in all_elements:
                element_data = nist_data['data'].get(element)
                if element_data:
                    for stage, df_stage in element_data.items():
                        if stage != "Sum" and not df_stage.empty:  # Only display ionization stage data
                            fig, ax = plt.subplots(figsize=(10, 5))
                            ax.plot(df_stage['Wellenlänge (nm)'], df_stage['Intensität'], label=f"{element} {stage}")
                            ax.set_title(f"{element} {stage}")
                            ax.set_xlabel("Wellenlänge (nm)")
                            ax.set_ylabel("Intensität")
                            ax.legend()

                            # Find and mark peaks in NIST data (only for ionization stage data)
                            peak_indices_nist = find_nist_peaks(nist_data, element, stage)
                            mark_nist_peaks(ax, df_stage['Wellenlänge (nm)'].values, df_stage['Intensität'].values, peak_indices_nist, stage)  # Markierungen hinzufügen
                            # Add position marker only in the assigned peak (ionization stage)
                            # if nist_peak_data and 0 <= current_nist_peak_index < len(nist_peak_data): # Check if nist_peak_data exists # Removed
                            #     current_nist_peak = nist_peak_data[current_nist_peak_index] # Removed
                            #     if current_nist_peak['element'] == element and current_nist_peak['stage'] == stage: # Removed
                            #         peak_wavelength = identified_peaks[aktueller_index]['peakpoint']  # Wavelength of the current peak # Removed
                            #         search_range = search_range_slider.value  # Retrieve search range from slider # Removed
                            #         ax.axvspan(peak_wavelength - search_range, peak_wavelength + search_range, color='lightgreen', alpha=0.5) # Removed
                            #         nist_stage_marker = ax.axvline(x=nist_peak_data[current_nist_peak_index]['peakpoint'], color='red') # Removed

                            plt.show()

def load_all_and_train_button_clicked(b):
    global nist_data, prepared_peaks_info, embedding_model, nist_peak_embeddings, all_elements_for_nn, validated_matches, nn_model_selection_widget # Globale Variablen

    with neur_output:
        clear_output(wait=True)
        print("Starting the process: Load elements & prepare NN...")

    # Define the list of ALL elements for NN training and embeddings here (ignores the widget)
    all_elements_for_nn = all_elements_for_nn
    # Add elements from additional_elements_widget (if they exist and are unique)
    additional_elements = [elem.strip() for elem in additional_elements_widget.value.split(',') if elem.strip() and elem.strip() not in all_elements_for_nn]
    all_elements_for_nn.extend(additional_elements)


    with neur_output:
        print(f"Elements for the process: {all_elements_for_nn}")

    # Determine model type and file paths based on selection
    selected_model_type = nn_model_selection_widget.value
    epochs = epochs_slider.value
    batch_size = batch_size_slider.value
    learning_rate = learning_rate_slider.value
    elements_str = "_".join(sorted(all_elements_for_nn))
    temp = temp_slider.value
    resolution = resolution_slider.value
    low_w = low_w_slider.value
    upp_w = upp_w_slider.value
    eden_exponent = eden_slider.value
    maxcharge = maxcharge_slider.value
    min_rel_int = min_rel_int_slider.value
    int_scale = int_scale_slider.value
    nist_params_str = f"lw{low_w:.1f}_uw{upp_w:.1f}_t{temp:.1f}_r{resolution}_e{eden_exponent}_mc{maxcharge}_mri{min_rel_int:.3f}_is{int_scale}".replace('.', '_')

    if selected_model_type == 'Raw NIST NN':
        model_filename = f"embedding_model_e{epochs}_bs{batch_size}_lr{learning_rate:.4f}_RawNISTNN_{nist_params_str}.keras"
        # Corrected embeddings filename format based on user's provided path
        embeddings_filename = f"nist_embeddings_all_available_elements_{nist_params_str}_RawNISTNN_e{epochs}.npy"
        # Define validated matches filename for Raw NIST NN (will likely not exist, but for consistency)
        validated_matches_filename = f"validated_matches_e{epochs}_bs{batch_size}_lr{learning_rate:.4f}_RawNISTNN_{nist_params_str}.pkl"

    elif selected_model_type == 'User fused NN':
        model_filename = f"embedding_model_e{epochs}_bs{batch_size}_lr{learning_rate:.4f}_UserFusedNN_{nist_params_str}.keras"
        # Corrected embeddings filename format based on user's provided path
        embeddings_filename = f"nist_embeddings_all_available_elements_{nist_params_str}_UserFused_e{epochs}.npy" # Adjusted to match UserFused in user's path
        # Define validated matches filename for User Fused NN
        validated_matches_filename = f"validated_matches_e{epochs}_bs{batch_size}_lr{learning_rate:.4f}_UserFusedNN_{nist_params_str}.pkl"

    else:
        with neur_output:
            print(f"Error: Unknown model selection type '{selected_model_type}'.")
        element_identification_button.disabled = True
        train_nn_button.disabled = True
        return

    model_path = os.path.join('/content/drive/MyDrive/Gemini/Gemini_Embeddings/Models/', model_filename)
    embeddings_save_path = os.path.join('/content/drive/MyDrive/Gemini/Gemini_Embeddings/Embeddings/', embeddings_filename)
    validated_matches_path = os.path.join('/content/drive/MyDrive/Gemini/Gemini_Embeddings/Validated_Matches/', validated_matches_filename) # Path for validated matches file


    # Create the storage folders if they don't exist
    model_dir = os.path.dirname(model_path)
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
        with neur_output:
             print(f"Created storage folder for model: {model_dir}")

    embeddings_dir = os.path.dirname(embeddings_save_path)
    if not os.path.exists(embeddings_dir):
        os.makedirs(embeddings_dir)
        with neur_output:
             print(f"Created storage folder for Embeddings: {embeddings_dir}")

    # Create validated matches storage folder
    validated_matches_dir = os.path.dirname(validated_matches_path)
    if not os.path.exists(validated_matches_dir):
        os.makedirs(validated_matches_dir)
        with neur_output:
             print(f"Created storage folder for Validated Matches: {validated_matches_dir}")


    # 1. Load NIST data for ALL elements (if not already loaded or incomplete)
    current_elements_in_nist_data = nist_data.get('elements', [])
    current_nist_params = nist_data.get('params', {})
    new_nist_params = {
        'low_w': low_w, 'upp_w': upp_w, 'temp': temp, 'resolution': resolution,
        'eden_exponent': eden_exponent, 'maxcharge': maxcharge,
        'min_rel_int': min_rel_int, 'int_scale': int_scale
    }

    # Using the logic from the provided old version for loading NIST data
    if not nist_data or sorted(current_elements_in_nist_data) != sorted(all_elements_for_nn) or current_nist_params != new_nist_params:
        with neur_output:
            print("Loading NIST data for the selected elements...")
        nist_data['elements'] = all_elements_for_nn
        nist_data['data'] = all_nist_data(all_elements_for_nn, low_w, upp_w, temp, resolution, eden_exponent, maxcharge, min_rel_int, int_scale)
        nist_data['params'] = new_nist_params
        if nist_data['data']:
             with neur_output:
                  print("NIST data successfully loaded.")
        else:
             print("Error loading NIST data.")
             element_identification_button.disabled = True
             train_nn_button.disabled = True
             return
    else:
         with neur_output:
              print("NIST data for the selected elements is already loaded.")


    # 2. Prepare Spectrum Peaks (if spectrum is loaded)
    if wavelengths is None or spectrum is None:
        with neur_output:
            print("Please load a spectrum first.")
        train_nn_button.disabled = True
        # We can still load the NN model and embeddings even without a spectrum loaded
        # So don't return here, proceed to model loading/training
    else:
        with neur_output:
            print("Preparing spectrum peaks for the NN...")
        window_size_nm = search_range_slider.value
        if isinstance(filtered_peaks, dict):
             prepare_all_detected_peaks(wavelengths, spectrum, list(filtered_peaks.values()), window_size_nm=window_size_nm, target_size=TARGET_SIZE)
        else:
             with neur_output:
                  print("Error: filtered_peaks is not a dictionary. Cannot prepare spectrum peaks.")
             # We can still load the NN model and embeddings
             pass # Continue execution


    # 3. Load Embedding Model
    embedding_model = None
    model_loaded_successfully = False

    # Using the loading logic from the provided old version
    # Print the model path before attempting to load
    with neur_output:
        print(f"Attempting to load existing '{selected_model_type}' Model from: {model_path}...")
    if os.path.exists(model_path):
        try:
            tf.keras.config.enable_unsafe_deserialization()
            loaded_model = tf.keras.models.load_model(model_path, custom_objects={'triplet_semi_hard_loss': triplet_semi_hard_loss})
            if hasattr(loaded_model, 'inputs') and len(loaded_model.inputs) == 1:
                embedding_model = loaded_model
                with neur_output:
                    print(f"'{selected_model_type}' Model successfully loaded.")
                model_loaded_successfully = True
            else:
                with neur_output:
                     print(f"Loaded model from {model_path} does not have the expected 1-input shape. It cannot be used as an embedding model.")
                # model_loaded_successfully remains False

        except Exception as e:
            print(f"Error loading model from {model_path}: {e}")
            print(f"Exception Type (Model Loading): {type(e).__name__}")
            with neur_output:
                 print("Model could not be loaded.")
            # model_loaded_successfully remains False

    # NEW: Load validated_matches if User Fused NN is selected and model loaded successfully
    validated_matches = [] # Initialize validated_matches list
    if selected_model_type == 'User fused NN' and model_loaded_successfully:
        with neur_output:
            print(f"Attempting to load validated matches from: {validated_matches_path}")
        if os.path.exists(validated_matches_path):
            try:
                with open(validated_matches_path, 'rb') as f:
                    validated_matches = pickle.load(f)
                with neur_output:
                    print(f"Validated matches successfully loaded. Count: {len(validated_matches)}")
            except Exception as e:
                with neur_output:
                    print(f"Error loading validated matches from {validated_matches_path}: {e}")
                    validated_matches = [] # Reset if loading fails
        else:
            with neur_output:
                print("No validated matches file found for this model version.")
            validated_matches = [] # Ensure it's an empty list if no file


    # 4. Load or Recalculate NIST Peak Embeddings
    nist_peak_embeddings = None
    embeddings_loaded_successfully = False

    # Using the loading logic from the provided old version
    # Attempt to load embeddings first, if a model was successfully loaded
    with neur_output:
         print(f"Attempting to load NIST Peak Embeddings from: {embeddings_save_path}") # Print embeddings path
    if model_loaded_successfully and os.path.exists(embeddings_save_path):
        try:
            nist_peak_embeddings = np.load(embeddings_save_path, allow_pickle=True).tolist()
            with neur_output:
                 print(f"NIST Peak Embeddings successfully loaded. Count: {len(nist_peak_embeddings)}")
            embeddings_loaded_successfully = True
        except Exception as e:
             with neur_output:
                  print(f"Error loading NIST Peak Embeddings from {embeddings_save_path}: {e}")
                  print("Embeddings will be recalculated...")
             # embeddings_loaded_successfully remains False


    # If embeddings were not loaded (either due to missing file or loading error), recalculate them
    if not embeddings_loaded_successfully and model_loaded_successfully and nist_data and nist_data.get('data') and embedding_model is not None and hasattr(embedding_model, 'inputs') and len(embedding_model.inputs) == 1: # Added model check here
         with neur_output:
             print("Calculating and storing NIST Peak Embeddings...")
         calculate_and_store_nist_embeddings(nist_data, embedding_model, window_size_nm=search_range_slider.value, target_size=TARGET_SIZE, embeddings_save_path=embeddings_save_path)
         with neur_output:
              if nist_peak_embeddings is not None and len(nist_peak_embeddings) > 0:
                  print("NIST Peak Embeddings process completed successfully.")
              else:
                  print("NIST Peak Embeddings could not be calculated.")


    # Ensure prepared_peaks_info is available to enable Element Identification Button
    # This should have been set in step 2 if a spectrum is loaded
    spectrum_peaks_prepared = prepared_peaks_info is not None and isinstance(prepared_peaks_info, dict) and len(prepared_peaks_info) > 0 # Added isinstance check


    # Enable/Disable buttons based on availability of model, embeddings, and spectrum peaks
    # Train NN button is enabled if spectrum is loaded AND NIST data is loaded AND all_elements_for_nn is not empty
    if wavelengths is not None and spectrum is not None and nist_data and nist_data.get('data') and all_elements_for_nn:
        train_nn_button.disabled = False
    else:
        train_nn_button.disabled = True

    # Element Identification button is enabled if a suitable embedding model is loaded AND NIST embeddings are available AND spectrum peaks are prepared
    if embedding_model is not None and hasattr(embedding_model, 'inputs') and len(embedding_model.inputs) == 1 and nist_peak_embeddings is not None and len(nist_peak_embeddings) > 0:
         with neur_output:
              print("Enabling Element Identification Button.")
         element_identification_button.disabled = False
    else:
         with neur_output:
              print("Conditions for enabling Element Identification Button not met after loading.") # Changed message
              print(f"  embedding_model is available and has 1 input: {embedding_model is not None and hasattr(embedding_model, 'inputs') and len(embedding_model.inputs) == 1}")
              print(f"  nist_peak_embeddings is available and has items: {nist_peak_embeddings is not None and len(nist_peak_embeddings) > 0 if nist_peak_embeddings is not None else 'N/A'}")
              print(f"  spectrum_peaks_prepared: {spectrum_peaks_prepared}")

         element_identification_button.disabled = True


def element_identification_button_clicked(b):
    global identified_peaks, prepared_peaks_info, aktueller_index # NEU: aktueller_index

    with neur_output:
        clear_output(wait=True)
        print("Starting Element Identification...")

    # Ensure peaks have been detected and prepared
    if not identified_peaks or not prepared_peaks_info:
        with neur_output:
            print("Please perform Peak Detection and NN Preparation (Load NN) first.")
        return

    # Ensure a peak is selected in the detail view
    if aktueller_index is None or not (0 <= aktueller_index < len(list(identified_peaks.values()))):
         with neur_output:
              print("Please select a peak in the detail view first.")
              return

    update_element_identification_outputs() # Call the function that performs and displays the identification

    # Set the visibility status of the NN output
    global nn_outputs_visible
    nn_outputs_visible = True


def display_current_nist_peak():
    """Zeigt die Details und den Plot des aktuell ausgewählten NIST-Peaks im nist_details_output an."""
    global nist_peak_data, current_nist_peak_index, nist_data, low_w_slider, upp_w_slider, temp_slider, resolution_slider, eden_slider, maxcharge_slider, min_rel_int_slider, int_scale_slider # Füge NIST parameter sliders hinzu

    # Create an Output widget to capture the plot
    plot_output = widgets.Output()

    with plot_output: # Display the plot within this Output widget
        clear_output(wait=True) # Clear the output within this widget

        if not nist_peak_data or not (0 <= current_nist_peak_index < len(nist_peak_data)):
            print("Kein NIST-Peak zum Anzeigen vorhanden.")
            # Create a dummy figure if no data
            fig, ax = plt.subplots(figsize=(8, 4))
            ax.text(0.5, 0.5, "Kein NIST-Peak zum Anzeigen vorhanden.", horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
            ax.set_title("NIST Details Plot")
            ax.axis('off')
            display(fig) # Use display to show the figure in the Output widget
            plt.close(fig) # Close the matplotlib figure
            return # Exit the function early if no data

        current_nist_peak = nist_peak_data[current_nist_peak_index]

        print(f"Details für NIST-Peak {current_nist_peak_index + 1}/{len(nist_peak_data)}:")
        print(f"  Element: {current_nist_peak.get('element', 'N/A')}")
        print(f"  Ionisationsstufe: {current_nist_peak.get('stage', 'N/A')}")
        print(f"  Wellenlänge: {current_nist_peak.get('peakpoint', 'N/A'):.4f} nm") # Verwende .get() und Formatierung

        # Optional: Weitere Details aus NIST-Daten hinzufügen, falls verfügbar
        # Dies würde erfordern, auf die ursprünglichen NIST-Daten (nist_data) zuzugreifen
        # und die entsprechenden Zeilen basierend auf Element, Stage und Wellenlänge zu finden.
        # Dies kann komplex sein, da die Wellenlaengen nicht exakt uebereinstimmen muessen.

        # Plotting the NIST peak
        matched_element = current_nist_peak.get('element')
        matched_stage = current_nist_peak.get('stage')
        matched_peakpoint = current_nist_peak.get('peakpoint')

        # Ensure element, stage, and peakpoint are present and NIST data is loaded
        if matched_element and matched_stage and matched_peakpoint is not None and nist_data and nist_data.get('data'):
             element_data = nist_data['data'].get(matched_element)
             if element_data and matched_stage in element_data:
                 df_stage = element_data[matched_stage]

                 fig, ax = plt.subplots(figsize=(8, 4))
                 ax.plot(df_stage['Wellenlänge (nm)'], df_stage['Intensität'], label=f"{matched_element} {matched_stage}")
                 # Set the title to include Element and Ionization Stage
                 ax.set_title(f"NIST Peak at {matched_peakpoint:.4f} nm ({matched_element} {matched_stage})")
                 ax.set_xlabel("Wellenlänge (nm)")
                 ax.set_ylabel("Intensität")
                 ax.legend()

                 # Optional: Mark the current NIST peak point in the plot
                 ax.axvline(x=matched_peakpoint, color='red', linestyle='--', label='NIST Peak')
                 ax.legend()

                 # Optional: Zoom into the area around the NIST peak
                 # REMOVED ZOOMING HERE as per user request for the link
                 # zoom_range = 5 # Example: Zoom 5 nm around the peak
                 # x_min = matched_peakpoint - zoom_range
                 # x_max = matched_peakpoint + zoom_range
                 # ax.set_xlim([x_min, x_max])

                 # Scale y-axis based on data in the full range
                 x_min = low_w_slider.value # Use full range from sliders
                 x_max = upp_w_slider.value
                 ax.set_xlim([x_min, x_max]) # Set to full range


                 df_filtered = df_stage[(df_stage['Wellenlänge (nm)'] >= x_min) & (df_stage['Wellenlänge (nm)'] <= x_max)]
                 if not df_filtered.empty:
                     y_min = df_filtered['Intensität'].min()
                     y_max = df_filtered['Intensität'].max()
                     ax.set_ylim([y_min * 0.9, y_max * 1.1]) # Added buffer

                 # --- Generate NIST URL and store in Figure metadata ---
                 # Retrieve parameters for the URL from global variables or sliders
                 # Use the FULL RANGE defined by the sliders for the link
                 low_w = low_w_slider.value
                 upp_w = upp_w_slider.value
                 temp = temp_slider.value
                 resolution = resolution_slider.value
                 eden_exponent = eden_slider.value
                 maxcharge = maxcharge_slider.value
                 min_rel_int = min_rel_int_slider.value
                 int_scale = int_scale_slider.value
                 eden = 10**eden_exponent

                 # Convert ionization stage to NIST format (I=0, II=1, III=2, etc.)
                 stage_code = matched_stage.replace('I', '0').replace('II', '1').replace('III', '2')

                 # Use the full range from the sliders for the link parameters
                 nist_link = f"https://physics.nist.gov/cgi-bin/ASD/lines1.pl?composition={matched_element}%3A100&mytext%5B%5D={matched_element}&myperc%5B%5D=100&spectra={matched_element}0-{stage_code}&low_w={low_w:.2f}&limits_type=0&upp_w={upp_w:.2f}&show_av=2&unit=1&resolution={resolution}&temp={temp}&eden={eden}&maxcharge={maxcharge}&min_rel_int={min_rel_int:.3f}&int_scale={int_scale}&libs=1"

                 # Store the link in the Figure metadata (as a custom attribute)
                 fig.nist_url = nist_link

                 display(fig) # Use display to show the figure in the Output widget
                 plt.close(fig) # Close the matplotlib figure to save memory

             else:
                 print(f"Could not find NIST data for {matched_element} {matched_stage}.")
                 fig, ax = plt.subplots(figsize=(8, 4))
                 ax.text(0.5, 0.5, "No NIST plot available", horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
                 ax.set_title("NIST Peak Plot")
                 ax.axis('off')
                 display(fig) # Use display
                 plt.close(fig) # Close figure

        else:
             print("NIST peak information is incomplete or NIST data is not loaded.")
             fig, ax = plt.subplots(figsize=(8, 4))
             ax.text(0.5, 0.5, "NIST peak information is incomplete or NIST data is not loaded.", horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
             ax.set_title("NIST Peak Plot")
             ax.axis('off')
             display(fig) # Use display
             plt.close(fig) # Close figure

        # Update the value of the peak_number_nist_input text field
        peak_number_nist_input.value = current_nist_peak_index + 1

    # Display the Output widget containing the plot in the nist_details_output area
    display(plot_output)
    # Attach the event handler to the figure's canvas.
    # The handle_nist_plot_click function needs to be modified to accept the event object.
    # This line is correct as long as handle_nist_plot_click is defined to accept the event.
    plot_output.on_mouse_down(handle_nist_plot_click)


def next_nist_button_clicked(b):
    global current_nist_peak_index, nist_peak_data
    if nist_peak_data:
        current_nist_peak_index = (current_nist_peak_index + 1) % len(nist_peak_data)
        display_current_nist_peak()
        update_nist_markers(current_nist_peak_index) # Update markers in other plots as well


def back_nist_button_clicked(b):
    global current_nist_peak_index, nist_peak_data
    if nist_peak_data:
        current_nist_peak_index = (current_nist_peak_index - 1 + len(nist_peak_data)) % len(nist_peak_data)
        display_current_nist_peak()
        update_nist_markers(current_nist_peak_index) # Update markers in other plots as well

def jump_to_nist_peak(change):
    global current_nist_peak_index, nist_peak_data
    # Ensure the value is valid (not 0 and within bounds)
    if nist_peak_data and change['new'] > 0 and change['new'] <= len(nist_peak_data):
        # The text field shows 1-based index, our list is 0-based
        new_index = change['new'] - 1
        if 0 <= new_index < len(nist_peak_data):
            current_nist_peak_index = new_index
            display_current_nist_peak()
            update_nist_markers(current_nist_peak_index) # Update markers in other plots as well
        # If the value is outside the valid range, reset the text field to the current value
        else:
             peak_number_nist_input.value = current_nist_peak_index + 1
    elif nist_peak_data is None or len(nist_peak_data) == 0: # If no NIST peaks exist
         peak_number_nist_input.value = 0

def next_match_button_clicked(b):
    global current_match_index, identified_matches
    if identified_matches:
        current_match_index = (current_match_index + 1) % len(identified_matches)
        display_current_match()
        update_match_navigation_state()

def back_match_button_clicked(b):
    global current_match_index, identified_matches
    if identified_matches:
        current_match_index = (current_match_index - 1 + len(identified_matches)) % len(identified_matches)
        display_current_match()
        update_match_navigation_state()

def jump_to_match(change):
    global current_match_index, identified_matches
    # Ensure the value is valid (not 0 and within bounds)
    if identified_matches and change['new'] > 0 and change['new'] <= len(identified_matches):
        # The text field shows 1-based index, our list is 0-based
        new_index = change['new'] - 1
        if 0 <= new_index < len(identified_matches):
            current_match_index = new_index
            display_current_match() # Call display_current_match to show the new match
            update_match_navigation_state() # Keep this if it updates button states based on index
        # If the value is outside the valid range, reset the text field to the current value
        elif identified_matches and len(identified_matches) > 0: # Only reset if there are matches
             match_number_input.value = current_match_index + 1
        else: # If no matches exist, set value to 0
            match_number_input.value = 0

# NEUE FUNKTION FÜR EXPORT BUTTON
def export_validated_matches_button_clicked(b):
    global validated_matches, wavelengths, spectrum, nist_data, search_range_slider, identified_peaks, noise_std_slider, noise_std_calculated
    # Access the potentially existing figures
    combined_peak_match_fig = globals().get('combined_peak_match_fig')
    nist_details_fig = globals().get('nist_details_fig')


    with neur_output:
        clear_output(wait=True)
        print("Starting export of validated matches...")

    if not validated_matches:
        with neur_output:
            print("No validated matches to export.")
        return

    export_dir = "/content/drive/MyDrive/Gemini/Validated_Matches"
    os.makedirs(export_dir, exist_ok=True)

    with neur_output:
        print(f"Exporting {len(validated_matches)} validated matches to '{export_dir}'...")

    for i, match_data in enumerate(validated_matches):
        try:
            # Create a new figure with a grid of subplots
            fig, axes = plt.subplots(2, 2, figsize=(20, 10))
            fig.suptitle(f"Validated Match {i+1}: Real Peak at {match_data['real_peak_wavelength']:.2f} nm -> NIST {match_data['nist_element']} {match_data['nist_stage']} at {match_data['nist_wavelength']:.2f} nm", fontsize=16)

            # --- 1. LIBS Spectrum with Position Indicator ---
            ax1 = axes[0, 0]
            ax1.plot(wavelengths, spectrum, label="LIBS-Data")
            # Ensure identified_peaks is a dictionary before iterating over values
            if isinstance(identified_peaks, dict):
                peak_wavelengths = [p['peakpoint'] for p in identified_peaks.values()]
                peak_intensities = [spectrum[np.where(wavelengths == pw)[0][0]] for pw in peak_wavelengths]
                ax1.scatter(peak_wavelengths, peak_intensities, s=8, color="red", label="All Peaks")

            current_noise_std = noise_std_slider.value if noise_std_slider.value > 0 else noise_std_calculated
            ax1.axhline(y=current_noise_std, color='red', linestyle='--', label=f"Noise ({current_noise_std:.2f})")

            real_peak_wavelength = match_data['real_peak_wavelength']
            real_peak_intensity = match_data['real_peak_intensity']
            ax1.scatter(real_peak_wavelength, real_peak_intensity, s=20, color="green", label="Validated Peak")

            # Find index of the real peak in the spectrum
            peak_index_num = np.abs(wavelengths - real_peak_wavelength).argmin()

            window_size = 20 # This is a fixed window size for the overview plot, not the search range
            # Calculate the start and end indices for the window
            start_index = max(0, peak_index_num - window_size // 2)
            end_index = min(len(wavelengths), peak_index_num + window_size // 2)
            ax1.axvspan(wavelengths[start_index], wavelengths[end_index], color='lightgreen', alpha=0.5, label="Detail Window")


            ax1.set_title("LIBS Spectrum Overview")
            ax1.set_xlabel("Wavelength (nm)")
            ax1.set_ylabel("Counts (a.u)")
            ax1.legend()


            # --- 2. Peak Details ---
            ax2 = axes[0, 1]
            peak_wavelengths_range = wavelengths[start_index:end_index]
            peak_spectrum_range = spectrum[start_index:end_index]
            ax2.plot(peak_wavelengths_range, peak_spectrum_range, label="LIBS Spectrum")
            ax2.plot(real_peak_wavelength, real_peak_intensity, "go", markersize=8, label="Validated Peak")
            ax2.set_title(f"Peak Detail at {real_peak_wavelength:.2f} nm")
            ax2.set_xlabel("Wavelength (nm)")
            ax2.set_ylabel("Counts (a.u)")
            ax2.legend()

            # --- 3. Combined Peak Match (use the existing figure if available) ---
            ax3 = axes[1, 0]
            if combined_peak_match_fig is not None and combined_peak_match_fig.axes:
                # Copy the content from the existing figure's axes to the new subplot's axes
                old_ax3 = combined_peak_match_fig.axes[0] # Assuming only one axis in the combined plot figure
                for line in old_ax3.lines:
                    ax3.plot(line.get_xdata(), line.get_ydata(), color=line.get_color(), linestyle=line.get_linestyle(), linewidth=line.get_linewidth(), label=line.get_label())
                for scatter in old_ax3.collections:
                    ax3.scatter(scatter.get_offsets()[:, 0], scatter.get_offsets()[:, 1], color=scatter.get_facecolors().tolist(), s=scatter.get_sizes().tolist(), label=scatter.get_label())
                for vline in old_ax3.lines: # Check lines again for vertical lines
                    if vline.get_xdata() and np.all(np.diff(vline.get_xdata()) == 0): # Check if it's a vertical line
                         ax3.axvline(x=vline.get_xdata()[0], color=vline.get_color(), linestyle=vline.get_linestyle(), linewidth=vline.get_linewidth(), label=vline.get_label())


                ax3.set_title(old_ax3.get_title())
                ax3.set_xlabel(old_ax3.get_xlabel())
                ax3.set_ylabel(old_ax3.get_ylabel())
                ax3.legend()
                ax3.grid(old_ax3.get_ygrid()) # Copy grid state
                ax3.set_xlim(old_ax3.get_xlim()) # Copy limits
                ax3.set_ylim(old_ax3.get_ylim()) # Copy limits

            else:
                # If the figure is not available, regenerate the plot directly in this subplot
                matched_element = match_data['nist_element']
                matched_stage = match_data['nist_stage']
                matched_peakpoint = match_data['nist_wavelength']
                element_data = nist_data['data'].get(matched_element)

                # Redraw the combined plot logic here if the global figure is not available
                # (This is the original logic from export_validated_matches_button_clicked)
                # Real peak plot
                if wavelengths is not None and spectrum is not None:
                    search_range = search_range_slider.value
                    min_wl = real_peak_wavelength - search_range
                    max_wl = real_peak_wavelength + search_range
                    idx_min = np.searchsorted(wavelengths, min_wl)
                    idx_max = np.searchsorted(wavelengths, max_wl, side='right')
                    idx_min = max(0, idx_min - 1)
                    idx_max = min(len(wavelengths), idx_max + 1)
                    if idx_max > idx_min:
                         ax3.plot(wavelengths[idx_min:idx_max], spectrum[idx_min:idx_max], label='Reales Spektrum', color='blue')
                         real_peak_index_in_range = np.abs(wavelengths[idx_min:idx_max] - real_peak_wavelength).argmin()
                         ax3.plot(wavelengths[idx_min:idx_max][real_peak_index_in_range], spectrum[idx_min:idx_max][real_peak_index_in_range], 'bo', markersize=8, label=f'Real Peak ({real_peak_wavelength:.2f} nm)')
                         ax3.axvline(x=real_peak_wavelength, color='blue', linestyle='--')

                # NIST peak plot
                if element_data and matched_stage in element_data:
                    df_stage = element_data[matched_stage]
                    if not df_stage.empty:
                         ax3.plot(df_stage['Wellenlänge (nm)'], df_stage['Intensität'], label=f"NIST {matched_element} {matched_stage}", color='red', linestyle='-')
                         nist_peak_index_in_df = np.abs(df_stage['Wellenlänge (nm)'].values - matched_peakpoint).argmin()
                         ax3.plot(df_stage['Wellenlänge (nm)'].values[nist_peak_index_in_df], df_stage['Intensität'].values[nist_peak_index_in_df], 'ro', markersize=8, label=f'NIST Match ({matched_peakpoint:.2f} nm)')
                         ax3.axvline(x=matched_peakpoint, color='red', linestyle='--')

                ax3.set_title("Combined Real Peak and NIST Match (Regenerated)") # Indicate regenerated
                ax3.set_xlabel("Wavelength (nm)")
                ax3.set_ylabel("Intensity")
                ax3.legend()
                ax3.grid(True)
                ax3.set_xlim([real_peak_wavelength - search_range_slider.value, real_peak_wavelength + search_range_slider.value])

                # Apply combined_peak_output y-axis scaling to ax3
                x_min_ax3, x_max_ax3 = ax3.get_xlim()
                combined_y_data = []
                if wavelengths is not None and spectrum is not None:
                     idx_min_vis_ax3 = np.searchsorted(wavelengths, x_min_ax3)
                     idx_max_vis_ax3 = np.searchsorted(wavelengths, x_max_ax3, side='right')
                     combined_y_data.extend(spectrum[max(0, idx_min_vis_ax3 - 1):min(len(spectrum), idx_max_vis_ax3 + 1)])
                if element_data and matched_stage in element_data:
                     df_stage = element_data[matched_stage]
                     if not df_stage.empty:
                          df_filtered_ax3 = df_stage[(df_stage['Wellenlänge (nm)'] >= x_min_ax3) & (df_stage['Wellenlänge (nm)'] <= x_max_ax3)]
                          if not df_filtered_ax3.empty:
                               combined_y_data.extend(df_filtered_ax3['Intensität'].values)

                if combined_y_data:
                     y_min_combined = min(combined_y_data)
                     y_max_combined = max(combined_y_data)
                     ax3.set_ylim([y_min_combined * 0.9, y_max_combined * 1.1])


            # --- 4. NIST Data with Peaks and Position Indicator (use the existing figure if available) ---
            ax4 = axes[1, 1]
            if nist_details_fig is not None and nist_details_fig.axes:
                # Copy the content from the existing figure's axes to the new subplot's axes
                old_ax4 = nist_details_fig.axes[0] # Assuming only one axis in the NIST details figure
                for line in old_ax4.lines:
                    ax4.plot(line.get_xdata(), line.get_ydata(), color=line.get_color(), linestyle=line.get_linestyle(), linewidth=line.get_linewidth(), label=line.get_label())
                for scatter in old_ax4.collections:
                    ax4.scatter(scatter.get_offsets()[:, 0], scatter.get_offsets()[:, 1], color=scatter.get_facecolors().tolist(), s=scatter.get_sizes().tolist(), label=scatter.get_label())
                for vline in old_ax4.lines: # Check lines again for vertical lines
                     if vline.get_xdata() and np.all(np.diff(vline.get_xdata()) == 0): # Check if it's a vertical line
                          ax4.axvline(x=vline.get_xdata()[0], color=vline.get_color(), linestyle=vline.get_linestyle(), linewidth=vline.get_linewidth(), label=vline.get_label())
                for collection in old_ax4.collections: # Check collections for spans (rectangles)
                     if isinstance(collection, matplotlib.collections.PolyCollection): # Check if it's a PolyCollection (often used for axvspan)
                          for path in collection.get_paths():
                               patch = matplotlib.patches.PathPatch(path, color=collection.get_facecolors()[0], alpha=collection.get_alpha(), label=collection.get_label())
                               ax4.add_patch(patch)


                ax4.set_title(old_ax4.get_title())
                ax4.set_xlabel(old_ax4.get_xlabel())
                ax4.set_ylabel(old_ax4.get_ylabel())
                ax4.legend()
                ax4.grid(old_ax4.get_ygrid()) # Copy grid state
                ax4.set_xlim(old_ax4.get_xlim()) # Copy limits
                ax4.set_ylim(old_ax4.get_ylim()) # Copy limits

            else:
                # If the figure is not available, regenerate the plot directly in this subplot
                matched_element = match_data['nist_element']
                matched_stage = match_data['nist_stage']
                matched_peakpoint = match_data['nist_wavelength']
                element_data = nist_data['data'].get(matched_element)

                # Redraw the NIST details plot logic here if the global figure is not available
                # (This is the original logic from export_validated_matches_button_clicked)
                if element_data and matched_stage in element_data:
                     df_stage = element_data[matched_stage]
                     ax4.plot(df_stage['Wellenlänge (nm)'], df_stage['Intensität'], label=f"NIST {matched_element} {matched_stage}")

                     nist_peaks_in_full_range = find_peaks(df_stage['Intensität'].values, df_stage['Wellenlänge (nm)'].values, nist_data=nist_data, element=matched_element, ionization_stage=matched_stage)
                     nist_peak_wavelengths = [p['peakpoint'] for p in nist_peaks_in_full_range]
                     # Ensure the index exists before accessing
                     nist_peak_intensities = [df_stage['Intensität'].values[np.where(df_stage['Wellenlänge (nm)'].values == pw)[0][0]] if np.where(df_stage['Wellenlänge (nm)'].values == pw)[0].size > 0 else 0 for pw in nist_peak_wavelengths]


                     ax4.plot(nist_peak_wavelengths, nist_peak_intensities, 'ro', markersize=4, label='All NIST Peaks')
                     # Ensure the index exists before accessing
                     matched_peak_index_in_df_export = np.abs(df_stage['Wellenlänge (nm)'].values - matched_peakpoint).argmin()
                     ax4.plot(df_stage['Wellenlänge (nm)'].values[matched_peak_index_in_df_export], df_stage['Intensität'].values[matched_peak_index_in_df_export], 'go', markersize=4, label='Assigned NIST-Peak')
                     ax4.axvline(x=matched_peakpoint, color='green', linestyle='--', label='Assigned NIST-Peak')
                     ax4.axvspan(real_peak_wavelength - search_range_slider.value, real_peak_wavelength + search_range_slider.value, color='lightgreen', alpha=0.3, label='Peak Search Area')

                ax4.set_title(f"NIST Data for {matched_element} {matched_stage} (Regenerated)") # Indicate regenerated
                ax4.set_xlabel("Wavelength (nm)")
                ax4.set_ylabel("Intensity")
                ax4.legend()
                ax4.grid(True)


            # Speichern und Schließen
            plt.tight_layout(rect=[0, 0.03, 1, 0.95])
            sanitized_element = match_data.get('nist_element', 'Unknown').replace(" ", "_").replace(".", "_") # Sanitize element name
            sanitized_stage = match_data.get('nist_stage', 'Unknown').replace(" ", "_").replace(".", "_") # Sanitize stage name
            filename = f"match_{i+1}_{sanitized_element}_{sanitized_stage}_{real_peak_wavelength:.2f}nm.png".replace(".", "_") # Sanitize filename
            filepath = os.path.join(export_dir, filename)
            fig.savefig(filepath)
            plt.close(fig)

            with neur_output:
                print(f"Exported '{filename}'")

        except Exception as e:
            with neur_output:
                print(f"Error exporting match {i+1}: {e}")
                # Optional: Print traceback for more detailed error
                # import traceback
                # traceback.print_exc(file=output_widget.stream)


    with neur_output:
        print("Export process completed.")


# NEUE FUNKTION: Event-Handler für den add_validated_to_training_button

def add_validated_to_training_button_clicked(b):
    global validated_matches, nist_data, embedding_model, nist_peak_embeddings, all_elements_for_nn, nn_model_selection_widget

    with neur_output:
        clear_output(wait=True)
        print("Adding validated matches to training data and retraining the model...")

    if not validated_matches:
        with neur_output:
            print("No validated matches available to add to training data. Proceeding with training on NIST data only (if available).")
        # We still proceed to train even if no validated matches are available,
        # as the user might want to train a base Raw NIST NN model this way initially.
        # The function will primarily use NIST triplets if validated_matches is empty.
        pass


    # Define the list of ALL elements for NN training and embeddings here (using the global list)
    if 'all_elements_for_nn' not in globals() or not all_elements_for_nn:
         with neur_output:
              print("Error: Global 'all_elements_for_nn' list is not defined or is empty. Cannot proceed with training.")
              element_identification_button.disabled = True
              train_nn_button.disabled = True
              return

    with neur_output:
        print(f"Elements for training process: {all_elements_for_nn}")


    # 1. Load NIST data for ALL elements (if not already loaded or incomplete)
    temp = temp_slider.value
    resolution = resolution_slider.value
    low_w = low_w_slider.value
    upp_w = upp_w_slider.value
    eden_exponent = eden_slider.value
    maxcharge = maxcharge_slider.value
    min_rel_int = min_rel_int_slider.value
    int_scale = int_scale_slider.value

    current_elements_in_nist_data = nist_data.get('elements', [])
    current_nist_params = nist_data.get('params', {})
    new_nist_params = {
        'low_w': low_w, 'upp_w': upp_w, 'temp': temp, 'resolution': resolution,
        'eden_exponent': eden_exponent, 'maxcharge': maxcharge,
        'min_rel_int': min_rel_int, 'int_scale': int_scale
    }

    # Only load NIST data if needed (elements or parameters have changed)
    if not nist_data or sorted(current_elements_in_nist_data) != sorted(all_elements_for_nn) or current_nist_params != new_nist_params:
        if not all_elements_for_nn:
             with neur_output:
                  print("No elements defined in all_elements_for_nn for NIST data loading.")
             nist_data = {'elements': [], 'data': {}, 'params': {}} # Clear NIST data if no elements are defined
             element_identification_button.disabled = True
             train_nn_button.disabled = True
             return # Cannot train without NIST data

        with neur_output:
            print("Loading NIST data for training...")
        nist_data['elements'] = all_elements_for_nn
        nist_data['data'] = all_nist_data(all_elements_for_nn, low_w, upp_w, temp, resolution, eden_exponent, maxcharge, min_rel_int, int_scale)
        nist_data['params'] = new_nist_params
        if nist_data['data']:
             with neur_output:
                  print("NIST data successfully loaded.")
        else:
             print("Error loading NIST data. Retraining aborted.")
             element_identification_button.disabled = True
             train_nn_button.disabled = True
             return
    else:
         with neur_output:
              print("NIST data for training is already loaded.")


    # 2. Create Training Triplets from NIST data (always include NIST data if available)
    with neur_output:
        print("Creating training triplets from NIST data...")
    window_size_nm = search_range_slider.value
    anker_data_nist, positive_data_nist, negative_data_nist = create_triplets_from_nist(nist_data, window_size_nm=window_size_nm, target_size=TARGET_SIZE)

    if anker_data_nist is None or len(anker_data_nist) == 0:
        with neur_output:
            print("Could not create training triplets from NIST data. Retraining aborted.")
        element_identification_button.disabled = True
        return

    # 3. Create Training Triplets from Validated Matches
    anker_data_validated = None
    positive_data_validated = None
    negative_data_validated = None

    if validated_matches: # Only create validated triplets if there are validated matches
        with neur_output:
            print(f"Creating training triplets from {len(validated_matches)} validated matches...")
        # Ensure NIST data is available for creating validated triplets
        if nist_data and nist_data.get('data'):
            anker_data_validated, positive_data_validated, negative_data_validated = convert_validated_matches_to_triplets(
                validated_matches,
                nist_data,
                window_size_nm=window_size_nm,
                target_size=TARGET_SIZE
            )
        else:
            with neur_output:
                print("NIST data not available. Cannot create training triplets from validated matches.")


    # Combine NIST and Validated Triplets
    anker_data_combined = [anker_data_nist]
    positive_data_combined = [positive_data_nist]
    negative_data_combined = [negative_data_nist]

    if anker_data_validated is not None and len(anker_data_validated) > 0: # Only add validated triplets if they were successfully created and are not empty
        anker_data_combined.append(anker_data_validated)
        positive_data_combined.append(positive_data_validated)
        negative_data_combined.append(negative_data_validated)
        with neur_output:
             print(f"Added {len(anker_data_validated)} triplets from validated matches.")


    anker_data_final = np.concatenate(anker_data_combined, axis=0)
    positive_data_final = np.concatenate(positive_data_combined, axis=0)
    negative_data_final = np.concatenate(negative_data_combined, axis=0)

    with neur_output:
         print(f"Total training triplets: {anker_data_final.shape[0]}")

    if anker_data_final.shape[0] == 0:
         with neur_output:
              print("No training data available after combining NIST and validated matches. Retraining aborted.")
         element_identification_button.disabled = True
         return


    # Create dummy validation data (split only if training data exists)
    split_index = int(anker_data_final.shape[0] * 0.8)
    # Ensure split_index is valid
    if split_index == 0 and anker_data_final.shape[0] > 0: # Handle case with very few triplets
         split_index = anker_data_final.shape[0] # Use all for training, no validation data
         with neur_output:
              print("Warning: Limited training data, no separate validation set will be used.")
    elif split_index == 0 and anker_data_final.shape[0] == 0: # Should be caught by the previous check
         with neur_output:
              print("Error: No training data to split.")
         element_identification_button.disabled = True
         return


    anker_train, anker_val = anker_data_final[:split_index], anker_data_final[split_index:]
    positive_train, positive_val = positive_data_final[:split_index], positive_data_final[split_index:]
    negative_train, negative_val = negative_data_final[:split_index], negative_data_final[split_index:]

    X_val = [anker_val, positive_val, negative_val]
    y_val = np.zeros((anker_val.shape[0], 1))


    # 4. Build or Load Embedding Model (always build a new one when adding validated matches)
    # Based on the user's request to always improve, building a new model and training
    # on the combined dataset is more robust than incremental training on a loaded model
    # with potentially different numbers of validated matches added previously.
    input_shape = (TARGET_SIZE, 1) # Ensure input_shape is defined
    embedding_model = build_embedding_model(input_shape, EMBEDDING_DIMENSION)
    with neur_output:
         print("A new Embedding model built for training with validated matches.")

    # 5. Train the model with the combined data
    with neur_output:
        print("Starting training/retraining...")

    # Calculate retraining epochs based on validated matches
    # Use a minimum of 10 epochs even if no matches are validated
    retraining_epochs = max(10, len(validated_matches) * 10) # 10 epochs per validated match, minimum 10

    with neur_output:
        print(f"Using {retraining_epochs} epochs for retraining based on {len(validated_matches)} validated matches.")

    # Check if validation data exists before passing it
    if anker_val.shape[0] > 0:
        train_embedding_model(
            embedding_model,
            anker_train, positive_train, negative_train,
            X_val, y_val,
            epochs=retraining_epochs, # Use the calculated retraining_epochs value
            batch_size=batch_size_slider.value, # Use slider value for batch size
            learning_rate=learning_rate_slider.value, # Use slider value for learning rate
        )
    else:
         train_embedding_model(
            embedding_model,
            anker_train, positive_train, negative_train,
            epochs=retraining_epochs, # Use the calculated retraining_epochs value
            batch_size=batch_size_slider.value, # Use slider value for batch size
            learning_rate=learning_rate_slider.value, # Use slider value for learning rate
            validation_data=None # Explicitly set validation_data to None
        )


    with neur_output:
        print("Training/Retraining completed.")

    # 6. Save the (re)trained model as User Fused NN
    # Use the filename format that includes epochs from the slider for saving (matching load function)
    selected_model_type_save = 'User fused NN' # Always save as User Fused NN after this process
    epochs_save = epochs_slider.value # Use epochs from slider for filename
    batch_size_save = batch_size_slider.value # Use slider value for filename
    learning_rate_save = learning_rate_slider.value # Use slider value for filename
    nist_params_str_save = f"lw{low_w:.1f}_uw{upp_w:.1f}_t{temp:.1f}_r{resolution}_e{eden_exponent}_mc{maxcharge}_mri{min_rel_int:.3f}_is{int_scale}".replace('.', '_') # Ensure nist_params_str is defined


    model_filename_save = f"embedding_model_e{epochs_save}_bs{batch_size_save}_lr{learning_rate_save:.4f}_{selected_model_type_save.replace(' ', '')}_{nist_params_str_save}.keras"
    model_path_save = os.path.join('/content/drive/MyDrive/Gemini/Gemini_Embeddings/Models/', model_filename_save)


    if embedding_model is not None:
        with neur_output:
            print(f"Attempting to save retrained model to: {model_path_save}")
        try:
            embedding_model.save(model_path_save)
            with neur_output:
                print(f"Retrained model successfully saved at: {model_path_save}")
            # After successfully saving the User Fused NN, set the radio button selection to it
            nn_model_selection_widget.value = 'User fused NN'
            with neur_output:
                 print("NN Model selection updated to 'User fused NN'.")

             # Save validated_matches after successful model save
            # Use the same filename format as load function for validated matches
            validated_matches_filename_save = f"validated_matches_e{epochs_save}_bs{batch_size_save}_lr{learning_rate_save:.4f}_{selected_model_type_save.replace(' ', '')}_{nist_params_str_save}.pkl"
            validated_matches_save_path = os.path.join('/content/drive/MyDrive/Gemini/Gemini_Embeddings/Validated_Matches/', validated_matches_filename_save)
            validated_matches_dir = os.path.dirname(validated_matches_save_path)
            if not os.path.exists(validated_matches_dir):
                os.makedirs(validated_matches_dir)
                with neur_output:
                     print(f"Created storage folder for Validated Matches: {validated_matches_dir}")

            with neur_output:
                 print(f"Attempting to save validated matches to: {validated_matches_save_path}")
            with open(validated_matches_save_path, 'wb') as f:
                pickle.dump(validated_matches, f)
            with neur_output:
                 print(f"Validated matches successfully saved at: {validated_matches_save_path}")


        except Exception as save_e:
             with neur_output:
                  print(f"Error saving the retrained model or validated matches to {model_path_save} or {validated_matches_save_path}: {save_e}")
                  element_identification_button.disabled = True
                  # return # Removed return here to allow recalculation of embeddings even if model save fails
    else:
        with neur_output:
            print("embedding_model is None after training. Model and validated matches will not be saved.")


    # 7. Recalculate and store NIST Peak Embeddings (based on the retrained model and potentially updated NIST data)
    # Recalculate embeddings using the NEWLY TRAINED model (User Fused NN)
    # Use the embeddings filename format that matches the load function (corrected format)
    embeddings_filename_retrained = f"nist_embeddings_all_available_elements_{nist_params_str_save}_{selected_model_type_save.replace(' ', '')}_e{epochs_save}.npy" # Corrected filename format for saving
    embeddings_save_path_retrained = os.path.join('/content/drive/MyDrive/Gemini/Gemini_Embeddings/Embeddings/', embeddings_filename_retrained)


    embeddings_dir = os.path.dirname(embeddings_save_path_retrained)
    if not os.path.exists(embeddings_dir):
        os.makedirs(embeddings_dir)
        with neur_output:
             print(f"Created storage folder for Embeddings: {embeddings_dir}")

    nist_peak_embeddings = None

    # Only recalculate embeddings if the model is available (either loaded or newly trained/retrained)
    if embedding_model is not None and hasattr(embedding_model, 'inputs') and len(embedding_model.inputs) == 1 and nist_data and nist_data.get('data'):
        with neur_output:
            print("Calculating and storing NIST Peak Embeddings after retraining...")
        calculate_and_store_nist_embeddings(nist_data, embedding_model, window_size_nm=search_range_slider.value, target_size=TARGET_SIZE, embeddings_save_path=embeddings_save_path_retrained)
        with neur_output:
              if nist_peak_embeddings is not None and len(nist_peak_embeddings) > 0:
                  print("NIST Peak Embeddings process completed successfully.")
              else:
                  print("NIST Peak Embeddings could not be calculated.")
    else:
         with neur_output:
              print("Model or NIST data not available after retraining. NIST Peak Embeddings will not be recalculated.")
         nist_peak_embeddings = None


    # Ensure prepared_peaks_info is available to enable Element Identification Button
    # This should have been set in step 2 if a spectrum is loaded
    spectrum_peaks_prepared = prepared_peaks_info is not None and isinstance(prepared_peaks_info, dict) and len(prepared_peaks_info) > 0


    # Enable/Disable buttons based on availability of model, embeddings, and spectrum peaks
    # Train NN button is enabled if spectrum is loaded AND NIST data is loaded AND all_elements_for_nn is not empty
    if wavelengths is not None and spectrum is not None and nist_data and nist_data.get('data') and all_elements_for_nn:
        train_nn_button.disabled = False
    else:
        train_nn_button.disabled = True

    # Element Identification button is enabled if a suitable embedding model is loaded AND NIST embeddings are available AND spectrum peaks are prepared
    if embedding_model is not None and hasattr(embedding_model, 'inputs') and len(embedding_model.inputs) == 1 and nist_peak_embeddings is not None and len(nist_peak_embeddings) > 0:
         with neur_output:
              print("Enabling Element Identification Button.")
         element_identification_button.disabled = False
    else:
         with neur_output:
              print("Conditions for enabling Element Identification Button not met after retraining.") # Changed message
              print(f"  embedding_model is available and has 1 input: {embedding_model is not None and hasattr(embedding_model, 'inputs') and len(embedding_model.inputs) == 1}")
              print(f"  nist_peak_embeddings is available and has items: {nist_peak_embeddings is not None and len(nist_peak_embeddings) > 0 if nist_peak_embeddings is not None else 'N/A'}")

         element_identification_button.disabled = True


In [13]:
# @title Event-Handler, Widget-Connections

apply_button = widgets.Button(description="Apply")


next_match_button = widgets.Button(description="Next Match", disabled=True)
back_match_button = widgets.Button(description="Previous Match", disabled=True)
match_number_input = widgets.BoundedIntText(
    value=0,
    min=0,
    max=0,
    step=1,
    disabled=True, # Initial deaktiviert
    layout=widgets.Layout(width='50px')
)

match_navigation_box = widgets.HBox([back_match_button, match_number_input, next_match_button])

# Buttons erstellen (initial deaktiviert)
next_button = widgets.Button(description="Next Peak", disabled=True)
back_button = widgets.Button(description="Previous Peak", disabled=True) # Fehler behoben: disabled=True statt disabled="disabled"
peak_number_input = widgets.BoundedIntText(
    value=0,  # Startwert auf 0 setzen
    min=0,  # Minimalwert auf 0 setzen
    max=0,  # Initialer Maximalwert auf 0 setzen
    step=1,
    #description='Index:',  # Beschreibung entfernt
    disabled=False,
    layout=widgets.Layout(width='50px')  # Breite auf 50px festgelegt
)

# Add new button for adding validated matches to training data
add_validated_to_training_button = widgets.Button(description="Add Validated Matches to Training Data", disabled=True)

#Event-Handler zuweisen
file_button.on_click(file_button_clicked)
nist_button.on_click(nist_button_clicked)
train_nn_button.on_click(train_nn_button_clicked)
peak_detection_button.on_click(peak_detection_button_clicked)
drive_selection_widget.observe(drive_selection_changed, names='value')
peak_details_button.on_click(peak_details_button_clicked)  # Event Handler für den "Peak Details" Button
next_button.on_click(next_button_clicked)  # Event Handler für den "Next" Button
back_button.on_click(back_button_clicked)  # Event Handler für den "Back" Button
#nist_peak_detection_button.on_click(nist_peak_detection_button_clicked)
element_identification_button.on_click(element_identification_button_clicked)
load_all_and_train_button.on_click(load_all_and_train_button_clicked)
next_match_button.on_click(next_match_button_clicked)
back_match_button.on_click(back_match_button_clicked)
mark_match_button.on_click(mark_match_button_clicked)
match_number_input.observe(jump_to_match, names='value')
# Event-Handler zuweisen
next_nist_button.on_click(next_nist_button_clicked)
back_nist_button.on_click(back_nist_button_clicked)
peak_number_nist_input.observe(jump_to_nist_peak, names='value')

# NEUER EVENT-HANDLER FÜR EXPORT BUTTON
export_matches_button.on_click(export_validated_matches_button_clicked) # Event Handler hinzufügen

# Explizite Zuweisung des Event-Handlers für den neuen Button
add_validated_to_training_button.on_click(add_validated_to_training_button_clicked)


# Beobachter für das Textfeld hinzufügen
peak_number_input.observe(jump_to_peak, names='value')
# Layout anpassen
# Layout anpassen
peak_detection_group = widgets.VBox([
    widgets.HBox([noise_std_slider, noise_std_text]),
    widgets.HBox([s_value_slider, s_value_text]),
    widgets.HBox([min_distance_slider, min_distance_text]),
    peak_detection_button,
    peak_details_button,
    widgets.HBox([back_button, peak_number_input, next_button]),
])


nist_group = widgets.VBox([
    #nist_button,
    widgets.HBox([tolerance_slider, tolerance_text]),
    widgets.HBox([temp_slider, temp_text]),
    widgets.HBox([resolution_slider, resolution_text]),
    #nist_peak_detection_button,
    #nist_details_button,

    #widgets.HBox([back_nist_button, peak_number_nist_input, next_nist_button])  # Navigations-Widgets für NIST-Details
])

training_widgets = widgets.VBox([
    train_nn_button,
    train_elements_selection,
    widgets.HBox([epochs_slider, epochs_text]),
    widgets.HBox([batch_size_slider, batch_size_text]),
    widgets.HBox([learning_rate_slider, learning_rate_text]),
])


# Moved jslink calls to cell_id: huV7dEJygnTu
# widgets.jslink((noise_std_slider, 'value'), (noise_std_text, 'value'));
# widgets.jslink((min_distance_slider, 'value'), (min_distance_text, 'value'));
# widgets.jslink((s_value_slider, 'value'), (s_value_text, 'value'));
# widgets.jslink((tolerance_slider, 'value'), (tolerance_text, 'value'));
# widgets.jslink((temp_slider, 'value'), (temp_text, 'value'));
# widgets.jslink((resolution_slider, 'value'), (resolution_text, 'value'));
# widgets.jslink((search_range_slider, 'value'), (search_range_text, 'value'));
# widgets.jslink((epochs_slider, 'value'), (epochs_text, 'value'));
# widgets.jslink((batch_size_slider, 'value'), (batch_size_text, 'value'));
# widgets.jslink((learning_rate_slider, 'value'), (learning_rate_text, 'value'));

In [14]:
# @title Display

display(widgets.HBox([
    widgets.VBox([  # Menü und Steuerelemente
        #drive_selection_widget,

        file_button,
        file_path_widget,
        file_upload_widget,
        peak_detection_group,
        #elements_widget, # Entferne das Elementauswahl-Widget
        #additional_elements_widget, # Entferne das Textfeld für zusätzliche Elemente
        element_identification_button,
        widgets.HBox([search_range_slider, search_range_text]),
        widgets.HBox([back_match_button, match_number_input, next_match_button]),
        mark_match_button,
        export_matches_button, # Export Button hinzufügen
        add_validated_to_training_button, # Add the new button to the display
        load_all_and_train_button,
        nn_model_selection_widget, # Füge das neue Modellauswahl-Widget hinzu


        #training_widgets # Entferne die Trainings-Widgets Gruppe, die train_elements_selection enthielt
        widgets.VBox([ # Füge die restlichen Trainings-Widgets wieder hinzu
            train_nn_button,
            widgets.HBox([epochs_slider, epochs_text]),
            widgets.HBox([batch_size_slider, batch_size_text]),
            widgets.HBox([learning_rate_slider, learning_rate_text]),
        ]),
        # NEUE NIST PARAMETER SLIDER HINZUFÜGEN
        widgets.Label("NIST Parameters:"),
        widgets.HBox([low_w_slider, low_w_text]),
        widgets.HBox([upp_w_slider, upp_w_text]),
        widgets.HBox([eden_slider, eden_text]),
        widgets.HBox([maxcharge_slider, maxcharge_text]),
        widgets.HBox([min_rel_int_slider, min_rel_int_text]),
        widgets.HBox([int_scale_slider, int_scale_text]),
        nist_group,
    ]),
    widgets.VBox([  # Spektrum, Position, NIST-Daten und Detailansicht
        widgets.HBox([  # HBox für Spektrum und Position
            output_spectrum,
            output_peak_details,
            #output_nist
            #nist_stages_output
        ]),
          # NIST-Daten unter dem Spektrum
        widgets.HBox([#neur_img_output,
                      neur_output,
                      combined_peak_output,
                      nist_details_output])  # Detailansicht unter den NIST-Daten
        #neur_output
    ])
]))

# Add jslink calls here, after the display() call
widgets.jslink((noise_std_slider, 'value'), (noise_std_text, 'value'));
widgets.jslink((min_distance_slider, 'value'), (min_distance_text, 'value'));
widgets.jslink((s_value_slider, 'value'), (s_value_text, 'value'));
widgets.jslink((tolerance_slider, 'value'), (tolerance_text, 'value'));
widgets.jslink((temp_slider, 'value'), (temp_text, 'value'));
widgets.jslink((resolution_slider, 'value'), (resolution_text, 'value'));
widgets.jslink((search_range_slider, 'value'), (search_range_text, 'value'));
widgets.jslink((epochs_slider, 'value'), (epochs_text, 'value'));
widgets.jslink((batch_size_slider, 'value'), (batch_size_text, 'value'));
widgets.jslink((learning_rate_slider, 'value'), (learning_rate_text, 'value'));
# NEUE JSLINK CALLS FÜR NIST URL PARAMETER
widgets.jslink((low_w_slider, 'value'), (low_w_text, 'value'));
widgets.jslink((upp_w_slider, 'value'), (upp_w_text, 'value'));
widgets.jslink((eden_slider, 'value'), (eden_text, 'value'));
widgets.jslink((maxcharge_slider, 'value'), (maxcharge_text, 'value'));
widgets.jslink((min_rel_int_slider, 'value'), (min_rel_int_text, 'value'));
widgets.jslink((int_scale_slider, 'value'), (int_scale_text, 'value'));

Loading spectrum...
Google Drive wird eingebunden
Mounted at /content/drive
